In [1]:
val path = "/chisel-bootcamp" + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

Compiling /workspace/Main.sc

Checking https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/chisel-iotesters_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/chisel-iotesters_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/chiseltest_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/chiseltest_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/dsptools_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/dsptools_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/firrtl-diagrammer_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/firrtl-diagrammer_2.12/maven-metadata.xml


Compiling /workspace/Main.sc #2

path: String = "/chisel-bootcamp/source/load-ivy.sc"

In [2]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

import chisel3._
import chisel3.util._
import chisel3.util.Fill
import chisel3.util.PriorityEncoder
import chisel3.experimental.ChiselEnum
import chisel3.util.experimental.loadMemoryFromFileInline

import scala.math
import scala.io.Source

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test


import chisel3._

import chisel3.util._

import chisel3.util.Fill

import chisel3.util.PriorityEncoder

import chisel3.experimental.ChiselEnum

import chisel3.util.experimental.loadMemoryFromFileInline


import scala.math

import scala.io.Source

## Header

In [3]:
class input_t extends Bundle {
    val empty = UInt(32.W)
    val data = UInt(256.W)
}
class output_t extends Bundle {
    val empty = UInt(12.W)
    val data = UInt(96.W)
}

defined class input_t
defined class output_t

## Gorilla

In [4]:
import chisel3._
import chisel3.util._

import scala.collection.mutable.ArrayBuffer
import scala.collection.mutable.HashMap

import chisel3._

import chisel3.util._


import scala.collection.mutable.ArrayBuffer

import scala.collection.mutable.HashMap

In [5]:
object compilerControl {
  val pcEnable = true
}

defined object compilerControl

In [6]:
import chisel3._
import chisel3.util._

import scala.collection.mutable.HashMap
import scala.collection.mutable.ArrayBuffer
import scala.collection.immutable.ListMap
import scala.util._

class DynamicBundle extends Bundle {
  private var _data = ArrayBuffer[(String, Data)]()

  private def modifyField(obj: AnyRef, name: String, value: Any) {
    def impl(clazz: Class[_] ) {
      Try(clazz.getDeclaredField(name)).toOption match {
        case Some(field) =>
          field.setAccessible(true)
          clazz.getMethod(name).invoke(obj) // force init
          field.set(obj, value)
        case None =>
          if (clazz.getSuperclass != null) {
            impl(clazz.getSuperclass)
          }
      }
    }
    impl(obj.getClass)
  }

  override def cloneType = DynamicBundle(_data).asInstanceOf[this.type]
}

object DynamicBundle {
  def apply(data: ArrayBuffer[(String, Data)] = ArrayBuffer()): DynamicBundle = {
    val dynamicBundle = new DynamicBundle
    dynamicBundle._data = data
    val new_elements = dynamicBundle.elements ++ data.foldLeft(ListMap[String, Data]())((a, b) => a + (b._1 -> b._2))
    dynamicBundle.modifyField(dynamicBundle, "elements", new_elements)
    dynamicBundle
  }
}

object gArbiterCtrl {
  def apply(request: Seq[Bool]): Seq[Bool] = request.length match {
    case 0 => Seq()
    case 1 => Seq(true.B)
    case _ => true.B +: request.tail.init.scanLeft(request.head)(_ || _).map(!_)
  }
}

class gioArbiter[T <: Data](n: Int, data: => T) extends Bundle {
  val out = new gFIFOIO(data)
  val in = Flipped(Vec(n, new gFIFOIO(data)))
  val chosen = Output(UInt((log2Up(n)).W))
}

class gioDistributor[T <: Data](n: Int, data: => T) extends Bundle {
  val out = Vec(n, new gFIFOIO(data))
  val in = Flipped(new gFIFOIO(data))
  val chosen = Output(UInt((log2Up(n)).W))
}

class RREncode(n: Int) extends Module {
  val io = IO(new Bundle {
    val valid = Input(Vec(n, Bool()))
    val chosen = Output(UInt((log2Up(n+1)).W))
    val ready = Input(Bool())
  })

  val last_grant = RegInit((0).U((log2Up(n)).W))
  val g = gArbiterCtrl((0 until n).map(i => io.valid(i) && (i).U > last_grant) ++ io.valid)
  val grant = (0 until n).map(i => g(i) && (i).U > last_grant || g(i+n))

  var choose = (n).U
  for (i <- n-1 to 0 by -1)
    choose = Mux(io.valid(i), (i).U, choose)
  for (i <- n-1 to 1 by -1)
    choose = Mux(io.valid(i) && (i).U > last_grant, (i).U, choose)
  val outValid = io.valid.foldLeft(false.B)( _ || _)
  when (outValid && io.ready) {
    last_grant := choose
  }
  io.chosen := choose
}

class gRRArbiter[T <: Data](n: Int, data: => T) extends Module with TagTrait {
  val io = IO(new gioArbiter(n, data))

  val last_grant = RegInit((0).U((log2Up(n)).W))
  val g = gArbiterCtrl((0 until n).map(i => io.in(i).valid && (i).U > last_grant) ++ io.in.map(_.valid))
  val grant = (0 until n).map(i => g(i) && (i).U > last_grant || g(i+n))
  (0 until n).map(i => io.in(i).ready := grant(i) && io.out.ready)

  var choose = (n-1).U
  for (i <- n-2 to 0 by -1)
    choose = Mux(io.in(i).valid, (i).U, choose)
  for (i <- n-1 to 1 by -1)
    choose = Mux(io.in(i).valid && (i).U > last_grant, (i).U, choose)
  when (io.out.valid && io.out.ready) {
    last_grant := choose
  }

  val dvec = Vec(n, data)
  val tvec = Vec(n, UInt((TAGWIDTH).W))
  (0 until n).map(i => dvec(i) := io.in(i).bits )
  (0 until n).map(i => tvec(i) := io.in(i).tag )

  io.out.valid := io.in.map(_.valid).foldLeft(false.B)( _ || _)
  io.out.bits := dvec(choose)
  io.out.tag := tvec(choose)
  io.chosen := choose
}

class gRRDistributor[T <: Data](n: Int, data: => T) extends Module {
  val io = IO(new gioDistributor(n, data))

  val last_grant = RegInit((0).U((log2Up(n)).W))
  val g = gArbiterCtrl((0 until n).map(i => io.out(i).ready && (i).U > last_grant) ++ io.out.map(_.ready))
  val grant = (0 until n).map(i => g(i) && (i).U > last_grant || g(i+n))
  (0 until n).map(i => io.out(i).valid := grant(i) && io.in.valid)
  //name_it()
  var choose = (n-1).U
  for (i <- n-2 to 0 by -1)
    choose = Mux(io.out(i).ready, (i).U, choose)
  for (i <- n-1 to 1 by -1)
    choose = Mux(io.out(i).ready && (i).U > last_grant, (i).U, choose)
  when (io.in.valid && io.in.ready) {
    last_grant := choose
  }

  (0 until n).map(i => io.out(i).bits := io.in.bits)
  (0 until n).map(i => io.out(i).tag := io.in.tag)
  io.in.ready := io.out.map(_.ready).foldLeft(false.B)( _ || _)
  io.chosen := choose
}

class gTaggedRRArbiter[T <: Data](n: Int, data: => T) extends Module with TagTrait {
  val io = IO(new gioArbiter(n, data))

  val last_grant = RegInit((0).U((log2Up(n)).W))
  val g = gArbiterCtrl((0 until n).map(i => io.in(i).valid && (i).U > last_grant) ++ io.in.map(_.valid))
  val grant = (0 until n).map(i => g(i) && (i).U > last_grant || g(i+n))
  (0 until n).map(i => io.in(i).ready := grant(i) && io.out.ready)

  var choose = (n-1).U
  for (i <- n-2 to 0 by -1)
    choose = Mux(io.in(i).valid, (i).U, choose)
  for (i <- n-1 to 1 by -1)
    choose = Mux(io.in(i).valid && (i).U > last_grant, (i).U, choose)
  when (io.out.valid && io.out.ready) {
    last_grant := choose
  }

  val dvec = Vec(n, data)
  val tvec = Vec(n, UInt((TAGWIDTH).W))
  (0 until n).map(i => dvec(i) := io.in(i).bits )
  (0 until n).map(i => tvec(i) := io.in(i).tag )

  io.out.valid := io.in.map(_.valid).foldLeft(false.B)( _ || _)
  io.out.bits := dvec(choose)
  io.out.tag := (choose << (TAGWIDTH).U) | tagLower(tvec(choose))
  io.chosen := choose
}

class gTaggedDistributor[T <: Data](n: Int, data: => T) extends Module with TagTrait {
  val io = IO(new gioDistributor(n, data))

  (0 until n).map(i => io.out(i).bits := io.in.bits)
  (0 until n).map(i => io.out(i).tag := tagLower(io.in.tag))
  (0 until n).map(i => io.out(i).valid := io.in.valid && ((i).U((TAGWIDTH).W) === tagUpper(io.in.tag)))
  io.in.ready := io.in.valid && io.out(tagUpper(io.in.tag)).ready
  io.chosen := tagUpper(io.in.tag)
}

class gFIFOIO[T <: Data](data: => T) extends Bundle with TagTrait {
  val ready = Input(Bool())
  val valid = Output(Bool())
  val last = Output(Bool())
  val bits = Output(data)
  val tag = Output(UInt((TAGWIDTH*2).W))

  def fire(dummy: Int = 0) = ready && valid

  override def cloneType = (new gFIFOIO(data)).asInstanceOf[this.type]
}

class gFIFOIOND[T <: Data](data: => T) extends Bundle with TagTrait {
  val ready = Wire(Bool())
  val valid = Wire(Bool())
  val bits = Wire(data)
  val tag = Wire(UInt((TAGWIDTH*2).W))

  def fire(dummy: Int = 0) = ready && valid

  override def cloneType = (new gFIFOIOND(data)).asInstanceOf[this.type]
}

class gInOutBundle[inT <: Data, outT <: Data](
  inData: => inT, outData: => outT
) extends Bundle {
  val in = Flipped(new gFIFOIO(inData))
  val out = new gFIFOIO(outData)
  val pcIn = Flipped(Valid(new PcBundle))
  val pcOut = Valid(new PcBundle)

  override def cloneType = (new gInOutBundle(inData, outData)).asInstanceOf[this.type]
}

class gRWInOutBundle[inReadT <: Data, inWriteT <: Data, outReadT <: Data, outWriteT <: Data](
  inReadData: => inReadT, inWriteData: => inWriteT, outReadData: => outReadT, outWriteData: => outWriteT
) extends Bundle {
  val read = new Bundle {
    val in = Flipped(new gFIFOIO(inReadData))
    val out = new gFIFOIO(outReadData)
  }
  val write = new Bundle {
    val in = Flipped(new gFIFOIO(inWriteData))
    val out = new gFIFOIO(outWriteData)
  }
  val pcIn = Flipped(Valid(new PcBundle))
  val pcOut = Valid(new PcBundle)

  override def cloneType = (new gRWInOutBundle(inReadData, inWriteData, outReadData, outWriteData)).asInstanceOf[this.type]
}

class gOffBundle[inT <: Data, outT <: Data](reqData: => inT, repData: => outT) extends Bundle {
  val req = new gFIFOIO(reqData)
  val rep = Flipped(new gFIFOIO(repData))

  override def cloneType = (new gOffBundle(reqData, repData)).asInstanceOf[this.type]
}

class gOffBundleND[inT <: Data, outT <: Data](reqData: => inT, repData: => outT) extends Bundle {
  val req = new gFIFOIOND(reqData)
  val rep = new gFIFOIOND(repData)

  override def cloneType = (new gOffBundleND(reqData, repData)).asInstanceOf[this.type]
}

class gInOutOffBundle[inT <: Data, outT <: Data](
  inData: => inT, outData: => outT,
  offBundle: => DynamicBundle = DynamicBundle()
)
  extends gInOutBundle(inData, outData)
{
  val off = offBundle
  val mem = new gMemBundle

  override def cloneType = (new gInOutOffBundle(inData, outData, offBundle)).asInstanceOf[this.type]
}

class gRWInOutOffBundle[inReadT <: Data, inWriteT <: Data, outReadT <: Data, outWriteT <: Data](
  inReadData: => inReadT, inWriteData: => inWriteT, outReadData: => outReadT, outWriteData: => outWriteT,
  offBundle: => DynamicBundle = DynamicBundle()
)
  extends gRWInOutBundle(inReadData, inWriteData, outReadData, outWriteData)
{
  val off = offBundle

  override def cloneType = (new gRWInOutOffBundle(inReadData, inWriteData, outReadData, outWriteData, offBundle)).asInstanceOf[this.type]
}

class gMemBundle extends Bundle with MemTrait {
  val mem_addr      = Output(UInt(ADDR_WIDTH.W))
  val read          = Output(Bool())
  val write         = Output(Bool())
  val writedata     = Output(UInt(DATA_WIDTH.W))
  val byteenable    = Output(UInt((DATA_WIDTH/8).W))
  val waitrequest   = Input(Bool())
  val readdatavalid = Input(Bool())
  val readdata      = Input(UInt(DATA_WIDTH.W))
}

trait include extends GorillaUtil {
  val dummy = 0
}

abstract class gComponentBase()
//abstract class gComponentBase[inT <: Data, outT <: Data](inData: => inT, outData: => outT)
  extends Module with include
{
  //val io = IO(new gInOutBundle(inData, outData))

  def cloneType: this.type
  //override def cloneType = (new gComponentBase(inData, outData)).asInstanceOf[this.type]
}

class PcBundle extends Bundle {
  val request = Output(Bool())
  val moduleId = Output(UInt((16).W))
  val portId = Output(UInt((8).W))
  val pcValue = Output(UInt((Pcounters.PCWIDTH).W))
  val pcType = Output(UInt((4).W))
}

class gComponentLeaf[inT <: Data, outT <: Data](
  inData: => inT, outData: => outT,
  offloadData: ArrayBuffer[(String, Data, Data)], extCompName: String = ""
) extends gComponent(inData, outData, offloadData, extCompName)
{
  override def cloneType = (new gComponentLeaf(inData, outData, offloadData, extCompName)).asInstanceOf[this.type]

  if (compilerControl.pcEnable) {
    val pcOutValid = RegInit(false.B)
    val pcOutRequest = RegInit(true.B)
    val pcOutModuleId = RegInit((0).U((16).W))
    val pcOutPortId = RegInit((0).U((8).W))
    val pcOutPcValue = RegInit((0).U((Pcounters.PCWIDTH).W))
    val pcOutPcType = RegInit((0).U((4).W))
    when (io.pcIn.valid && io.pcIn.bits.moduleId === (moduleId).U((16).W) && io.pcIn.bits.request) {
      pcOutValid := true.B
      pcOutPcValue := pcMuxed
      pcOutRequest := false.B
      pcOutPcType := io.pcIn.bits.pcType
      pcOutModuleId := io.pcIn.bits.moduleId
      pcOutPortId := io.pcIn.bits.portId
    }
    .otherwise {
      pcOutValid := io.pcIn.valid
      pcOutPcValue := io.pcIn.bits.pcValue
      pcOutRequest := io.pcIn.bits.request
      pcOutPcType := io.pcIn.bits.pcType
      pcOutModuleId := io.pcIn.bits.moduleId
      pcOutPortId := io.pcIn.bits.portId
    }
    io.pcOut.valid := pcOutValid
    io.pcOut.bits.request := pcOutRequest
    io.pcOut.bits.moduleId := pcOutModuleId
    io.pcOut.bits.portId := pcOutPortId
    io.pcOut.bits.pcValue := pcOutPcValue //mypcMuxed //inPCBackPressure //pcOutPcValue
    io.pcOut.bits.pcType := pcOutPcType
  }
}

class gRWComponentLeaf[inReadT <: Data, inWriteT <: Data, outReadT <: Data, outWriteT <: Data](
  inReadData: => inReadT, inWriteData: => inWriteT, outReadData: => outReadT, outWriteData: => outWriteT,
  offloadData: ArrayBuffer[(String, Data, Data)], extCompName: String = ""
) extends gRWComponent(inReadData, inWriteData, outReadData, outWriteData, offloadData, extCompName)
{
  override def cloneType = (new gRWComponentLeaf(inReadData, inWriteData, outReadData, outWriteData, offloadData, extCompName)).asInstanceOf[this.type]

  if (compilerControl.pcEnable) {
    val pcOutValid = RegInit(false.B)
    val pcOutRequest = RegInit(true.B)
    val pcOutModuleId = RegInit((0).U((16).W))
    val pcOutPortId = RegInit((0).U((8).W))
    val pcOutPcValue = RegInit((0).U((Pcounters.PCWIDTH).W))
    val pcOutPcType = RegInit((0).U((4).W))
    when (io.pcIn.valid && io.pcIn.bits.moduleId === (moduleId).U((16).W) && io.pcIn.bits.request) {
      pcOutValid := true.B
      pcOutPcValue := pcMuxed
      pcOutRequest := false.B
      pcOutPcType := io.pcIn.bits.pcType
      pcOutModuleId := io.pcIn.bits.moduleId
      pcOutPortId := io.pcIn.bits.portId
    }
    .otherwise {
      pcOutValid := io.pcIn.valid
      pcOutPcValue := io.pcIn.bits.pcValue
      pcOutRequest := io.pcIn.bits.request
      pcOutPcType := io.pcIn.bits.pcType
      pcOutModuleId := io.pcIn.bits.moduleId
      pcOutPortId := io.pcIn.bits.portId
    }
    io.pcOut.valid := pcOutValid
    io.pcOut.bits.request := pcOutRequest
    io.pcOut.bits.moduleId := pcOutModuleId
    io.pcOut.bits.portId := pcOutPortId
    io.pcOut.bits.pcValue := pcOutPcValue
    io.pcOut.bits.pcType := pcOutPcType
  }
}

class gComponent[inT <: Data, outT <: Data](
  inData: => inT, outData: => outT, offloadData: ArrayBuffer[(String, Data, Data)], extCompName: String = ""
) extends gComponentBase()
{
  override def cloneType = (new gComponent(inData, outData, offloadData, extCompName)).asInstanceOf[this.type]

  val offBundle = DynamicBundle(offloadData.map((d) => (d._1, { val dBundle = new gOffBundle(d._2, d._3); dBundle.suggestName(d._1); dBundle})))
  //offBundle.suggestName("off")
  val io = IO(new gInOutOffBundle(inData, outData, offBundle))

  //name_it()
  if (extCompName != "") {
    suggestName(extCompName + "__class__" + this.getClass.getSimpleName)
  }
  println("In module " + toNamed + ", num of offload ports " + offloadData.size)

  //val nullOff = Wire(new Bundle{ val nullOff = new gOffBundle(UInt(32.W), UInt(32.W)) })
  //val nullOffOff = Wire(new gOffBundle(UInt(32.W), UInt(32.W)))
  //def ioOff = io.elements.getOrElse("off", nullOff).asInstanceOf[Bundle]
  def ioOff = io.elements("off").asInstanceOf[Bundle]

  val moduleId = Pcounters.registerModule(toNamed.toString)
  var pcMuxed = (0).U((Pcounters.PCWIDTH).W)
  val offloadRateArray =
    if (offloadData.size == 0)
      RegInit(VecInit(Seq.fill(1)((0).U((Pcounters.PCWIDTH).W))))
    else
      RegInit(VecInit(Seq.fill(offloadData.size)((0).U((Pcounters.PCWIDTH).W))))
  val pcPaused = RegInit(false.B)
  val engineUtilization = RegInit(0.U((Pcounters.PCWIDTH).W))
  val inTokens = RegInit(0.U((Pcounters.PCWIDTH).W))
  val outTokens = RegInit(0.U((Pcounters.PCWIDTH).W))

  if (compilerControl.pcEnable) {
    //offloadData.foreach(x => println("Port name is " + x._1))
    val offPCBackPressure =
      if (offloadData.size == 0)
        RegInit(VecInit(Seq.fill(1)((0).U((Pcounters.PCWIDTH).W))))
      else
        RegInit(VecInit(Seq.fill(offloadData.size)((0).U((Pcounters.PCWIDTH).W))))

    //println("Module name is " + toNamed.toString)
    var portId = 1 //leave zero for broadcast
    val inPCBackPressure = RegInit((0).U((Pcounters.PCWIDTH).W))
    val outPCBackPressure = RegInit((0).U((Pcounters.PCWIDTH).W))

    val IsPcReset = io.pcIn.valid && io.pcIn.bits.request && io.pcIn.bits.pcType === Pcounters.pcReset
    when (IsPcReset) {
      pcPaused := false.B
    }
    when (io.pcIn.valid && io.pcIn.bits.request && io.pcIn.bits.pcType === Pcounters.pcPause) {
      pcPaused := !pcPaused
    }
    //Input back pressure pc
    Pcounters.registerPC("in", Pcounters.backPressure, moduleId.asInstanceOf[Int], portId)
    when (IsPcReset) {
      inPCBackPressure := (0).U((Pcounters.PCWIDTH).W)
    }
    .elsewhen (io.in.valid && !io.in.ready && !pcPaused) {
      inPCBackPressure := inPCBackPressure + (1).U((Pcounters.PCWIDTH).W)
    }
    pcMuxed = Mux(io.pcIn.bits.portId === (portId).U((Pcounters.PCWIDTH).W) &&
    io.pcIn.bits.pcType === Pcounters.backPressure, inPCBackPressure, pcMuxed)

    //Output back pressure pc
    portId = portId + 1
    Pcounters.registerPC("out", Pcounters.backPressure, moduleId.asInstanceOf[Int], portId)
    when (IsPcReset) {
      engineUtilization := (0).U((Pcounters.PCWIDTH).W)
      outPCBackPressure := (0).U((Pcounters.PCWIDTH).W)
    }
    .elsewhen (io.out.valid && !io.out.ready && !pcPaused) {
      outPCBackPressure := outPCBackPressure + (1).U((Pcounters.PCWIDTH).W)
    }
    when (IsPcReset) {
      inTokens := (0).U((Pcounters.PCWIDTH).W)
    }
    .elsewhen (io.in.valid && io.in.ready && !pcPaused) {
      inTokens := inTokens + (1).U((Pcounters.PCWIDTH).W)
    }
    when (IsPcReset) {
      outTokens := (0).U((Pcounters.PCWIDTH).W)
    }
    .elsewhen (io.out.valid && io.out.ready && !pcPaused) {
      outTokens := outTokens + (1).U((Pcounters.PCWIDTH).W)
    }
    pcMuxed = Mux(io.pcIn.bits.portId === (portId).U((Pcounters.PCWIDTH).W) &&
    io.pcIn.bits.pcType === Pcounters.backPressure, outPCBackPressure, pcMuxed)
    portId = portId + 1

    //Offload backpressure/rate pcs
    for ((n, i) <- ioOff.elements) {
      Pcounters.registerPC(n, Pcounters.backPressure, moduleId.asInstanceOf[Int], portId)
      when (IsPcReset) {
        offloadRateArray(portId-3) := (0).U((Pcounters.PCWIDTH).W)
        offPCBackPressure(portId-3) := (0).U((Pcounters.PCWIDTH).W)
      }
      .elsewhen (
        i.asInstanceOf[gOffBundle[Data, Data]].req.valid &&
        !i.asInstanceOf[gOffBundle[Data, Data]].req.ready && !pcPaused)
      {
        offPCBackPressure(portId-3) := offPCBackPressure(portId-3) + (1).U((Pcounters.PCWIDTH).W)
      }
      //Mux the pcounter based on the index
      pcMuxed = Mux(
        io.pcIn.bits.portId === (portId).U((Pcounters.PCWIDTH).W) && io.pcIn.bits.pcType === Pcounters.backPressure,
        offPCBackPressure(portId-3),
        pcMuxed
      )
      portId = portId + 1
    }
    Pcounters.registerPC("offloadRate", Pcounters.offloadRate, moduleId.asInstanceOf[Int], 0)
    val offloadRate = offloadRateArray.reduceLeft((x,y) => Mux(x>y, x, y))
    pcMuxed = Mux(io.pcIn.bits.pcType === Pcounters.offloadRate, offloadRate, pcMuxed)
    Pcounters.registerPC("engineUtilization", Pcounters.engineUtilization, moduleId.asInstanceOf[Int], 0)
    pcMuxed = Mux(io.pcIn.bits.pcType === Pcounters.engineUtilization, engineUtilization, pcMuxed)
    Pcounters.registerPC("inTokens", Pcounters.inTokens, moduleId.asInstanceOf[Int], 0)
    pcMuxed = Mux(io.pcIn.bits.pcType === Pcounters.inTokens, inTokens, pcMuxed)
    Pcounters.registerPC("outTokens", Pcounters.outTokens, moduleId.asInstanceOf[Int], 0)
    pcMuxed = Mux(io.pcIn.bits.pcType === Pcounters.outTokens, outTokens, pcMuxed)
  }
  else {
    for (i <- 0 until offloadData.size) {
      offloadRateArray(i) := (0).U((Pcounters.PCWIDTH).W)
    }
    pcPaused := false.B
    engineUtilization := (0).U((Pcounters.PCWIDTH).W)
    inTokens := (0).U((Pcounters.PCWIDTH).W)
    outTokens := (0).U((Pcounters.PCWIDTH).W)
  }
}

class gRWComponent[inReadT <: Data, inWriteT <: Data, outReadT <: Data, outWriteT <: Data](
  inReadData: => inReadT, inWriteData: => inWriteT, outReadData: => outReadT, outWriteData: => outWriteT,
  offloadData: ArrayBuffer[(String, Data, Data)], extCompName: String = ""
) extends gComponentBase()
{
  override def cloneType = (new gRWComponent(inReadData, inWriteData, outReadData, outWriteData, offloadData, extCompName)).asInstanceOf[this.type]

  val offBundle = DynamicBundle(offloadData.map((d) => (d._1, { val dBundle = new gOffBundle(d._2, d._3); dBundle.suggestName(d._1); dBundle})))
  val io = IO(new gRWInOutOffBundle(inReadData, inWriteData, outReadData, outWriteData, offBundle))

  if (extCompName != "") {
    suggestName(extCompName + "__class__" + this.getClass.getSimpleName)
  }
  println("In module " + toNamed + ", num of offload ports " + offloadData.size)

  def ioOff = io.elements("off").asInstanceOf[Bundle]

  val moduleId = Pcounters.registerModule(toNamed.toString)
  var pcMuxed = (0).U((Pcounters.PCWIDTH).W)
  val offloadRateArray =
    if (offloadData.size == 0)
      RegInit(VecInit(Seq.fill(1)((0).U((Pcounters.PCWIDTH).W))))
    else
      RegInit(VecInit(Seq.fill(offloadData.size)((0).U((Pcounters.PCWIDTH).W))))
  val pcPaused = RegInit(false.B)
  val engineUtilization = RegInit(0.U((Pcounters.PCWIDTH).W))
  val inTokens = RegInit(0.U((Pcounters.PCWIDTH).W))
  val outTokens = RegInit(0.U((Pcounters.PCWIDTH).W))

  if (compilerControl.pcEnable) {
    val offPCBackPressure =
      if (offloadData.size == 0)
        RegInit(VecInit(Seq.fill(1)((0).U((Pcounters.PCWIDTH).W))))
      else
        RegInit(VecInit(Seq.fill(offloadData.size)((0).U((Pcounters.PCWIDTH).W))))

    var portId = 1 //leave zero for broadcast
    val inPCBackPressure = RegInit((0).U((Pcounters.PCWIDTH).W))
    val outPCBackPressure = RegInit((0).U((Pcounters.PCWIDTH).W))

    val IsPcReset = io.pcIn.valid && io.pcIn.bits.request && io.pcIn.bits.pcType === Pcounters.pcReset
    when (IsPcReset) {
      pcPaused := false.B
    }
    when (io.pcIn.valid && io.pcIn.bits.request && io.pcIn.bits.pcType === Pcounters.pcPause) {
      pcPaused := !pcPaused
    }
    //Input back pressure pc
    Pcounters.registerPC("in", Pcounters.backPressure, moduleId.asInstanceOf[Int], portId)
    when (IsPcReset) {
      inPCBackPressure := (0).U((Pcounters.PCWIDTH).W)
    }
    .elsewhen (io.read.in.valid && !io.read.in.ready && io.write.in.valid && !io.write.in.ready && !pcPaused) {
      inPCBackPressure := inPCBackPressure + (1).U((Pcounters.PCWIDTH).W)
    }
    pcMuxed = Mux(io.pcIn.bits.portId === (portId).U((Pcounters.PCWIDTH).W) &&
    io.pcIn.bits.pcType === Pcounters.backPressure, inPCBackPressure, pcMuxed)

    //Output back pressure pc
    portId = portId + 1
    Pcounters.registerPC("out", Pcounters.backPressure, moduleId.asInstanceOf[Int], portId)
    when (IsPcReset) {
      engineUtilization := (0).U((Pcounters.PCWIDTH).W)
      outPCBackPressure := (0).U((Pcounters.PCWIDTH).W)
    }
    .elsewhen (io.read.out.valid && !io.read.out.ready && io.write.out.valid && !io.write.out.ready && !pcPaused) {
      outPCBackPressure := outPCBackPressure + (1).U((Pcounters.PCWIDTH).W)
    }
    when (IsPcReset) {
      inTokens := (0).U((Pcounters.PCWIDTH).W)
    }
    .elsewhen (io.read.in.valid && io.read.in.ready && io.write.in.valid && io.write.in.ready && !pcPaused) {
      inTokens := inTokens + (1).U((Pcounters.PCWIDTH).W)
    }
    when (IsPcReset) {
      outTokens := (0).U((Pcounters.PCWIDTH).W)
    }
    .elsewhen (io.read.out.valid && io.read.out.ready && io.write.out.valid && io.write.out.ready && !pcPaused) {
      outTokens := outTokens + (1).U((Pcounters.PCWIDTH).W)
    }
    pcMuxed = Mux(io.pcIn.bits.portId === (portId).U((Pcounters.PCWIDTH).W) &&
    io.pcIn.bits.pcType === Pcounters.backPressure, outPCBackPressure, pcMuxed)
    portId = portId + 1

    //Offload backpressure/rate pcs
    for ((n, i) <- ioOff.elements) {
      Pcounters.registerPC(n, Pcounters.backPressure, moduleId.asInstanceOf[Int], portId)
      when (IsPcReset) {
        offloadRateArray(portId-3) := (0).U((Pcounters.PCWIDTH).W)
        offPCBackPressure(portId-3) := (0).U((Pcounters.PCWIDTH).W)
      }
      .elsewhen (
        i.asInstanceOf[gOffBundle[Data, Data]].req.valid &&
        !i.asInstanceOf[gOffBundle[Data, Data]].req.ready && !pcPaused)
      {
        offPCBackPressure(portId-3) := offPCBackPressure(portId-3) + (1).U((Pcounters.PCWIDTH).W)
      }
      //Mux the pcounter based on the index
      pcMuxed = Mux(
        io.pcIn.bits.portId === (portId).U((Pcounters.PCWIDTH).W) && io.pcIn.bits.pcType === Pcounters.backPressure,
        offPCBackPressure(portId-3),
        pcMuxed
      )
      portId = portId + 1
    }
    Pcounters.registerPC("offloadRate", Pcounters.offloadRate, moduleId.asInstanceOf[Int], 0)
    val offloadRate = offloadRateArray.reduceLeft((x,y) => Mux(x>y, x, y))
    pcMuxed = Mux(io.pcIn.bits.pcType === Pcounters.offloadRate, offloadRate, pcMuxed)
    Pcounters.registerPC("engineUtilization", Pcounters.engineUtilization, moduleId.asInstanceOf[Int], 0)
    pcMuxed = Mux(io.pcIn.bits.pcType === Pcounters.engineUtilization, engineUtilization, pcMuxed)
    Pcounters.registerPC("inTokens", Pcounters.inTokens, moduleId.asInstanceOf[Int], 0)
    pcMuxed = Mux(io.pcIn.bits.pcType === Pcounters.inTokens, inTokens, pcMuxed)
    Pcounters.registerPC("outTokens", Pcounters.outTokens, moduleId.asInstanceOf[Int], 0)
    pcMuxed = Mux(io.pcIn.bits.pcType === Pcounters.outTokens, outTokens, pcMuxed)
  }
  else {
    for (i <- 0 until offloadData.size) {
      offloadRateArray(i) := (0).U((Pcounters.PCWIDTH).W)
    }
    pcPaused := false.B
    engineUtilization := (0).U((Pcounters.PCWIDTH).W)
    inTokens := (0).U((Pcounters.PCWIDTH).W)
    outTokens := (0).U((Pcounters.PCWIDTH).W)
  }
}

abstract class gComponentGenBase(
  _offloadData: ArrayBuffer[(String, Data, Data)], _extCompName: String = ""
)
{
  val offloadData = _offloadData
  val extCompName = _extCompName
}

class gComponentGen[inT <: Data, outT <: Data](
  comp: => gComponent[inT, outT],
  _inData: => inT, _outData: => outT,
  _offloadData: ArrayBuffer[(String, Data, Data)], _extCompName: String = ""
)
  extends gComponentGenBase(_offloadData, _extCompName)
{
  val inData = _inData
  val outData = _outData

  def apply(): gComponent[inT, outT] = comp
}

class gRWComponentGen[inReadT <: Data, inWriteT <: Data, outReadT <: Data, outWriteT <: Data](
  comp: => gRWComponent[inReadT, inWriteT, outReadT, outWriteT],
  _inReadData: => inReadT, _inWriteData: => inWriteT, _outReadData: => outReadT, _outWriteData: => outWriteT,
  _offloadData: ArrayBuffer[(String, Data, Data)], _extCompName: String = ""
)
  extends gComponentGenBase(_offloadData, _extCompName)
{
  val inReadData = _inReadData
  val inWriteData = _inWriteData
  val outReadData = _outReadData
  val outWriteData = _outWriteData

  def apply(): gRWComponent[inReadT, inWriteT, outReadT, outWriteT] = comp
}

//class gComponentMD[inT <: Data, outT <: Data](inData: => inT, outData: => outT, offloadData: ArrayBuffer[(String, Data, Data)]) {
//  val inDataGen = inData
//  val outDataGen = outData
//  val offloadDataGen = offloadData
//
//  override def clone = { new gComponentMD(inData, outData, offloadData).asInstanceOf[this.type] }
//}

class gChainedComponent[inT <: Data, connT <: Data, outT <: Data](
  inDataSrc: => inT, outDataSrc: => connT, inDataSink: => connT, outDataSink: => outT,
  offloadData: ArrayBuffer[(String, Data, Data)],
  srcCompGen: gComponentGen[inT, connT], sinkCompGen: gComponentGen[connT, outT],
  extCompName: String
) extends gComponent(inDataSrc, outDataSink, offloadData, extCompName) with include
{
  val srcComp = Module(srcCompGen())
  val sinkComp = Module(sinkCompGen())

  //if (srcComp.parent != this) {
  //  srcComp.parent.children -= srcComp
  //  srcComp.parent = this
  //}
  //if (!this.children.contains(srcComp))
  //  this.children += srcComp
  //if (sinkComp.parent != this) {
  //  sinkComp.parent.children -= sinkComp
  //  sinkComp.parent = this
  //}
  //if (!this.children.contains(sinkComp))
  //  this.children += sinkComp

  //println("In gChained")
  //printChildren(this)

  io.in <> srcComp.io.in
  io.out <> sinkComp.io.out
  srcComp.io.out <> sinkComp.io.in
  //Connect the offload interfaces of offloaded component to the enclosing
  //component's offload interfaces
  //def sourceOff = srcComp.io.elements.getOrElse("off", nullOff).asInstanceOf[Bundle]
  def sourceOff = srcComp.io.elements("off").asInstanceOf[Bundle]
  //def sinkOff = sinkComp.io.elements.getOrElse("off", nullOff).asInstanceOf[Bundle]
  def sinkOff = sinkComp.io.elements("off").asInstanceOf[Bundle]
  val cOffElements = sinkOff.elements.filter((t) => {
    sourceOff.elements.exists((t1) => {t._1 == t1._1})
  })
  val cOffData = offloadData.filter((t) => {
    sinkOff.elements.exists((t1) => {t._1 == t1._1}) &&
    sourceOff.elements.exists((t2) => {t._1 == t2._1})
  })
  for ((n, i) <- ioOff.elements) {
    for ((n1, i1) <- sinkOff.elements) {
      if (n == n1 && !sourceOff.elements.exists((t) => {t._1 == n1})) {
        i1.asInstanceOf[gOffBundle[Data, Data]].req <> i.asInstanceOf[gOffBundle[Data, Data]].req
        i1.asInstanceOf[gOffBundle[Data, Data]].rep <> i.asInstanceOf[gOffBundle[Data, Data]].rep
      }
    }
  }
  for ((n, i) <- ioOff.elements) {
    for ((n1, i1) <- sourceOff.elements) {
      if (n == n1 && !sinkOff.elements.exists((t) => {t._1 == n1})) {
        i1.asInstanceOf[gOffBundle[Data, Data]].req <> i.asInstanceOf[gOffBundle[Data, Data]].req
        i1.asInstanceOf[gOffBundle[Data, Data]].rep <> i.asInstanceOf[gOffBundle[Data, Data]].rep
      }
    }
  }

  val reqArbs = cOffData.map(i => new gTaggedRRArbiter(2, i._2))
  val repDists = cOffData.map(i => new gTaggedDistributor(2, i._3))
  var i = 0

  for ((name1, reqInterface, replyInterface) <- cOffData) {
    for ((name, interface) <- ioOff.elements) {
      if (name1 == name) {
        interface.asInstanceOf[gOffBundle[Data, Data]].req <> reqArbs(i).io.out
        interface.asInstanceOf[gOffBundle[Data, Data]].rep <> repDists(i).io.in
      }
    }
    for ((name, interface) <- sourceOff.elements) {
      if (name1 == name) {
        interface.asInstanceOf[gOffBundle[Data, Data]].req <> reqArbs(i).io.in(0)
        interface.asInstanceOf[gOffBundle[Data, Data]].rep <> repDists(i).io.out(0)
      }
    }
    for ((name, interface) <- sinkOff.elements) {
      if (name1 == name) {
        interface.asInstanceOf[gOffBundle[Data, Data]].req <> reqArbs(i).io.in(1)
        interface.asInstanceOf[gOffBundle[Data, Data]].rep <> repDists(i).io.out(1)
      }
    }
    i = i + 1
  }

  //attach the performance counter interfaces
  io.pcIn <> srcComp.io.pcIn
  io.pcOut <> sinkComp.io.pcOut
  srcComp.io.pcOut <> sinkComp.io.pcIn
}

class gOffloadedComponent[
  inT <: Data, outT <: Data,
  inOffT <: Data, outOffT <: Data
](
  inData: => inT, outData: => outT,
  inOffData: => inOffT, outOffData: => outOffT,
  offloadData: ArrayBuffer[(String, Data, Data)],
  mainCompGen: gComponentGen[inT, outT], offCompGen: gComponentGen[inOffT, outOffT],
  offPort: String, extCompName: String
) extends gComponent(inData, outData, offloadData, extCompName) with include
{
  //println("This is gOffloadedComponent, offPort is " , offPort)

  val mainComp = Module(mainCompGen())
  val offComp = Module(offCompGen())

  //if (mainComp.parent != this) {
  //  mainComp.parent.children -= mainComp
  //  mainComp.parent = this
  //}
  //if (!this.children.contains(mainComp))
  //  this.children += mainComp
  //if (offComp.parent != this) {
  //  offComp.parent.children -= offComp
  //  offComp.parent = this
  //}
  //if (!this.children.contains(offComp))
  //  this.children += offComp

  //println("In gOffloaded")
  //printChildren(this)

  //val (left, right) = offloadDataMain.span( _._1 == offPort)
  //if (left.isEmpty) println("Gorilla++ Error: no offloaded port " + offPort + " in module")
  //val restOfOffloadedDataMain = { if (left.isEmpty) right else left.init ++ right}
  //val offoff = Bundle((mergeOffloads(restOfOffloadedDataMain, offloadDataOff).map((t) => (t._1, t._2()))))
  //offoff.name = "off"
  //io += offoff

  io.in <> mainComp.io.in
  io.out <> mainComp.io.out

  //def mainOff = mainComp.io.elements.getOrElse("off", nullOff).asInstanceOf[Bundle]
  def mainOff = mainComp.io.elements("off").asInstanceOf[Bundle]
  //def offOff = offComp.io.elements.getOrElse("off", nullOff).asInstanceOf[Bundle]
  def offOff = offComp.io.elements("off").asInstanceOf[Bundle]
  //Connect main component offload interface with argument port name to in/out of the offload component
  //Connect the rest of main component offload interfaces to the enclosing component's offload interfaces
  for ((n, i) <- mainOff.elements) {
    //println("offload name is " + n)
    if (n == offPort && i.isInstanceOf[gOffBundle[inOffT, outOffT]]) {
      i.asInstanceOf[gOffBundle[inOffT, outOffT]].req <> offComp.io.in
      i.asInstanceOf[gOffBundle[inOffT, outOffT]].rep <> offComp.io.out
    }
    else {
      for ((n1, i1) <- ioOff.elements) {
        if (n == n1) {
          i1.asInstanceOf[gOffBundle[Data, Data]].req <> i.asInstanceOf[gOffBundle[Data, Data]].req
          i1.asInstanceOf[gOffBundle[Data, Data]].rep <> i.asInstanceOf[gOffBundle[Data, Data]].rep
        }
      }
    }
  }
  //Connect the offload interfaces of offloaded compoenent to the enclosing component's offload interfaces
  for ((n, i) <- offOff.elements) {
    for ((n1, i1) <- ioOff.elements) {
      if (n == n1) {
        i1.asInstanceOf[gOffBundle[Data, Data]].req <> i.asInstanceOf[gOffBundle[Data, Data]].req
        i1.asInstanceOf[gOffBundle[Data, Data]].rep <> i.asInstanceOf[gOffBundle[Data, Data]].rep
      }
    }
  }
  //attache the performance counter interfaces
  io.pcIn <> mainComp.io.pcIn
  io.pcOut <> offComp.io.pcOut
  mainComp.io.pcOut <> offComp.io.pcIn
}

class gOffloadedRWComponent[
  inT <: Data, outT <: Data,
  inReadOffT <: Data, inWriteOffT <: Data, outReadOffT <: Data, outWriteOffT <: Data
](
  inData: => inT, outData: => outT,
  inReadOffData: => inReadOffT, inWriteOffData: => inWriteOffT, outReadOffData: => outReadOffT, outWriteOffData: => outWriteOffT,
  offloadData: ArrayBuffer[(String, Data, Data)],
  mainCompGen: gComponentGen[inT, outT], offCompGen: gRWComponentGen[inReadOffT, inWriteOffT, outReadOffT, outWriteOffT],
  offPort: String, extCompName: String
) extends gComponent(inData, outData, offloadData, extCompName) with include
{
  val mainComp = Module(mainCompGen())
  val offComp = Module(offCompGen())

  //if (mainComp.parent != this) {
  //  mainComp.parent.children -= mainComp
  //  mainComp.parent = this
  //}
  //if (!this.children.contains(mainComp))
  //  this.children += mainComp
  //if (offComp.parent != this) {
  //  offComp.parent.children -= offComp
  //  offComp.parent = this
  //}
  //if (!this.children.contains(offComp))
  //  this.children += offComp

  io.in <> mainComp.io.in
  io.out <> mainComp.io.out

  //def mainOff = mainComp.io.elements.getOrElse("off", nullOff).asInstanceOf[Bundle]
  def mainOff = mainComp.io.elements("off").asInstanceOf[Bundle]
  //def offOff = offComp.io.elements.getOrElse("off", nullOff).asInstanceOf[Bundle]
  def offOff = offComp.io.elements("off").asInstanceOf[Bundle]
  //Connect main component offload interface with argument port name to in/out of the offload component
  //Connect the rest of main component offload interfaces to the enclosing component's offload interfaces
  for ((n, i) <- mainOff.elements) {
    //println("offload name is " + n)
    if (n == (offPort + "Read") && i.isInstanceOf[gOffBundle[inReadOffT, outReadOffT]]) {
      i.asInstanceOf[gOffBundle[inReadOffT, outReadOffT]].req <> offComp.io.read.in
      i.asInstanceOf[gOffBundle[inReadOffT, outReadOffT]].rep <> offComp.io.read.out
    }
    else if (n == (offPort + "Write") && i.isInstanceOf[gOffBundle[inWriteOffT, outWriteOffT]]) {
      i.asInstanceOf[gOffBundle[inWriteOffT, outWriteOffT]].req <> offComp.io.write.in
      i.asInstanceOf[gOffBundle[inWriteOffT, outWriteOffT]].rep <> offComp.io.write.out
    }
    else {
      for ((n1, i1) <- ioOff.elements) {
        if (n == n1) {
          i1.asInstanceOf[gOffBundle[Data, Data]].req <> i.asInstanceOf[gOffBundle[Data, Data]].req
          i1.asInstanceOf[gOffBundle[Data, Data]].rep <> i.asInstanceOf[gOffBundle[Data, Data]].rep
        }
      }
    }
  }
  //We dont Connect the offload interfaces of offloaded compoenent to the enclosing
  //component's offload interfaces
  //rwSpMem does not have offload ports.
  //for ((n, i) <- offOff.elements) {
  //  for ((n1, i1) <- ioOff.elements) {
  //    if (n == n1) {
  //      i1.asInstanceOf[gOffBundle[Data, Data]].req <> i.asInstanceOf[gOffBundle[Data, Data]].req
  //      i1.asInstanceOf[gOffBundle[Data, Data]].rep <> i.asInstanceOf[gOffBundle[Data, Data]].rep
  //    }
  //  }
  //}
  //attach the performance counter interfaces. We keep the spMems out of rings
  //Cause they are gComponnet and do not have PCs. TODO: change this
  io.pcIn <> mainComp.io.pcIn
  mainComp.io.pcOut <> offComp.io.pcIn
  io.pcOut <> mainComp.io.pcOut
}

class distributorComponent[T <: Data](n: Int, data: => T) extends Module {
  val io = IO(new gioDistributor(n, data))
}

class RRDistributorComponent[T <: Data](n: Int, data: => T) extends distributorComponent(n, data) {
  val rrDist = new gRRDistributor(n, data)
  //name_it()
  io <> rrDist.io
}

class distributorEngine[T <: Data](n: Int, data: => T) extends Module {
  val io = IO(new Bundle {
    val out = new gFIFOIO(data)
    val in = Flipped(new gFIFOIO(data))
    val outIndex = Output(UInt((log2Up(n)).W))
  })
}

class aggregatorComponent[T <: Data](n: Int, data: => T) extends Module {
  val io = IO(new gioArbiter(n, data))
}

class RRAggregatorComponent[T <: Data](n: Int, data: => T) extends aggregatorComponent(n, data) {
  val rrArb = new gRRArbiter(n, data)
  //rrArb.name_it()
  //name_it()
  io <> rrArb.io
}

class aggregatorEngine[T <: Data](n: Int, data: => T) extends Module {
  val io = IO(new Bundle {
    val out = Flipped(new gFIFOIO(data))
    val in = new gFIFOIO(data)
    val inIndex = Output(UInt((log2Up(n)).W))
  })
}

class pDistributor[T <: Data](n: Int, data: => T,
  distributorEngineGen: (Int, => T) => distributorEngine[T]
) extends distributorComponent[T](n, data) with include
{
  val distEngine = distributorEngineGen(n, data)
  val readies = Vec(n, Bool())

  io.in <> distEngine.io.in
  val broadcast =
    distEngine.io.outIndex === broadcastDistribute && io.out.map(x => x.ready).reduce(_&&_) && distEngine.io.out.valid

  for (i <- 0 to n) {
    io.out(i).valid := Mux(broadcast, true.B, (i).U === distEngine.io.outIndex && distEngine.io.out.valid)
    io.out(i).bits <> io.in.bits
    readies(i) := io.out(i).ready && (i).U === distEngine.io.outIndex
  }
  distEngine.io.out.ready := broadcast || readies.reduce(_&&_)
  io.in.ready := distEngine.io.in.ready
}

//class pAggregrator[T <: Data](n: Int, data: => T,
//  aggregatorEngineGen: (Int, => T) => aggregatorEngine[T]
//) extends aggregatorComponent(n, data) with include
//{
//  val aggEngine = aggregatorEngineGen(n, data)
//  val rrArb = new gRRArbiter(n, data) //TODO: if aggregator wants an specific input RR should be off
//  io.in <> rrArb.io.in
//  rrArb.io.out <> aggEngine.io.in
//  io.out <> aggEngine.io.out
//}

//class gReplicatedComponentDistAgg[inT <: Data, outT <: Data](
//  inData: => inT, outData: => outT, offloadData: ArrayBuffer[(String, Data, Data)],
//  compGen: gComponentGen[inT, outT], n: Int,
//  distributorGen: (Int, => inT) => distributorComponent[inT],
//  aggregatorGen: (Int, => outT) => aggregatorComponent[outT]
//) extends gComponent(inData, outData, offloadData) with include
//{}

class gReplicatedComponent[inT <: Data, outT <: Data](
  inData: => inT, outData: => outT, offloadData: ArrayBuffer[(String, Data, Data)],
  compGen: gComponentGen[inT, outT], n: Int, extCompName: String
) extends gComponent(inData, outData, offloadData, extCompName) with include
{
  val components = Range(0, n).map(i => Module(compGen()))
  //components.foreach(i => {
  //  if (i.parent != this) {
  //    i.parent.children -= i
  //    i.parent = this
  //  }
  //  if (!this.children.contains(i))
  //    this.children += i
  //})

  //val inputDist = distributorGen(n, inData)
  val inputDist = (new RRDistributorComponent(n, inData))
  val outputArb = (new RRAggregatorComponent(n, outData))
  //outputArb.name_it()
  //val outputArb = aggregatorGen(n, outData)
  val reqArbs = offloadData.map(i => new gTaggedRRArbiter(n, i._2))
  //val reqArbs = offloadData.map(i => new gRRArbiter(n, i._2))
  val repDists = offloadData.map(i => new gTaggedDistributor(n, i._3))
  //val repDists = offloadData.map(i => new gRRDistributor(n, i._3))
  //val repDists = offloadData.map(i => (new gRRDistributor(n)) {i._3()})
  //reqArbs.foreach(i => i.name_it())

  //println("In gReplicated")
  //printChildren(this)

  //offloadData.map(i => Range(0, n-1).map(reqArbs(i).in(j) := components(j)
  io.in <> inputDist.io.in
  Range(0, n).foreach(i => inputDist.io.out(i) <> components(i).io.in)
  io.out <> outputArb.io.out
  Range(0, n).foreach(i => outputArb.io.in(i) <> components(i).io.out)

  var i = 0
  //def cOff = io.elements.getOrElse("off", nullOff).asInstanceOf[Bundle]
  def cOff = io.elements("off").asInstanceOf[Bundle]

  for ((name, interface) <- cOff.elements) {
    interface.asInstanceOf[gOffBundle[Data, Data]].req <> reqArbs(i).io.out
    interface.asInstanceOf[gOffBundle[Data, Data]].rep <> repDists(i).io.in
    i = i + 1
  }

  for (j <- 0 until n) {
    i = 0
    //def cOff = components(j).io.elements.getOrElse("off", nullOff).asInstanceOf[Bundle]
    def cOff = components(j).io.elements("off").asInstanceOf[Bundle]
    //Chain the performance counter interfaces
    if (j > 0) {
      //def pcIn = components(j).io.elements.getOrElse("pcIn", nullOff).asInstanceOf[Bundle]
      def pcIn = components(j).io.elements("pcIn").asInstanceOf[Bundle]
      //def pcOutPrevious = components(j-1).io.elements.getOrElse("pcOut", nullOff).asInstanceOf[Bundle]
      def pcOutPrevious = components(j-1).io.elements("pcOut").asInstanceOf[Bundle]
      pcIn <> pcOutPrevious
    }
    for ((name, interface) <- cOff.elements) {
      //if (interface.isInstanceOf[gOffBundle[Data, Data]]) {
        interface.asInstanceOf[gOffBundle[Data, Data]].req <> reqArbs(i).io.in(j)
        interface.asInstanceOf[gOffBundle[Data, Data]].rep <> repDists(i).io.out(j)
      //}
      i = i + 1
    }
  }

  //Attached the first component pc input to the main pc input
  //def pcIn0 = components(0).io.elements.getOrElse("pcIn", nullOff).asInstanceOf[Bundle]
  def pcIn0 = components(0).io.elements("pcIn").asInstanceOf[Bundle]
  io.pcIn <> pcIn0
  //Attached the last component pc out to the main pc output
  //def pcOutN = components(n-1).io.elements.getOrElse("pcOut", nullOff).asInstanceOf[Bundle]
  def pcOutN = components(n-1).io.elements("pcOut").asInstanceOf[Bundle]
  io.pcOut <> pcOutN
}

class PcElement(myName: String, myPCType: UInt, myModuleId: Int, myPortId: Int) {
  val name = myName
  val pcType = myPCType
  val moduleId = myModuleId
  val portId = myPortId
  var pcValue = 0
}

object Pcounters {
  val PCWIDTH = 20
  val nopc::backPressure::offloadRate::engineUtilization::pcReset::pcPause::inTokens::outTokens::Nil = Enum(8)
  var moduleId = 1 //leave zero for broadcast
  var elements = new ArrayBuffer[PcElement]()
  var moduleIDs = new HashMap[String, Int]()
  def numOfOffloadPorts(moduleName: String) = {
    elements.count(e => e.moduleId == moduleIDs.getOrElse(moduleName, 0) && e.pcType == backPressure)-2
  }
  def registerPC(name: String, pcType: UInt, moduleId: Int, portId: Int) {
    elements += new PcElement(name, pcType, moduleId, portId)
    //println("PCREPORT: PC is registered name: " + name + " type " + pcType.litValue().intValue() + " moduleId " + moduleId + " portId " + portId)
  }
  def registerModule(name: String) = {
    //println("PCREPORT: module " + name + " registered for pc moduleId is " + moduleId)
    moduleIDs += (name -> moduleId)
    moduleId = moduleId + 1
    moduleId - 1
  }
}

trait TagTrait {
  val TAGWIDTH = 5
  def tagUpper(x: UInt) = ((x >> (TAGWIDTH).U) & (((1).U << (TAGWIDTH).U) - (1).U))
  def tagLower(x: UInt) = (x & (((1).U << (TAGWIDTH).U) - (1).U))
}

trait MemTrait {
  val ADDR_WIDTH = 32
  val DATA_WIDTH = 512
}

trait GorillaUtil extends TagTrait {
  //def updateElementsCache(bundle: Bundle, field: Data, fieldName: String): Unit = {
  //  if (bundle.elementsCache != null) {
  //  var i = bundle.elementsCache.findIndexOf(x => x._1 == fieldName)
  //    if (i == -1) {
  //      println("Gorilla++Error: bundle does not have field " + fieldName)
  //    }
  //    bundle.elementsCache.update(i, (fieldName, field))
  //  }
  //}

  def mergeOffloads(offloadData1: ArrayBuffer[(String, Data, Data)], offloadData2: ArrayBuffer[(String, Data, Data)]) = {
    val offNames1 = offloadData1.map((t) => t._1).toSet
    val offNames2 = offloadData2.map((t) => t._1).toSet
    val offloadData1Minus2 = offloadData1.filter((t) => {
      !offNames2.contains(t._1)
    })
    offloadData1Minus2 ++ offloadData2
  }

  def Chain(
    extCompName: String,
    aGen: gComponentGen[Data, Data],
    bGen: gComponentGen[Data, Data]
  ): gComponentGen[Data, Data] =
  {
    val newExtCompName = extCompName + "__type__chained__"
    val inData = aGen.inData
    val outData = bGen.outData
    val offloadData = mergeOffloads(aGen.offloadData, bGen.offloadData)

    new gComponentGen(
      new gChainedComponent(aGen.inData, aGen.outData, bGen.inData, bGen.outData, offloadData, aGen, bGen, newExtCompName),
      inData, outData, offloadData, newExtCompName
    )
  }
  def Chain(
    extCompName: String,
    aGen: gComponentGen[Data, Data],
    bGen: gComponentGen[Data, Data],
    cGen: gComponentGen[Data, Data]
  ): gComponentGen[Data, Data] =
  {
    Chain(extCompName + "__" + "Chained__2", Chain(extCompName + "__" + "Chained_1", aGen, bGen), cGen)
  }
  def Chain(
    extCompName: String, compGens: gComponentGen[Data, Data]*
  ): gComponentGen[Data, Data] =
  {
    var i = 0
    compGens.reduceLeft {(a, b) => { i += 1; Chain(extCompName + "__" + "Chained__" + i, a, b)} }
  }

  def Offload[
    inT <: Data, outT <: Data,
    inOffT<: Data, outOffT<: Data
  ](
    extCompName: String,
    mainGen: gComponentGen[inT, outT],
    offGen: gComponentGen[inOffT, outOffT],
    offPort: String
  ): gComponentGen[inT, outT] =
  {
    val (left, right) = mainGen.offloadData.partition(_._1 != offPort)

    //println("Offload called for port " + offPort)
    //println("main offlaod ports are")
    //mainGen.offloadData.foreach(x => println(x._1))
    //println("left offlaod ports are")
    //left.foreach(x => println(x._1))
    //println("right offlaod ports are")
    //right.foreach(x => println(x._1))

    val restOfOffloadedDataMain = {if (right.isEmpty) left else left ++ right.tail}

    //println("in Offload main offPorts is ")
    //mainGen.offloadData.foreach(x => println(x._1))
    //println("in Offload offload offPorts is ")
    //offGen.offloadData.foreach(x => println(x._1))

    val offOff = mergeOffloads(restOfOffloadedDataMain, offGen.offloadData)
    //println("merged offlaod ports are")
    //offOff.foreach(x => println(x._1))

    val newExtCompName = extCompName + "__type__offloaded__" + offPort

    new gComponentGen(
      new gOffloadedComponent(
        mainGen.inData, mainGen.outData,
        offGen.inData, offGen.outData,
        offOff,
        mainGen, offGen,
        offPort, newExtCompName
      ),
      mainGen.inData, mainGen.outData, offOff, newExtCompName
    )
  }
  def Offload[
    inT <: Data, outT <: Data,
    inReadOffT <: Data, inWriteOffT <: Data, outReadOffT <: Data, outWriteOffT <: Data
  ](
    extCompName: String,
    mainGen: gComponentGen[inT, outT],
    offGen: gRWComponentGen[inReadOffT, inWriteOffT, outReadOffT, outWriteOffT],
    offPort: String
  ): gComponentGen[inT, outT] =
  {
    val (leftRead, rightRead) = mainGen.offloadData.partition(_._1 == offPort + "Read")
    val (leftWrite, rightWrite) = rightRead.partition(_._1 == offPort + "Write")
    val offOff = rightWrite

    val newExtCompName = extCompName + "__type__offloaded__" + offPort

    new gComponentGen(
      new gOffloadedRWComponent(
        mainGen.inData, mainGen.outData,
        offGen.inReadData, offGen.inWriteData, offGen.outReadData, offGen.outWriteData,
        offOff,
        mainGen, offGen,
        offPort, newExtCompName
      ),
      mainGen.inData, mainGen.outData, offOff, newExtCompName
    )
  }
  //def Offload[
  //  inT <: Data, outT <: Data,
  //  inOffT <: Data, outOffT <: Data
  //](
  //  extCompName: String,
  //  mainGen: gComponentGen[inT, outT],
  //  offGens: (gComponentGen[inOffT, outOffT], String)*
  //): gComponentGen[inT, outT] =
  //{
  //  offGens.foldLeft(mainGen) { (x, y) => Offload(extCompName + "__type__offloaded__" + y._2, x, y._1, y._2) }
  //}
  //def Offload[inT <: Data, outT <: Data, offCompGenT <: gComponentGenBase](
  //  extCompName: String,
  //  mainGen: gComponentGen[inT, outT],
  //  offGens: ArrayBuffer[(offCompGenT, String)]
  //): off[inT, outT] =
  //{
  //  offGens.foldLeft(mainGen) { (x, y) => Offload(extCompName + "__type__offloaded__" + y._2, x, y._1, y._2) }
  //}
  def Offload[
    inT <: Data, outT <: Data,
    inOffT <: Data, outOffT <: Data
  ](
    extCompName: String,
    mainGen: gComponentGen[inT, outT],
    offGens: ArrayBuffer[(gComponentGen[inOffT, outOffT], String)]
  ): gComponentGen[inT, outT] =
  {
    offGens.foldLeft(mainGen) { (x, y) => Offload(extCompName + "__type__offloaded__" + y._2, x, y._1, y._2) }
  }
  def Offload[
    inT <: Data, outT <: Data,
    inReadOffT <: Data, inWriteOffT <: Data, outReadOffT <: Data, outWriteOffT <: Data
  ](
    extCompName: String,
    mainGen: gComponentGen[inT, outT],
    offGens: ArrayBuffer[(gRWComponentGen[inReadOffT, inWriteOffT, outReadOffT, outWriteOffT], String)],
    dummy: String = "rw"  // FIXME: hack so function signature is different from the other Offload array buffer after type erasure
  ): gComponentGen[inT, outT] =
  {
    //offGens.foldLeft(mainGen) { (x, y) => Offload(extCompName + "__type__offloaded__" + y._2, x, y._1, y._2) }
    val test = offGens.foldLeft(mainGen) { (x, y) => Offload(extCompName + "__type__offloaded__" + y._2, x, y._1, y._2) }
    println(test)
    return test
  }

  def Replicate[inT <: Data, outT <: Data, OffT <: Data](
    extCompName: String,
    compGen: gComponentGen[inT, outT],
    n: Int
  ): gComponentGen[inT, outT] =
  {
    val newExtCompName = extCompName + "__type__replicated__"

    new gComponentGen(
      new gReplicatedComponent(
        compGen.inData, compGen.outData, compGen.offloadData, compGen, n, newExtCompName
      ),
      compGen.inData, compGen.outData, compGen.offloadData, newExtCompName
    )
  }

  val broadcastDistribute = (255).U

  //def printChildren(p: Module) {
  //  p.children.foreach(c => {
  //    println("Design hierarchy --- parent " + p.name + " child " + c.name)
  //    printChildren(c)
  //  })
  //}

  //def printPC(p: Module) {
  //  if (compilerControl.pcEnable) {
  //    p.children.foreach(c => {
  //      if (p.isInstanceOf[gComponent[Data, Data]]) {
  //        println("Backpressure in module " + p.name + " is " + p.asInstanceOf[gComponent[Data, Data]].inPCBackPressure.litValue().intValue())
  //      }
  //      printPC(c)
  //    })
  //  }
  //}
}


import chisel3._

import chisel3.util._


import scala.collection.mutable.HashMap

import scala.collection.mutable.ArrayBuffer

import scala.collection.immutable.ListMap

import scala.util._


defined class DynamicBundle
defined object DynamicBundle
defined object gArbiterCtrl
defined class gioArbiter
defined class gioDistributor
defined class RREncode
defined class gRRArbiter
defined class gRRDistributor
defined class gTaggedRRArbiter
defined class gTaggedDistributor
defined class gFIFOIO
defined class gFIFOIOND
defined class gInOutBundle
defined class gRWInOutBundle
defined class gOffBundle
defined class gOffBundleND
defined class gInOutOffBundle
defined class gRWInOutOffBundle
defined class gMemBundle
defined trait include
defined class gComponentBase
defined class PcBundle
defined class gComponentLeaf
defined class gRWComponentLeaf
defined class gComponent
defined class gRWComponent
defined class gComponentGenBase
defined class gComponentGen
defined class gRWComponentGen
defined cl

## ALU block

In [7]:
import chisel3._
import chisel3.util._

class ALU(reg_width: Int) extends Module {
  val io = IO(new Bundle {
    val rs1       = Input(UInt(reg_width.W))
    val rs2       = Input(UInt(32.W))
    val addEn     = Input(Bool())
    val subEn     = Input(Bool())
    val sltEn     = Input(Bool())
    val sltuEn    = Input(Bool())
    val andEn     = Input(Bool())
    val orEn      = Input(Bool())
    val xorEn     = Input(Bool())
    val sllEn     = Input(Bool())
    val srEn      = Input(Bool())
    val srMode    = Input(Bool())
    val luiEn     = Input(Bool())
    val catEn     = Input(Bool())
    val immSel    = Input(Bool())
    val imm       = Input(SInt(32.W))
    val dout      = Output(UInt(reg_width.W))
  })

  val opA = Reg(SInt(32.W))
  val opB = Reg(SInt(32.W))
  val res = Wire(SInt(32.W))
  val rs1_r = Reg(UInt(reg_width.W))
  val addEn_r   = RegInit(false.B)
  val subEn_r   = RegInit(false.B)
  val sltEn_r   = RegInit(false.B)
  val sltuEn_r  = RegInit(false.B)
  val andEn_r   = RegInit(false.B)
  val orEn_r    = RegInit(false.B)
  val xorEn_r   = RegInit(false.B)
  val sllEn_r   = RegInit(false.B)
  val srEn_r    = RegInit(false.B)
  val srMode_r  = RegInit(false.B)
  val luiEn_r   = RegInit(false.B)
  val catEn_r   = RegInit(false.B)

  opA := io.rs1(31, 0).asSInt
  opB := Mux(io.immSel, io.imm, io.rs2(31, 0).asSInt)
  rs1_r := io.rs1
  addEn_r  := io.addEn
  subEn_r  := io.subEn
  sltEn_r  := io.sltEn
  sltuEn_r := io.sltuEn
  andEn_r  := io.andEn
  orEn_r   := io.orEn
  xorEn_r  := io.xorEn
  sllEn_r  := io.sllEn
  srEn_r   := io.srEn
  srMode_r := io.srMode
  luiEn_r  := io.luiEn
  res := DontCare

  when (addEn_r) {
    res := opA + opB
  } .elsewhen (subEn_r) {
    res := opA - opB
  } .elsewhen (sltEn_r) {
    when (opA < opB) {
      res := 1.S
    } .otherwise {
      res := 0.S
    }
  } .elsewhen(sltuEn_r) {
    val opA_u = opA.asUInt
    val opB_u = opB.asUInt
    when (opA_u < opB_u) {
      res := 1.S
    } .otherwise {
      res := 0.S
    }
  } .elsewhen (andEn_r) {
    res := opA & opB
  } .elsewhen (orEn_r) {
    res := opA | opB
  } .elsewhen (xorEn_r) {
    res := opA ^ opB
  } .elsewhen (sllEn_r) {
    val shamt = opB(4, 0).asUInt
    res := opA << shamt
  } .elsewhen (sllEn_r) {
    val shamt = opB(4, 0).asUInt
    when (srMode_r) {
      res := opA >> shamt
    } .otherwise {
      val opA_u = opA.asUInt
      res := (opA_u >> shamt).asSInt
    }
  } .elsewhen (luiEn_r) {
    res := opB
  } .elsewhen (catEn_r) {
    val opA_u = opA(8, 0).asUInt
    val opB_u = opB(8, 0).asUInt
    res := Cat(0.U(14.W), opB_u, opA_u).asSInt
  }

  io.dout := Cat(rs1_r(reg_width-1, 32), res.asUInt)

}


import chisel3._

import chisel3.util._


defined class ALU

## LSU

In [8]:
// lsu.scala
// IMPLEMENTS THE LOAD STORE UNIT. 
// Supports 32-bit loads, and object loads
// 
// Wraps a scratchpad. The scratchpad is implemented as SRAM blocks. 

import chisel3._
import chisel3.util._
import chisel3.util.Fill
import scala.math._


class loadStoreUnit(tag_width: Int, reg_width: Int, opcode_width: Int, num_threads: Int, ip_width: Int) extends Module {
  val io = IO(new Bundle{
    val in_valid      = Input(Bool())
    val in_tag        = Input(UInt(tag_width.W))
    val in_opcode     = Input(UInt(opcode_width.W))
    val in_imm        = Input(UInt(12.W))
    val in_bits       = Input(Vec(2, UInt(reg_width.W)))
    val in_ready      = Output(Bool())
    val out_valid     = Output(Bool())
    val out_tag       = Output(UInt(tag_width.W))
    val out_flag      = Output(UInt(ip_width.W))
    val out_bits      = Output(UInt(reg_width.W))
    val out_ready     = Input(Bool())

    val mem           = new gMemBundle

  })

  val MEM_WIDTH = 256
  val NUM_BLOCKS = MEM_WIDTH >> 5
  val RATIO = (math.floor(reg_width / MEM_WIDTH)).toInt
  println("LSU inited with params: ")
  println(" NUM_BLOCKS: " + NUM_BLOCKS)
  println(" RATIO: " + RATIO)
  assert(RATIO * MEM_WIDTH == reg_width, "Register file width must be multiple of MEM_WIDTH")
 
  val MEM_SIZE = 512
  val ADDR_WIDTH_B = log2Up(MEM_SIZE) + log2Up(MEM_WIDTH) - 3
  val ADDR_WIDTH_W = ADDR_WIDTH_B + 2

  val mem = Seq.fill(NUM_BLOCKS)(SyncReadMem(MEM_SIZE, UInt(MEM_WIDTH.W)))
  val mem_addr = RegInit(0.U(ADDR_WIDTH_B.W))
  val mem_index = Wire(UInt(log2Up(MEM_SIZE).W))
  val mem_offset = Wire(UInt(log2Up(MEM_WIDTH).W))
  val mem_wr = RegInit(false.B)
  val valid_r = RegInit(false.B)
  val tag_r = Reg(UInt(tag_width.W))
  val base_addr = Wire(SInt(32.W))
  val disp = Wire(SInt(32.W))
  val wr_data_obj = Reg(Vec(RATIO, UInt(MEM_WIDTH.W)))
  val wr_data = Reg(Vec(NUM_BLOCKS, UInt(32.W)))
  val readmeta = RegInit(false.B)
  val isObj = RegInit(false.B)

  io.mem.mem_addr   := DontCare
  io.mem.read       := false.B
  io.mem.write      := false.B
  io.mem.writedata  := DontCare
  io.mem.byteenable := DontCare

  io.out_flag := 0.U
  base_addr := io.in_bits(0)(31, 0).asSInt
  disp := io.in_imm.asSInt

  // Calculate address
  val mem_state = RegInit(0.U(1.W))
  val counter = RegInit(0.U(log2Up(RATIO).W))
  io.in_ready := false.B
  when (mem_state === 0.U) {
    io.in_ready := io.out_ready
    when (io.out_ready) {
      valid_r := io.in_valid
    }
    when (io.in_valid && io.out_ready) {
      when (io.in_opcode(2, 0) === 1.U) {
        (0 until NUM_BLOCKS).map(i => wr_data(i) := io.in_bits(1)(i*32+31, i*32))
        isObj := true.B
        when (counter < (RATIO-1).U) {
          counter := counter + 1.U
          mem_state := 1.U
        }
      } .otherwise {
        (0 until NUM_BLOCKS).map(i => wr_data(i) := io.in_bits(1)(31, 0))
        isObj := false.B
      }
      tag_r := io.in_tag
      mem_wr := io.in_opcode(4)
      (0 until RATIO).map(i => wr_data_obj(i) := io.in_bits(1)(i*MEM_WIDTH+MEM_WIDTH-1, i*MEM_WIDTH))
      val addr = base_addr + disp
      mem_addr := addr.asUInt
      when (addr === -1.S) {
        readmeta := true.B
      } .otherwise {
        readmeta := false.B
      }
    }
  } .otherwise {
    // multicycle load/store object
    when (io.out_ready) {
      mem_addr := mem_addr + (MEM_WIDTH >> 3).U
      for (i <- 0 until NUM_BLOCKS) {
        val cases = (0 until RATIO).map(x => (x.U === counter) -> wr_data_obj(x)(i*32+31, i*32))
        wr_data(i) := MuxCase(DontCare, cases)
      }
      //(0 until NUM_BLOCKS).map(i => wr_data(i) := wr_data_obj(counter*MEM_WIDTH+i*32+31, counter*MEM_WIDTH+i*32))
      when (counter < (RATIO-1).U) {
        counter := counter + 1.U
      } .otherwise {
        counter := 0.U
        mem_state := 0.U
      }
    }

  }
  mem_offset := mem_addr(log2Up(MEM_WIDTH)-4, 2)
  mem_index := mem_addr(ADDR_WIDTH_B-1, log2Up(MEM_WIDTH)-3)

  // Mem read/write
  val rd_data = Wire(Vec(NUM_BLOCKS, UInt(32.W)))
  val counter_r = Reg(UInt(log2Up(RATIO).W))
  val valid_r2 = RegInit(false.B)
  val isObj_r = RegInit(false.B)
  val tag_r2 = Reg(UInt(tag_width.W))
  val readmeta_r = RegInit(false.B)
  val mem_offset_r = Reg(UInt(log2Up(MEM_WIDTH).W))
  rd_data := DontCare
  io.out_bits := DontCare
  when (io.out_ready) {
    valid_r2 := false.B
    counter_r := counter
    isObj_r := isObj
    mem_offset_r := mem_offset
    tag_r2 := tag_r
    readmeta_r := readmeta
    when (valid_r) {
      for (i <- 0 until NUM_BLOCKS) {
        val rdwrPort = mem(i)(mem_index)
        when (mem_wr) {
          when (mem_offset === i.U) {
            rdwrPort := wr_data(i)
          }
        } .otherwise {
          when (io.out_ready) {
            rd_data(i) := rdwrPort
          }
        }
      }
      valid_r2 := true.B
    }
  }

  // Extract word
  val valid_out = RegInit(false.B)
  val rd_data_obj = Reg(Vec(RATIO, UInt(MEM_WIDTH.W)))
  val tag_out = Reg(UInt(tag_width.W))
  val readmeta_out = RegInit(false.B)
  when (io.out_ready) {
    valid_out := false.B
    readmeta_out := readmeta_r
    when (valid_r2) {
      tag_out := tag_r
      when (isObj_r) {
        rd_data_obj(counter_r) := rd_data.asUInt
        when (counter_r === (RATIO-1).U) {
          valid_out := true.B
        }
      } .otherwise {
        rd_data_obj(0) := rd_data(mem_offset_r)
        valid_out := true.B
      }
    }
  }

  when (readmeta_out) {
    io.out_bits := tag_out
  } .otherwise {
    io.out_bits := rd_data_obj.asUInt
  }

  io.out_valid := valid_out
  io.out_tag := tag_out

}

import chisel3._

import chisel3.util._

import chisel3.util.Fill

import scala.math._



defined class loadStoreUnit

## alu_bfu0

In [9]:
import chisel3._
import chisel3.util._

class alu_bfu0(tag_width: Int, reg_width: Int, opcode_width: Int, num_threads: Int, ip_width: Int) extends Module {
  val io = IO(new Bundle {
    val in_valid      = Input(Bool())
    val in_alu_bfu    = Input(Bool())
    val in_tag        = Input(UInt(tag_width.W))
    val in_opcode     = Input(UInt(opcode_width.W))
    val in_ready      = Output(Bool())
    val in_rs1        = Input(UInt(reg_width.W))
    val in_rs2        = Input(UInt(32.W))
    val in_addEn      = Input(Bool())
    val in_subEn      = Input(Bool())
    val in_sltEn      = Input(Bool())
    val in_sltuEn     = Input(Bool())
    val in_andEn      = Input(Bool())
    val in_orEn       = Input(Bool())
    val in_xorEn      = Input(Bool())
    val in_sllEn      = Input(Bool())
    val in_srEn       = Input(Bool())
    val in_srMode     = Input(Bool())
    val in_luiEn      = Input(Bool())
    val in_catEn      = Input(Bool())
    val in_immSel     = Input(Bool())
    val in_imm        = Input(SInt(32.W))
    val out_valid     = Output(Bool())
    val out_tag       = Output(UInt(tag_width.W))
    val out_flag      = Output(UInt(32.W))
    val out_bits      = Output(UInt(reg_width.W))
    val out_ready     = Input(Bool())

    val mem           = new gMemBundle
  })

  val aluInst = Module(new ALU(reg_width))
  val bfuInst = Module(new loadStoreUnit(tag_width, reg_width, opcode_width, num_threads, ip_width))
  val alu_valid_d0 = RegInit(false.B)
  val tag_r = RegNext(io.in_tag)
  
  // in_alu_bfu: 0: select alu; 1: select bfu
  alu_valid_d0      := io.in_valid && (!io.in_alu_bfu)
  aluInst.io.rs1    := io.in_rs1   
  aluInst.io.rs2    := io.in_rs2   
  aluInst.io.addEn  := io.in_addEn 
  aluInst.io.subEn  := io.in_subEn 
  aluInst.io.sltEn  := io.in_sltEn 
  aluInst.io.sltuEn := io.in_sltuEn
  aluInst.io.andEn  := io.in_andEn 
  aluInst.io.orEn   := io.in_orEn  
  aluInst.io.xorEn  := io.in_xorEn 
  aluInst.io.sllEn  := io.in_sllEn 
  aluInst.io.srEn   := io.in_srEn  
  aluInst.io.srMode := io.in_srMode
  aluInst.io.luiEn  := io.in_luiEn 
  aluInst.io.catEn  := io.in_catEn 
  aluInst.io.immSel := io.in_immSel
  aluInst.io.imm    := io.in_imm   

  bfuInst.io.in_valid   := io.in_valid && io.in_alu_bfu
  bfuInst.io.in_tag     := io.in_tag
  bfuInst.io.in_opcode  := io.in_opcode
  bfuInst.io.in_imm     := io.in_imm.asUInt
  bfuInst.io.in_bits(0) := io.in_rs1
  bfuInst.io.in_bits(1) := io.in_rs2

  io.mem.mem_addr   := bfuInst.io.mem.mem_addr
  io.mem.read       := bfuInst.io.mem.read
  io.mem.write      := bfuInst.io.mem.write
  io.mem.writedata  := bfuInst.io.mem.writedata
  io.mem.byteenable := bfuInst.io.mem.byteenable
  bfuInst.io.mem.waitrequest    := io.mem.waitrequest
  bfuInst.io.mem.readdatavalid  := io.mem.readdatavalid
  bfuInst.io.mem.readdata       := io.mem.readdata

  when (io.in_alu_bfu) {
    io.in_ready := bfuInst.io.in_ready
  } .otherwise {
    io.in_ready := true.B
  }

  when (alu_valid_d0) {
    io.out_valid := true.B
    io.out_tag := tag_r
    io.out_flag := aluInst.io.dout
    io.out_bits := aluInst.io.dout
    bfuInst.io.out_ready := false.B
  } .otherwise {
    io.out_valid := bfuInst.io.out_valid
    io.out_tag := bfuInst.io.out_tag
    io.out_flag := (bfuInst.io.out_flag << 2)
    io.out_bits := bfuInst.io.out_bits
    bfuInst.io.out_ready := true.B
  }

}


import chisel3._

import chisel3.util._


defined class alu_bfu0

## alu_bfu1

In [10]:
import chisel3._
import chisel3.util._

class alu_bfu1(tag_width: Int, reg_width: Int, opcode_width: Int, num_threads: Int, ip_width: Int) extends Module {
  val io = IO(new Bundle {
    val in_valid      = Input(Bool())
    val in_alu_bfu    = Input(Bool())
    val in_tag        = Input(UInt(tag_width.W))
    val in_opcode     = Input(UInt(opcode_width.W))
    val in_ready      = Output(Bool())
    val in_rs1        = Input(UInt(reg_width.W))
    val in_rs2        = Input(UInt(32.W))
    val in_addEn      = Input(Bool())
    val in_subEn      = Input(Bool())
    val in_sltEn      = Input(Bool())
    val in_sltuEn     = Input(Bool())
    val in_andEn      = Input(Bool())
    val in_orEn       = Input(Bool())
    val in_xorEn      = Input(Bool())
    val in_sllEn      = Input(Bool())
    val in_srEn       = Input(Bool())
    val in_srMode     = Input(Bool())
    val in_luiEn      = Input(Bool())
    val in_catEn      = Input(Bool())
    val in_immSel     = Input(Bool())
    val in_imm        = Input(SInt(32.W))
    val out_valid     = Output(Bool())
    val out_tag       = Output(UInt(tag_width.W))
    val out_flag      = Output(UInt(32.W))
    val out_bits      = Output(UInt(reg_width.W))
    val out_ready     = Input(Bool())

    val mem           = new gMemBundle
  })

  val aluInst = Module(new ALU(reg_width))
  val alu_valid_d0 = RegInit(false.B)
  val tag_r = RegNext(io.in_tag)
  
  // in_alu_bfu: 0: select alu; 1: select bfu
  alu_valid_d0      := io.in_valid && (!io.in_alu_bfu)
  aluInst.io.rs1    := io.in_rs1   
  aluInst.io.rs2    := io.in_rs2   
  aluInst.io.addEn  := io.in_addEn 
  aluInst.io.subEn  := io.in_subEn 
  aluInst.io.sltEn  := io.in_sltEn 
  aluInst.io.sltuEn := io.in_sltuEn
  aluInst.io.andEn  := io.in_andEn 
  aluInst.io.orEn   := io.in_orEn  
  aluInst.io.xorEn  := io.in_xorEn 
  aluInst.io.sllEn  := io.in_sllEn 
  aluInst.io.srEn   := io.in_srEn  
  aluInst.io.srMode := io.in_srMode
  aluInst.io.luiEn  := io.in_luiEn 
  aluInst.io.catEn  := io.in_catEn 
  aluInst.io.immSel := io.in_immSel
  aluInst.io.imm    := io.in_imm   

  io.mem.mem_addr   := DontCare
  io.mem.read       := false.B
  io.mem.write      := false.B
  io.mem.writedata  := DontCare
  io.mem.byteenable := 0.U

  io.in_ready := true.B

  io.out_valid := alu_valid_d0
  io.out_tag := tag_r
  io.out_flag := aluInst.io.dout
  io.out_bits := aluInst.io.dout

}


import chisel3._

import chisel3.util._


defined class alu_bfu1

In [11]:
class ALU(reg_width: Int) extends Module {
  val io = IO(new Bundle {
    val rs1       = Input(UInt(reg_width.W))
    val rs2       = Input(UInt(32.W))
    val addEn     = Input(Bool())
    val subEn     = Input(Bool())
    val sltEn     = Input(Bool())
    val sltuEn    = Input(Bool())
    val andEn     = Input(Bool())
    val orEn      = Input(Bool())
    val xorEn     = Input(Bool())
    val sllEn     = Input(Bool())
    val srEn      = Input(Bool())
    val srMode    = Input(Bool())
    val luiEn     = Input(Bool())
    val catEn     = Input(Bool())
    val immSel    = Input(Bool())
    val imm       = Input(SInt(32.W))
    val dout      = Output(UInt(reg_width.W))
  })

  val opA = Reg(SInt(32.W))
  val opB = Reg(SInt(32.W))
  val res = Wire(SInt(32.W))
  val rs1_r = Reg(UInt(reg_width.W))
  val addEn_r   = RegInit(false.B)
  val subEn_r   = RegInit(false.B)
  val sltEn_r   = RegInit(false.B)
  val sltuEn_r  = RegInit(false.B)
  val andEn_r   = RegInit(false.B)
  val orEn_r    = RegInit(false.B)
  val xorEn_r   = RegInit(false.B)
  val sllEn_r   = RegInit(false.B)
  val srEn_r    = RegInit(false.B)
  val srMode_r  = RegInit(false.B)
  val luiEn_r   = RegInit(false.B)
  val catEn_r   = RegInit(false.B)

  opA := io.rs1(31, 0).asSInt
  opB := Mux(io.immSel, io.imm, io.rs2(31, 0).asSInt)
  rs1_r := io.rs1
  addEn_r  := io.addEn
  subEn_r  := io.subEn
  sltEn_r  := io.sltEn
  sltuEn_r := io.sltuEn
  andEn_r  := io.andEn
  orEn_r   := io.orEn
  xorEn_r  := io.xorEn
  sllEn_r  := io.sllEn
  srEn_r   := io.srEn
  srMode_r := io.srMode
  luiEn_r  := io.luiEn
  res := DontCare

  when (addEn_r) {
    res := opA + opB
  } .elsewhen (subEn_r) {
    res := opA - opB
  } .elsewhen (sltEn_r) {
    when (opA < opB) {
      res := 1.S
    } .otherwise {
      res := 0.S
    }
  } .elsewhen(sltuEn_r) {
    val opA_u = opA.asUInt
    val opB_u = opB.asUInt
    when (opA_u < opB_u) {
      res := 1.S
    } .otherwise {
      res := 0.S
    }
  } .elsewhen (andEn_r) {
    res := opA & opB
  } .elsewhen (orEn_r) {
    res := opA | opB
  } .elsewhen (xorEn_r) {
    res := opA ^ opB
  } .elsewhen (sllEn_r) {
    val shamt = opB(4, 0).asUInt
    res := opA << shamt
  } .elsewhen (sllEn_r) {
    val shamt = opB(4, 0).asUInt
    when (srMode_r) {
      res := opA >> shamt
    } .otherwise {
      val opA_u = opA.asUInt
      res := (opA_u >> shamt).asSInt
    }
  } .elsewhen (luiEn_r) {
    res := opB
  } .elsewhen (catEn_r) {
    val opA_u = opA(8, 0).asUInt
    val opB_u = opB(8, 0).asUInt
    res := Cat(0.U(14.W), opB_u, opA_u).asSInt
  }

  io.dout := Cat(rs1_r(reg_width-1, 32), res.asUInt)

}


defined class ALU

## shifter and inputUnit_core

In [12]:
class pkt_buf_t(num_threads: Int) extends Bundle {
  val data = UInt(512.W)
  val tag = UInt(log2Up(num_threads).W)
  val empty = UInt(7.W)
  val last = Bool()

  override def cloneType = (new pkt_buf_t(num_threads).asInstanceOf[this.type])
}

// BFU regfile dump interface
class BFU_regfile_req_t(num_threads_lg: Int, num_regs_lg: Int) extends Bundle {
  val tag = UInt(num_threads_lg.W)
  val rdAddr1 = UInt(num_regs_lg.W)
  val rdAddr2 = UInt(num_regs_lg.W)

  override def cloneType = (new BFU_regfile_req_t(num_threads_lg, num_regs_lg).asInstanceOf[this.type])
}

class BFU_regfile_rsp_t(reg_width: Int) extends Bundle {
  val rdData1 = UInt(reg_width.W)
  val rdData2 = UInt(reg_width.W)

  override def cloneType = (new BFU_regfile_rsp_t(reg_width).asInstanceOf[this.type])
}

class Fetch(num: Int, ipWidth: Int, instrWidth: Int, inst_ram_size: Int) extends Module {
  val io = IO(new Bundle {
    val ip         = Input(UInt(ipWidth.W))
    val instr      = Output(UInt(instrWidth.W))
  })

  // FIXME: implement i$

  println("fetch params: ")
  println(" INSTR WIDTH: " + instrWidth)
  println(" NUM_ISNTRS:  " + inst_ram_size)

  val mem = SyncReadMem(inst_ram_size, UInt(instrWidth.W))
  loadMemoryFromFileInline(mem, "./primate_pgm.bin")

  io.instr := mem(io.ip)
}

defined class pkt_buf_t
defined class BFU_regfile_req_t
defined class BFU_regfile_rsp_t
defined class Fetch

In [13]:
import chisel3._
import chisel3.util._

class Shifter(num_bytes: Int) extends Module {
  val io = IO(new Bundle{
    val fifo_valid       = Input(Bool())
    val fifo_in          = Input(Vec(num_bytes, UInt(8.W)))
    val fifo_in_last     = Input(Bool())
    val fifo_in_empty    = Input(UInt(log2Up(num_bytes).W))
    val shift_read_width = Input(UInt((log2Up(num_bytes)+1).W))
    val in_valid         = Input(Bool())
    val shift_en         = Input(Bool())
    val read_en          = Input(Bool())
    val clear            = Input(Bool())
    val in_ready         = Output(Bool())
    val fifo_ready       = Output(Bool())
    val out_valid        = Output(Bool())
    val out_data         = Output(UInt((num_bytes*8).W))
    val buf_data         = Output(UInt((num_bytes*8).W))
    val buf_length       = Output(UInt((log2Up(num_bytes)+1).W))
    val buf_last         = Output(Bool())
  })

  val buf_r = Reg(Vec(num_bytes, UInt(8.W)))
  val fifo_in_r = Reg(Vec(num_bytes, UInt(8.W)))
  val out_buf = Reg(Vec(num_bytes, UInt(8.W)))
  val length_buf = RegInit(0.U((log2Up(num_bytes)+1).W))
  val empty_fifo = RegInit(0.U((log2Up(num_bytes)+1).W))
  val valid_r = RegInit(false.B)
  val buf_out = Wire(Vec(num_bytes, UInt(8.W)))
  val shift = Wire(UInt((log2Up(num_bytes)+1).W))
  val remain_r = Reg(UInt((log2Up(num_bytes)+1).W))
  val fifo_shift_r = Reg(SInt((log2Up(num_bytes)+1).W))
  val shift_r = RegInit(0.U((log2Up(num_bytes)+1).W))
  val isLast = RegInit(false.B)

  shift_r := shift
  remain_r := length_buf - shift
  fifo_in_r := io.fifo_in
  fifo_shift_r := length_buf.asSInt - shift.asSInt - empty_fifo.asSInt
  for (i <- 0 until num_bytes) {
    when (i.U < remain_r) {
      buf_out(i) := buf_r(i.U + shift_r)
    } .otherwise {
      buf_out(i) := fifo_in_r((i.S - fifo_shift_r).asUInt)
    }
  }

  val shift_state = RegInit(0.U(1.W))

  io.fifo_ready := false.B
  io.out_valid := false.B
  valid_r := false.B
  shift := 0.U
  io.in_ready := false.B
  when (shift_state === 0.U) {
    when (io.in_valid && (io.shift_read_width <= length_buf)) {
      io.in_ready := true.B
      valid_r := true.B
      out_buf := buf_r
      when (io.shift_en) {
        shift_state := 1.U
        shift := io.shift_read_width
        when (io.fifo_valid) {
          when (length_buf - io.shift_read_width <= empty_fifo + io.fifo_in_empty) {
            length_buf := length_buf - io.shift_read_width + 32.U - empty_fifo - io.fifo_in_empty
            empty_fifo := 0.U
            isLast := io.fifo_in_last
            io.fifo_ready := true.B
          } .otherwise {
            length_buf := 32.U
            empty_fifo := empty_fifo + (32.U - length_buf + io.shift_read_width)
          }
        } .otherwise {
          length_buf := length_buf - io.shift_read_width
        }
      }
    } .elsewhen(!io.clear) {
      shift := 0.U
      valid_r := false.B
      io.in_ready := false.B
      when (io.fifo_valid) {
        when (length_buf =/= 32.U) {
          shift_state := 1.U
        }
        when (length_buf <= empty_fifo + io.fifo_in_empty) {
          length_buf := length_buf + 32.U - empty_fifo - io.fifo_in_empty
          empty_fifo := 0.U
          isLast := io.fifo_in_last
          io.fifo_ready := true.B
        } .otherwise {
          length_buf := 32.U
          empty_fifo := empty_fifo + (32.U - length_buf)
        }
      }
    } .otherwise {
      isLast := false.B
      length_buf := 0.U
      empty_fifo := 0.U
    }
  } .otherwise {
    buf_r := buf_out
    shift_state := 0.U
  }

  io.out_valid := valid_r
  io.out_data := out_buf.asUInt
  io.buf_data := buf_r.asUInt
  io.buf_length := length_buf - empty_fifo
  io.buf_last := isLast

}

class inputUnit_core(reg_width: Int, num_regs_lg: Int, opcode_width: Int, num_threads: Int) extends Module {
  val io = IO(new Bundle {
    val in_valid     = Input(Bool())
    val in_tag       = Input(UInt(log2Up(num_threads).W))
    val in_data      = Input(UInt(512.W))
    val in_empty     = Input(UInt(6.W))
    val in_last      = Input(Bool())
    val in_ready     = Output(Bool())

    val out_ready    = Input(Bool())
    val out_valid    = Output(Bool())
    val out_tag      = Output(UInt(log2Up(num_threads).W))
    val out_wen      = Output(Bool())
    val out_addr     = Output(UInt(num_regs_lg.W))
    val out_data     = Output(UInt(reg_width.W))

    val ar_valid     = Input(Bool())
    val ar_tag       = Input(UInt(log2Up(num_threads).W))
    val ar_opcode    = Input(UInt(opcode_width.W))
    val ar_rd        = Input(UInt(num_regs_lg.W))
    val ar_bits      = Input(UInt(32.W))
    val ar_imm       = Input(UInt(32.W))
    val ar_ready     = Output(Bool())

    // To output unit
    val pkt_buf_data  = Output(new pkt_buf_t(num_threads))
    val pkt_buf_valid = Output(Bool())
    val pkt_buf_ready = Input(Bool())
  })
  // opcode(0) = 0: not write back, 1: write back
  // opcode(1) = 0: not shift, 1: shift
  // opcode(2) = 0: parse not done, 1: parse done
  // opcode(3) = 0: select imm, 1: select reg

  val parseFifo_t = new Bundle {
    val bits = UInt(256.W)
    val empty = UInt(6.W)
    val last = Bool()
  }

  val parseFifo = Module(new Queue(parseFifo_t, 2))
  val parseDone = RegInit(false.B)
  val parseFifoEnqState = RegInit(0.U(1.W))
  val shifter = Module(new Shifter(32))
  val length_buf = RegInit(0.U(7.W))
  val length_fifo = RegInit(0.U(7.W))
  val isLast = RegInit(false.B)

  io.in_ready := false.B
  parseFifo.io.enq.valid := false.B
  parseFifo.io.enq.bits := DontCare
  when (!parseDone || (parseFifoEnqState === 1.U)) {
    when (io.in_valid && parseFifo.io.enq.ready) {
      when (parseFifoEnqState === 0.U) {
        parseFifo.io.enq.valid := true.B
        parseFifo.io.enq.bits.bits := io.in_data(255, 0)
        when (io.in_empty >= 32.U) {
          parseFifo.io.enq.bits.last := io.in_last
          parseFifo.io.enq.bits.empty := io.in_empty - 32.U
        } .otherwise {
          parseFifo.io.enq.bits.last := false.B
          parseFifo.io.enq.bits.empty := 0.U
        }
        parseFifoEnqState := 1.U
      } .otherwise {
        parseFifo.io.enq.bits.bits := io.in_data(511, 256)
        parseFifo.io.enq.bits.empty := io.in_empty
        when (io.in_empty < 32.U) {
          parseFifo.io.enq.valid := true.B
        }
        parseFifo.io.enq.bits.last := io.in_last
        io.in_ready := true.B
        parseFifoEnqState := 0.U
      }
    }
  }

  val seek_valid = RegInit(false.B)
  val seekState = RegInit(0.U(3.W))
  val tag = Reg(UInt(log2Up(num_threads).W))
  val addr = Reg(UInt(num_regs_lg.W))
  val headerLen = Reg(UInt(6.W))
  val opcode = Reg(UInt(opcode_width.W))
  val fillEn = Reg(Bool())
  val pktFifo_in_r = Reg(UInt(256.W))
  val pktFifo_empty_r = Reg(UInt(6.W))
  val lastSeen = RegInit(false.B)

  shifter.io.fifo_in := parseFifo.io.deq.bits.bits.asTypeOf(chiselTypeOf(shifter.io.fifo_in))
  shifter.io.fifo_in_last := parseFifo.io.deq.bits.last
  shifter.io.fifo_in_empty := parseFifo.io.deq.bits.empty
  shifter.io.fifo_valid := parseFifo.io.deq.valid
  parseFifo.io.deq.ready := shifter.io.fifo_ready
  shifter.io.shift_read_width := 0.U
  shifter.io.in_valid := false.B
  shifter.io.shift_en := false.B
  shifter.io.read_en := false.B
  shifter.io.clear := false.B

  io.out_valid := false.B
  io.out_wen := false.B
  io.out_tag := tag
  io.out_addr := addr
  io.out_data := shifter.io.out_data
  io.ar_ready := false.B
  io.pkt_buf_valid := false.B
  io.pkt_buf_data := DontCare
  when (seekState === 0.U) {
    // IDLE
    io.ar_ready := true.B
    when (io.ar_valid) {
      tag := io.ar_tag
      addr := io.ar_rd
      // val ar_src = Mux(io.ar_opcode(3).asBool, io.ar_bits, io.ar_imm)
      val ar_src = io.ar_imm + io.ar_bits
      headerLen := ar_src(5, 0)
      opcode := io.ar_opcode
      shifter.io.shift_read_width := ar_src(5, 0)
      shifter.io.shift_en := io.ar_opcode(1).asBool
      shifter.io.read_en := io.ar_opcode(0).asBool
      when (io.ar_opcode(1, 0) =/= 0.U) {
        shifter.io.in_valid := true.B
        when (shifter.io.in_ready) {
          seekState := 2.U
        } .otherwise {
          seekState := 1.U
        }
      } .elsewhen (io.ar_opcode(2) === 1.U) {
        parseDone := true.B
        seekState := 4.U
      }
    }
  } .elsewhen (seekState === 1.U) {
    // Resume
    shifter.io.shift_read_width := headerLen
    shifter.io.shift_en := opcode(1).asBool
    shifter.io.read_en := opcode(0).asBool
    shifter.io.in_valid := true.B
    when (shifter.io.in_ready) {
      seekState := 2.U
    }
  } .elsewhen (seekState === 2.U) {
    // Output, shift
    when (opcode(0) === 1.U) {
      io.out_wen := true.B
    }
    when (opcode(0) === 0.U || io.out_ready) {
      when (opcode(2) === 1.U) {
        parseDone := true.B
        seekState := 4.U
      } .otherwise {
        io.out_valid := true.B
        seekState := 0.U
      }
    }
  } .elsewhen (seekState === 4.U) {
    // dump out the remaining data in the buffer
    val pktFifo_in = Wire(new pkt_buf_t(num_threads))
    pktFifo_in.data := shifter.io.buf_data
    pktFifo_in.last := shifter.io.buf_last
    pktFifo_in.tag := tag
    pktFifo_in.empty := 64.U - shifter.io.buf_length
    when (shifter.io.buf_length =/= 0.U) {
      io.pkt_buf_valid := true.B
    }
    io.pkt_buf_data := pktFifo_in
    when (io.pkt_buf_ready) {
      shifter.io.clear := true.B
      when (shifter.io.buf_last) {
        io.out_valid := true.B
        parseDone := false.B
        seekState := 0.U
      } .otherwise {
        seekState := 5.U
      }
    }
  } .elsewhen (seekState === 5.U) {
    // drain parseFifo
    shifter.io.clear := true.B
    when (parseFifo.io.deq.valid) {
      pktFifo_in_r := parseFifo.io.deq.bits.bits
      pktFifo_empty_r := parseFifo.io.deq.bits.empty
      parseFifo.io.deq.ready := true.B
      seekState := 6.U
      when (parseFifo.io.deq.bits.last) {
        lastSeen := true.B
      }
    } .otherwise {
      seekState := 7.U
    }
  } .elsewhen (seekState === 6.U) {
    shifter.io.clear := true.B
    when (lastSeen || (!parseFifo.io.deq.valid) || (pktFifo_empty_r =/= 0.U)) {
      io.pkt_buf_data.data := pktFifo_in_r
      io.pkt_buf_data.tag := tag
      io.pkt_buf_data.last := lastSeen
      io.pkt_buf_data.empty := 32.U + pktFifo_empty_r
      io.pkt_buf_valid := true.B
      when (io.pkt_buf_ready) {
        when (lastSeen) {
          io.out_valid := true.B
          parseDone := false.B
          lastSeen := false.B
          seekState := 0.U
        } .elsewhen (!parseFifo.io.deq.valid) {
          seekState := 7.U
        } .otherwise {
          seekState := 5.U
        }
      }
    } .elsewhen (parseFifo.io.deq.valid) {
      io.pkt_buf_data.data := Cat(parseFifo.io.deq.bits.bits, pktFifo_in_r)
      io.pkt_buf_data.tag := tag
      io.pkt_buf_data.last := parseFifo.io.deq.bits.last
      io.pkt_buf_data.empty := pktFifo_empty_r
      io.pkt_buf_valid := true.B
      when (io.pkt_buf_ready) {
        parseFifo.io.deq.ready := true.B
        when (parseFifo.io.deq.bits.last) {
          io.out_valid := true.B
          parseDone := false.B
          seekState := 0.U
        } .otherwise {
          seekState := 5.U
        }
      }
    }
  } .elsewhen (seekState === 7.U) {
    // drain remaining flits
    shifter.io.clear := true.B
    io.in_ready := io.pkt_buf_ready
    when (io.in_valid) {
      io.pkt_buf_data.data := io.in_data
      io.pkt_buf_data.tag := tag
      io.pkt_buf_data.last := io.in_last
      io.pkt_buf_data.empty := io.in_empty
      io.pkt_buf_valid := true.B
      when (io.pkt_buf_ready) {
        when (io.in_last) {
          io.out_valid := true.B
          parseDone := false.B
          seekState := 0.U
        }
      }
    }
  }

}

import chisel3._

import chisel3.util._


defined class Shifter
defined class inputUnit_core

## inputUnit

In [14]:
import chisel3._
import chisel3.util._

class inputUnit(reg_width: Int, num_regs_lg: Int, opcode_width: Int, num_threads: Int, ip_width: Int) extends Module {
  val io = IO(new Bundle {
    val in_valid     = Input(Bool())
    val in_tag       = Input(UInt(log2Up(num_threads).W))
    val in_data      = Input(UInt(512.W))
    val in_empty     = Input(UInt(6.W))
    val in_last      = Input(Bool())
    val in_ready     = Output(Bool())

    val out_ready    = Input(Bool())
    val out_valid    = Output(Bool())
    val out_tag      = Output(UInt(log2Up(num_threads).W))
    val out_flag     = Output(UInt(ip_width.W))
    val out_wen      = Output(Vec(2, Bool()))
    val out_addr     = Output(Vec(2, UInt(num_regs_lg.W)))
    val out_data     = Output(Vec(2, UInt(reg_width.W)))

    val idle_threads = Input(Vec(num_threads, Bool()))
    val new_thread   = Output(Bool())
    val new_tag      = Output(UInt(log2Up(num_threads).W))

    val ar_valid     = Input(Bool())
    val ar_tag       = Input(UInt(log2Up(num_threads).W))
    val ar_opcode    = Input(UInt(opcode_width.W))
    val ar_rd        = Input(UInt(num_regs_lg.W))
    val ar_bits      = Input(UInt(32.W))
    val ar_imm       = Input(UInt(32.W))
    val ar_ready     = Output(Bool())

    // To output unit
    val pkt_buf_data  = Output(new pkt_buf_t(num_threads))
    val pkt_buf_valid = Output(Bool())
    val pkt_buf_ready = Input(Bool())
  })
  // opcode(0) = 0: not write back, 1: write back
  // opcode(1) = 0: not shift, 1: shift
  // opcode(2) = 0: parse not done, 1: parse done
  // opcode(3) = 0: select imm, 1: select reg

  val PKT_BUF_DEPTH = 512
  val pktFifo = Module(new Queue(new pkt_buf_t(num_threads), PKT_BUF_DEPTH))

  val threadState = RegInit(0.U(1.W))
  val sThreadEncoder = Module(new RREncode(num_threads))
  val sThread = sThreadEncoder.io.chosen

  sThreadEncoder.io.valid := io.idle_threads
  sThreadEncoder.io.ready := sThread =/= (num_threads.U)
  io.new_thread := false.B
  io.new_tag := sThread
  when (threadState === 0.U) {
    when (io.in_valid && io.in_ready && (sThread =/= (num_threads.U))) {
      io.new_thread := true.B
      threadState := 1.U
    }
  } .otherwise {
    when (io.ar_valid && (io.ar_opcode(2) === 1.U)) {
      threadState := 0.U
    }
  }

  val inputCore = Module(new inputUnit_core(reg_width, num_regs_lg, opcode_width, num_threads))
  inputCore.io.in_valid := io.in_valid
  inputCore.io.in_tag := io.in_tag
  inputCore.io.in_data := io.in_data
  inputCore.io.in_empty := io.in_empty
  inputCore.io.in_last := io.in_last
  io.in_ready := inputCore.io.in_ready
  inputCore.io.ar_valid := io.ar_valid
  inputCore.io.ar_tag := io.ar_tag
  inputCore.io.ar_opcode := io.ar_opcode
  inputCore.io.ar_rd := io.ar_rd
  inputCore.io.ar_bits := io.ar_bits
  inputCore.io.ar_imm := io.ar_imm
  io.ar_ready := inputCore.io.ar_ready
  inputCore.io.out_ready := io.out_ready
  io.out_valid := inputCore.io.out_valid
  io.out_tag   := inputCore.io.out_tag
  io.out_wen(0)   := inputCore.io.out_wen
  io.out_addr(0)  := inputCore.io.out_addr
  io.out_data(0)  := inputCore.io.out_data
  io.out_wen(1) := false.B
  io.out_addr(1) := 0.U
  io.out_data(1) := DontCare
  io.out_flag := 0.U

  pktFifo.io.enq.valid := inputCore.io.pkt_buf_valid
  pktFifo.io.enq.bits := inputCore.io.pkt_buf_data
  inputCore.io.pkt_buf_ready := pktFifo.io.enq.ready

  io.pkt_buf_data := pktFifo.io.deq.bits
  io.pkt_buf_valid := pktFifo.io.deq.valid
  pktFifo.io.deq.ready := io.pkt_buf_ready

}

import chisel3._

import chisel3.util._


defined class inputUnit

## outputUnit

In [15]:
import chisel3._
import chisel3.util._

class outputUnit(reg_width: Int, num_regs_lg: Int, opcode_width: Int, num_threads: Int, ip_width: Int) extends Module {
  val io = IO(new Bundle {
    val out_ready    = Input(Bool())
    val out_valid    = Output(Bool())
    val out_tag      = Output(UInt(log2Up(num_threads).W))
    val out_data     = Output(UInt(512.W))
    val out_empty    = Output(UInt(6.W))
    val out_last     = Output(Bool())

    val ar_valid     = Input(Bool())
    val ar_tag       = Input(UInt(log2Up(num_threads).W))
    val ar_opcode    = Input(UInt(opcode_width.W))
    val ar_rs        = Input(UInt(num_regs_lg.W))
    val ar_bits      = Input(UInt(reg_width.W))
    val ar_imm       = Input(UInt(32.W))
    val ar_ready     = Output(Bool())

    val r_valid      = Output(Bool())
    val r_flag       = Output(UInt(ip_width.W))
    val r_tag        = Output(UInt(log2Up(num_threads).W))

    // from input unit
    val pkt_buf_data  = Input(new pkt_buf_t(num_threads))
    val pkt_buf_valid = Input(Bool())
    val pkt_buf_ready = Output(Bool())

    // dump regfile interface
    val rd_req_valid  = Output(Bool())
    val rd_req_ready  = Input(Bool())
    val rd_req        = Output(new BFU_regfile_req_t(log2Up(num_threads), num_regs_lg))
    val rd_rsp_valid  = Input(Bool())
    val rd_rsp_ready  = Output(Bool())
    val rd_rsp        = Input(new BFU_regfile_rsp_t(reg_width))
  })

  io.ar_ready := io.out_ready
  io.out_valid := io.ar_valid
  io.out_tag := io.ar_tag
  io.out_data := io.ar_bits
  io.out_empty := 64.U - io.ar_imm
  io.out_last := true.B

  io.r_valid := RegNext(io.ar_valid & io.out_ready)
  io.r_flag := DontCare
  io.r_tag := RegNext(io.ar_tag)

  io.pkt_buf_ready := true.B

  io.rd_req_valid := false.B
  io.rd_req := DontCare
  io.rd_rsp_ready := true.B
}

import chisel3._

import chisel3.util._


defined class outputUnit

## inOutUnit

In [16]:
import chisel3._
import chisel3.util._

class inOutUnit(reg_width: Int, num_regs_lg: Int, opcode_width: Int, num_threads: Int, ip_width: Int) extends MultiIOModule {
  val io = IO(new Bundle {
    val in_valid     = Input(Bool())
    val in_tag       = Input(UInt(log2Up(num_threads).W))  // io.rd_req_valid  ioUnit.io.rd_req  ioUnit.io.rd_req_ready 
    val in_data      = Input(UInt(512.W))
    val in_empty     = Input(UInt(6.W))
    val in_last      = Input(Bool())
    val in_ready     = Output(Bool())

    val out_ready    = Input(Bool())
    val out_valid    = Output(Bool())
    val out_tag      = Output(UInt(log2Up(num_threads).W))
    val out_data     = Output(UInt(512.W))
    val out_empty    = Output(UInt(6.W))
    val out_last     = Output(Bool())

    val ar_valid     = Input(Bool())
    val ar_tag       = Input(UInt(log2Up(num_threads).W))
    val ar_opcode    = Input(UInt(opcode_width.W))
    val ar_rd        = Input(UInt(num_regs_lg.W))
    val ar_bits      = Input(UInt(reg_width.W))
    val ar_imm       = Input(UInt(32.W))
    val ar_ready     = Output(Bool())

    val w_ready      = Input(Bool())
    val w_valid      = Output(Bool())
    val w_tag        = Output(UInt(log2Up(num_threads).W))
    val w_flag       = Output(UInt(ip_width.W))
    val w_wen        = Output(Vec(2, Bool()))
    val w_addr       = Output(Vec(2, UInt(num_regs_lg.W)))
    val w_data       = Output(Vec(2, UInt(reg_width.W)))

    val idle_threads = Input(Vec(num_threads, Bool()))
    val new_thread   = Output(Bool())
    val new_tag      = Output(UInt(log2Up(num_threads).W))

    val r_valid      = Output(Bool())
    val r_flag       = Output(UInt(ip_width.W))
    val r_tag        = Output(UInt(log2Up(num_threads).W))

    val rd_req_valid = Output(Bool())
    val rd_req_ready = Input(Bool())
    val rd_req       = Output(new BFU_regfile_req_t(log2Up(num_threads), num_regs_lg))
    val rd_rsp_valid = Input(Bool())
    val rd_rsp_ready = Output(Bool())
    val rd_rsp       = Input(new BFU_regfile_rsp_t(reg_width))

  })

  val ar_ready_in = Wire(Bool())
  val ar_ready_out = Wire(Bool())
  val ar_valid_in = Wire(Bool())
  val ar_valid_out = Wire(Bool())

  io.ar_ready := Mux((io.ar_opcode(opcode_width-1) === 1.U), ar_ready_out, ar_ready_in)
  ar_valid_in := Mux((io.ar_opcode(opcode_width-1) === 1.U), false.B, io.ar_valid)
  ar_valid_out := Mux((io.ar_opcode(opcode_width-1) === 1.U), io.ar_valid, false.B)

  //***************************************************************//
  //***************    Input Unit     *****************************//
  //***************************************************************//
  val inputU = Module(new inputUnit(reg_width, num_regs_lg, opcode_width, num_threads, ip_width))

  inputU.io.in_valid     := io.in_valid
  inputU.io.in_tag       := io.in_tag
  inputU.io.in_data      := io.in_data
  inputU.io.in_empty     := io.in_empty
  inputU.io.in_last      := io.in_last
  io.in_ready            := inputU.io.in_ready
  inputU.io.out_ready    := io.w_ready
  io.w_valid             := inputU.io.out_valid
  io.w_tag               := inputU.io.out_tag
  io.w_flag              := inputU.io.out_flag
  io.w_wen               := inputU.io.out_wen
  io.w_addr              := inputU.io.out_addr
  io.w_data              := inputU.io.out_data
  inputU.io.idle_threads := io.idle_threads
  io.new_thread          := inputU.io.new_thread
  io.new_tag             := inputU.io.new_tag
  inputU.io.ar_valid     := ar_valid_in
  inputU.io.ar_tag       := io.ar_tag
  inputU.io.ar_opcode    := io.ar_opcode
  inputU.io.ar_rd        := io.ar_rd
  inputU.io.ar_bits      := io.ar_bits
  inputU.io.ar_imm       := io.ar_imm
  ar_ready_in            := inputU.io.ar_ready

  //***************************************************************//
  //***************    Output Unit     ****************************//
  //***************************************************************//

  val outputU = Module(new outputUnit(reg_width, num_regs_lg, opcode_width, num_threads, ip_width))

  outputU.io.out_ready    := io.out_ready
  io.out_valid            := outputU.io.out_valid
  io.out_tag              := outputU.io.out_tag
  io.out_data             := outputU.io.out_data
  io.out_empty            := outputU.io.out_empty
  io.out_last             := outputU.io.out_last
  outputU.io.ar_valid     := ar_valid_out
  outputU.io.ar_tag       := io.ar_tag
  outputU.io.ar_opcode    := io.ar_opcode
  outputU.io.ar_rs        := io.ar_rd
  outputU.io.ar_bits      := io.ar_bits
  outputU.io.ar_imm       := io.ar_imm
  ar_ready_out            := outputU.io.ar_ready
  io.r_valid              := outputU.io.r_valid
  io.r_flag               := outputU.io.r_flag
  io.r_tag                := outputU.io.r_tag
  io.rd_req_valid         := outputU.io.rd_req_valid
  outputU.io.rd_req_ready := io.rd_req_ready
  io.rd_req               := outputU.io.rd_req
  outputU.io.rd_rsp_valid := io.rd_rsp_valid
  io.rd_rsp_ready         := outputU.io.rd_rsp_ready
  outputU.io.rd_rsp       := io.rd_rsp

  //***************************************************************//
  //*************  Input Unit <->  Output Unit ********************//
  //***************************************************************//

  outputU.io.pkt_buf_data := inputU.io.pkt_buf_data
  outputU.io.pkt_buf_valid := inputU.io.pkt_buf_valid
  inputU.io.pkt_buf_ready := outputU.io.pkt_buf_ready
}

import chisel3._

import chisel3.util._


defined class inOutUnit

## ram

In [17]:
import chisel3._
import chisel3.util._
import chisel3.util.Fill
 
import java.io.File
import chisel3._
import chisel3.util.{HasBlackBoxInline, HasBlackBoxPath, HasBlackBoxResource}
import chisel3.stage.ChiselGeneratorAnnotation


class ram_simple2port(num: Int, width: Int) extends
  BlackBox(Map("AWIDTH" -> log2Up(num),
               "DWIDTH" -> width,
               "DEPTH"  -> num)) with HasBlackBoxPath {
  val io = IO(new Bundle {
    val clock     = Input(Clock())
    val data      = Input(UInt(width.W))
    val rdaddress = Input(UInt(log2Up(num).W))
    val rden      = Input(Bool())
    val wraddress = Input(UInt(log2Up(num).W))
    val wren      = Input(Bool())
    val q         = Output(UInt(width.W))
  })

  // addResource("/ram_simple2port_sim.v")
  addPath(new File("/chisel-bootcamp/workspace/build/hw-gen/ram_simple2port_sim.v").getCanonicalPath)

}

class ram_qp(num: Int, width: Int) extends 
  BlackBox(Map("AWIDTH" -> log2Up(num),
               "DWIDTH" -> width,
               "DEPTH"  -> num)) with HasBlackBoxPath {
  val io = IO(new Bundle {
    val read_address_a  = Input(UInt(log2Up(num).W))
    val read_address_b  = Input(UInt(log2Up(num).W))
    val q_a             = Output(UInt(width.W))
    val q_b             = Output(UInt(width.W))

    val wren_a          = Input(Bool())
    val wren_b          = Input(Bool())
    val write_address_a = Input(UInt(log2Up(num).W))
    val write_address_b = Input(UInt(log2Up(num).W))
    val data_a          = Input(UInt(width.W))
    val data_b          = Input(UInt(width.W))

    val clock           = Input(Clock())
  })

  // addResource("/ram_qp_sim.v")
  addPath(new File("/chisel-bootcamp/workspace/build/hw-gen/ram_qp_sim.v").getCanonicalPath)

}

import chisel3._

import chisel3.util._

import chisel3.util.Fill
 

import java.io.File

import chisel3._

import chisel3.util.{HasBlackBoxInline, HasBlackBoxPath, HasBlackBoxResource}

import chisel3.stage.ChiselGeneratorAnnotation



defined class ram_simple2port
defined class ram_qp

## regfile regread

In [18]:
import chisel3._
import chisel3.util._
import chisel3.util.Fill
import chisel3.util.PriorityEncoder
import chisel3.experimental.ChiselEnum

class Regfile(num: Int, width: Int, num_blocks: Int, block_widths: Array[Int]) extends Module {
  val io = IO(new Bundle {
    val rdAddr1 = Input(UInt(log2Up(num).W))
    val rdAddr2 = Input(UInt(log2Up(num).W))
    val rdData1 = Output(UInt(width.W))
    val rdData2 = Output(UInt(width.W))

    val wrEn1   = Input(Bool())
    val wrEn2   = Input(Bool())
    val wrBen1  = Input(UInt(num_blocks.W))
    val wrBen2  = Input(UInt(num_blocks.W))
    val wrAddr1 = Input(UInt(log2Up(num).W))
    val wrAddr2 = Input(UInt(log2Up(num).W))
    val wrData1 = Input(UInt(width.W))
    val wrData2 = Input(UInt(width.W))
  })

  val mems = for (i <- 0 until num_blocks) yield {
    val mem = Module(new ram_qp(num, block_widths(i)))
    mem
  }
  val rdData1 = Wire(MixedVec((0 until num_blocks) map {i => UInt(block_widths(i).W)}))
  val rdData2 = Wire(MixedVec((0 until num_blocks) map {i => UInt(block_widths(i).W)}))
  var pos = 0
  for (i <- 0 until num_blocks) {
    mems(i).io.clock := clock
    mems(i).io.read_address_a := io.rdAddr1
    mems(i).io.read_address_b := io.rdAddr2
    mems(i).io.write_address_a := io.wrAddr1
    mems(i).io.write_address_b := io.wrAddr2
    mems(i).io.wren_a := io.wrEn1 && io.wrBen1(i)
    mems(i).io.wren_b := io.wrEn2 && io.wrBen2(i)
    mems(i).io.data_a := io.wrData1(pos+block_widths(i)-1, pos)
    mems(i).io.data_b := io.wrData2(pos+block_widths(i)-1, pos)
    pos = pos + block_widths(i)
    rdData1(i) := mems(i).io.q_a
    rdData2(i) := mems(i).io.q_b
  }

  io.rdData1 := rdData1.asUInt
  io.rdData2 := rdData2.asUInt

}

class RegRead(threadnum: Int, num_rd: Int, num_wr: Int, num_regs: Int, reg_w: Int, num_blocks: Int, block_widths: Array[Int]) extends Module {
  val io = IO(new Bundle {
    val rdEn        = Input(Bool())
    val thread_rd   = Input(UInt(log2Up(threadnum).W))
    val rdAddr1     = Input(Vec(num_rd, UInt(log2Up(num_regs).W)))
    val rdAddr2     = Input(Vec(num_rd, UInt(log2Up(num_regs).W)))
    val rdData1     = Output(Vec(num_rd, UInt(reg_w.W)))
    val rdData2     = Output(Vec(num_rd, UInt(reg_w.W)))

    val wrEn        = Input(Bool())
    val wrEn1       = Input(Vec(num_wr, Bool()))
    val wrEn2       = Input(Vec(num_wr, Bool()))
    val thread_wr   = Input(UInt(log2Up(threadnum).W))
    val wrBen1      = Input(Vec(num_wr, UInt(num_blocks.W)))
    val wrBen2      = Input(Vec(num_wr, UInt(num_blocks.W)))
    val wrAddr1     = Input(Vec(num_wr, UInt(log2Up(num_regs).W)))
    val wrAddr2     = Input(Vec(num_wr, UInt(log2Up(num_regs).W)))
    val wrData1     = Input(Vec(num_wr, UInt(reg_w.W)))
    val wrData2     = Input(Vec(num_wr, UInt(reg_w.W)))
  })

  val num_regfile = scala.math.pow(2, log2Up(num_rd)).toInt
  val regfile = Seq.fill(num_regfile)(Module(new Regfile(num_regs*threadnum/num_regfile, reg_w, num_blocks, block_widths)))

  // Read logic
  val thread_rd_vec = Reg(Vec(num_rd+1, UInt(log2Up(threadnum).W)))
  if (num_rd > 1) {
    val state_rd = RegInit(VecInit(Seq.fill(num_regfile)(0.U(log2Up(num_rd).W))))
    val thread_rd = Reg(Vec(num_regfile, UInt(log2Up(threadnum/num_regfile).W)))
    val rdAddr1 = Reg(Vec(num_regfile, Vec(num_rd, UInt(log2Up(num_regs).W))))
    val rdAddr2 = Reg(Vec(num_regfile, Vec(num_rd, UInt(log2Up(num_regs).W))))
    for (i <- 0 until num_regfile) {
      when (state_rd(i) === 0.U) {
        when (io.rdEn && (io.thread_rd(log2Up(num_regfile)-1, 0) === i.U)) {
          regfile(i).io.rdAddr1 := Cat(io.thread_rd(log2Up(threadnum)-1, log2Up(num_regfile)), io.rdAddr1(state_rd(i)))
          regfile(i).io.rdAddr2 := Cat(io.thread_rd(log2Up(threadnum)-1, log2Up(num_regfile)), io.rdAddr2(state_rd(i)))
          rdAddr1(i) := io.rdAddr1
          rdAddr2(i) := io.rdAddr2
          thread_rd(i) := io.thread_rd(log2Up(threadnum)-1, log2Up(num_regfile))
          state_rd(i) := 1.U
        } .otherwise {
          regfile(i).io.rdAddr1 := Cat(io.thread_rd(log2Up(threadnum)-1, log2Up(num_regfile)), io.rdAddr1(0))
          regfile(i).io.rdAddr2 := Cat(io.thread_rd(log2Up(threadnum)-1, log2Up(num_regfile)), io.rdAddr2(0))
        }
      } .otherwise {
        regfile(i).io.rdAddr1 := Cat(thread_rd(i), rdAddr1(i)(state_rd(i)))
        regfile(i).io.rdAddr2 := Cat(thread_rd(i), rdAddr2(i)(state_rd(i)))
        when (state_rd(i) + 1.U === num_rd.U) {
          state_rd(i) := 0.U
        } .otherwise {
          state_rd(i) := state_rd(i) + 1.U
        }
      }
    }
  } else {
    regfile(0).io.rdAddr1 := Cat(io.thread_rd, io.rdAddr1(0))
    regfile(0).io.rdAddr2 := Cat(io.thread_rd, io.rdAddr2(0))
  }

  if (num_rd > 1) {
    thread_rd_vec(0) := io.thread_rd
    for (i <- 0 until num_rd) {
      thread_rd_vec(i+1) := thread_rd_vec(i)
    }
    val rdData1 = Wire(Vec(num_regfile, UInt(reg_w.W)))
    val rdData2 = Wire(Vec(num_regfile, UInt(reg_w.W)))
    for (i <- 0 until num_regfile) {
      rdData1(i) := regfile(i).io.rdData1
      rdData2(i) := regfile(i).io.rdData2
    }
    for (i <- 0 until num_rd-1) {
      val rdData1_vec = Reg(Vec(num_rd-1-i, UInt(reg_w.W)))
      val rdData2_vec = Reg(Vec(num_rd-1-i, UInt(reg_w.W)))
      val regfile_slct = Wire(UInt(log2Up(num_rd).W))
      regfile_slct := thread_rd_vec(1+i)(log2Up(num_regfile)-1, 0)
      rdData1_vec(0) := rdData1(regfile_slct)
      rdData2_vec(0) := rdData2(regfile_slct)
      if (i < num_rd-2) {
        for (j <- 1 until num_rd-1-i) {
          rdData1_vec(j) := rdData1_vec(j-1)
          rdData2_vec(j) := rdData2_vec(j-1)
        }
      }
      io.rdData1(i) := rdData1_vec(num_rd-2-i)
      io.rdData2(i) := rdData2_vec(num_rd-2-i)
    }
    val regfile_slct = Wire(UInt(log2Up(num_rd).W))
    regfile_slct := thread_rd_vec(num_rd)(log2Up(num_regfile)-1, 0)
    io.rdData1(num_rd-1) := rdData1(regfile_slct)
    io.rdData2(num_rd-1) := rdData2(regfile_slct)
  } else {
    io.rdData1(0) := regfile(0).io.rdData1
    io.rdData2(0) := regfile(0).io.rdData2
  }

  // Write logic
  if (num_wr > 1) {
    for (i <- 0 until num_regfile) {
      val state_wr = RegInit(0.U(log2Up(num_wr).W))
      val thread_wr = Reg(UInt(log2Up(threadnum/num_regfile).W))
      val wrAddr1 = Reg(Vec(num_wr, UInt(log2Up(num_regs).W)))
      val wrAddr2 = Reg(Vec(num_wr, UInt(log2Up(num_regs).W)))
      val wrEn1 = Reg(Vec(num_wr, Bool()))
      val wrEn2 = Reg(Vec(num_wr, Bool()))
      val wrBen1 = Reg(Vec(num_wr, UInt(num_blocks.W)))
      val wrBen2 = Reg(Vec(num_wr, UInt(num_blocks.W)))
      val wrData1 = Reg(Vec(num_wr, UInt(reg_w.W)))
      val wrData2 = Reg(Vec(num_wr, UInt(reg_w.W)))
      when (state_wr === 0.U) {
        when (io.thread_wr(log2Up(num_regfile)-1, 0) === i.U) {
          regfile(i).io.wrAddr1 := Cat(io.thread_wr(log2Up(threadnum)-1, log2Up(num_regfile)), io.wrAddr1(state_wr))
          regfile(i).io.wrAddr2 := Cat(io.thread_wr(log2Up(threadnum)-1, log2Up(num_regfile)), io.wrAddr2(state_wr))
          regfile(i).io.wrBen1 := io.wrBen1(state_wr)
          regfile(i).io.wrBen2 := io.wrBen2(state_wr)
          regfile(i).io.wrEn1 := io.wrEn1(state_wr)
          regfile(i).io.wrEn2 := io.wrEn2(state_wr)
          regfile(i).io.wrData1 := io.wrData1(state_wr)
          regfile(i).io.wrData2 := io.wrData2(state_wr)
          when (io.wrEn) {
            thread_wr := io.thread_wr(log2Up(threadnum)-1, log2Up(num_regfile))
            wrAddr1 := io.wrAddr1
            wrAddr2 := io.wrAddr2
            wrEn1 := io.wrEn1
            wrEn2 := io.wrEn2
            wrBen1 := io.wrBen1
            wrBen2 := io.wrBen2
            wrData1 := io.wrData1
            wrData2 := io.wrData2
            state_wr := 1.U
            printf("Writing to reg %x: %x\n", wrAddr1.asUInt, wrData1.asUInt)
            printf("Writing to reg %x: %x\n", wrAddr2.asUInt, wrData2.asUInt)
          }
        } .otherwise {
          regfile(i).io.wrAddr1 := DontCare
          regfile(i).io.wrAddr2 := DontCare
          regfile(i).io.wrBen1 := DontCare
          regfile(i).io.wrBen2 := DontCare
          regfile(i).io.wrEn1 := false.B
          regfile(i).io.wrEn2 := false.B
          regfile(i).io.wrData1 := DontCare
          regfile(i).io.wrData2 := DontCare
        }
      } .otherwise {
        regfile(i).io.wrAddr1 := Cat(thread_wr, wrAddr1(state_wr))
        regfile(i).io.wrAddr2 := Cat(thread_wr, wrAddr2(state_wr))
        regfile(i).io.wrBen1 := wrBen1(state_wr)
        regfile(i).io.wrBen2 := wrBen2(state_wr)
        regfile(i).io.wrEn1 := wrEn1(state_wr)
        regfile(i).io.wrEn2 := wrEn2(state_wr)
        regfile(i).io.wrData1 := wrData1(state_wr)
        regfile(i).io.wrData2 := wrData2(state_wr)
        when (state_wr + 1.U === num_wr.U) {
          state_wr := 0.U
        } .otherwise {
          state_wr := state_wr + 1.U
        }
      }
    }
  } else {
    if (num_regfile > 1) {
      for (i <- 0 until num_regfile) {
        when (io.thread_wr(log2Up(num_regfile)-1, 0) === i.U) {
          regfile(i).io.wrAddr1 := Cat(io.thread_wr(log2Up(threadnum)-1, log2Up(num_regfile)), io.wrAddr1(0))
          regfile(i).io.wrAddr2 := Cat(io.thread_wr(log2Up(threadnum)-1, log2Up(num_regfile)), io.wrAddr2(0))
          regfile(i).io.wrBen1 := io.wrBen1(0)
          regfile(i).io.wrBen2 := io.wrBen2(0)
          regfile(i).io.wrEn1 := io.wrEn1(0)
          regfile(i).io.wrEn2 := io.wrEn2(0)
          regfile(i).io.wrData1 := io.wrData1(0)
          regfile(i).io.wrData2 := io.wrData2(0)
          when (regfile(i).io.wrEn1) {
            printf("Writing to reg %x: %x\n", regfile(i).io.wrAddr1.asUInt, regfile(i).io.wrData1.asUInt)
          }
          when (regfile(i).io.wrEn2) {
            printf("Writing to reg %x: %x\n", regfile(i).io.wrAddr2.asUInt, regfile(i).io.wrData2.asUInt)
          }
        } .otherwise {
          regfile(i).io.wrAddr1 := DontCare
          regfile(i).io.wrAddr2 := DontCare
          regfile(i).io.wrBen1 := DontCare
          regfile(i).io.wrBen2 := DontCare
          regfile(i).io.wrEn1 := false.B
          regfile(i).io.wrEn2 := false.B
          regfile(i).io.wrData1 := DontCare
          regfile(i).io.wrData2 := DontCare
        }
      }
    } else {
      regfile(0).io.wrAddr1 := Cat(io.thread_wr, io.wrAddr1(0))
      regfile(0).io.wrAddr2 := Cat(io.thread_wr, io.wrAddr2(0))
      regfile(0).io.wrBen1 := io.wrBen1(0)
      regfile(0).io.wrBen2 := io.wrBen2(0)
      regfile(0).io.wrEn1 := io.wrEn1(0)
      regfile(0).io.wrEn2 := io.wrEn2(0)
      regfile(0).io.wrData1 := io.wrData1(0)
      regfile(0).io.wrData2 := io.wrData2(0)
    }
  }

}

import chisel3._

import chisel3.util._

import chisel3.util.Fill

import chisel3.util.PriorityEncoder

import chisel3.experimental.ChiselEnum


defined class Regfile
defined class RegRead

## decode.scala

In [19]:
import chisel3._
import chisel3.util._

class DecodeBranch(inst_width: Int, num_regs_lg: Int) extends Module {
  val io = IO(new Bundle {
    val instr     = Input(UInt(inst_width.W))
    val brValid   = Output(Bool())
    val brMode    = Output(UInt(4.W))
    val rs1       = Output(UInt(num_regs_lg.W))
    val rs2       = Output(UInt(num_regs_lg.W))
    val rd        = Output(UInt(num_regs_lg.W))
    val rdWrEn    = Output(Bool())
    val pcOffset  = Output(SInt(21.W))
  })

  val OPCODE_LOW = 0
  val OPCODE_HIGH = OPCODE_LOW + 6
  val RD_LOW = OPCODE_HIGH + 1
  val RD_HIGH = RD_LOW + num_regs_lg - 1
  val FUNCT3_LOW = RD_HIGH + 1
  val FUNCT3_HIGH = FUNCT3_LOW + 2
  val RS1_LOW = FUNCT3_HIGH + 1
  val RS1_HIGH = RS1_LOW + num_regs_lg - 1
  val RS2_LOW = RS1_HIGH + 1
  val RS2_HIGH = RS2_LOW + num_regs_lg - 1
  val IMM_LOW = RS2_HIGH + 1
  val IMM_HIGH = IMM_LOW + 6

  val opcode = Wire(UInt(7.W))
  opcode := io.instr(OPCODE_HIGH, OPCODE_LOW)
  io.rd := io.instr(RD_HIGH, RD_LOW)
  io.rs1 := io.instr(RS1_HIGH, RS1_LOW)
  io.rs2 := io.instr(RS2_HIGH, RS2_LOW)
  io.rdWrEn := false.B
  io.pcOffset := 0.S
  io.brMode := DontCare
  io.brValid := true.B

  when (opcode === 0x6f.U) {
    when (io.rd =/= 0.U) {
      io.brMode := 2.U
      io.rdWrEn := true.B
    } .otherwise {
      io.brMode := 3.U
    }
    val tmp = Cat(io.instr(FUNCT3_LOW+19), io.instr(FUNCT3_LOW+7, FUNCT3_LOW), io.instr(FUNCT3_LOW+8), io.instr(FUNCT3_LOW+18, FUNCT3_LOW+9)).asSInt
    io.pcOffset := tmp
  } .elsewhen (opcode === 0x63.U) {
    io.brMode := io.instr(FUNCT3_LOW+2, FUNCT3_LOW)
    val tmp = Cat(io.instr(IMM_HIGH), io.instr(RD_LOW), io.instr(IMM_HIGH-1, IMM_LOW), io.instr(RD_LOW+4, RD_LOW+1)).asSInt
    io.pcOffset := tmp
  } .elsewhen (opcode === 0x67.U) {
    val tmp = io.instr(RS2_LOW+11, RS2_LOW).asSInt
    io.brMode := 8.U
    io.pcOffset := tmp
  } .otherwise {
    io.brValid := false.B
  }
}

class DecodeEIU(inst_width: Int, num_regs_lg: Int, num_src_pos_lg: Int, num_src_modes_lg: Int, num_dst_pos_lg: Int, num_dst_modes_lg: Int) extends Module {
  val io = IO(new Bundle {
    val instr     = Input(UInt(inst_width.W))
    val rs        = Output(UInt(num_regs_lg.W))
    val rs_pos    = Output(UInt(num_src_pos_lg.W))
    val rs_mode   = Output(UInt(num_src_modes_lg.W))
    val rd        = Output(UInt(num_regs_lg.W))
    val rd_pos    = Output(UInt(num_dst_pos_lg.W))
    val rd_mode   = Output(UInt(num_dst_modes_lg.W))
    val rdWrEn    = Output(Bool())
  })

  val OPCODE_LOW = 0
  val OPCODE_HIGH = OPCODE_LOW + 6
  val RD_LOW = OPCODE_HIGH + 1
  val RD_HIGH = RD_LOW + num_regs_lg - 1
  val FUNCT3_LOW = RD_HIGH + 1
  val FUNCT3_HIGH = FUNCT3_LOW + 2
  val RS1_LOW = FUNCT3_HIGH + 1
  val RS1_HIGH = RS1_LOW + num_regs_lg - 1
  val IMM_LOW = RS1_HIGH + 1
  val IMM_HIGH = IMM_LOW + 11

  io.rs := io.instr(RS1_HIGH, RS1_LOW)
  io.rs_pos := io.instr(IMM_LOW+num_src_pos_lg-1, IMM_LOW)
  io.rs_mode := io.instr(IMM_LOW+num_src_pos_lg+num_src_modes_lg-1, IMM_LOW+num_src_pos_lg)
  io.rd := io.instr(RD_HIGH, RD_LOW)
  io.rd_pos := io.instr(IMM_LOW+num_dst_pos_lg-1, IMM_LOW)
  io.rd_mode := io.instr(IMM_LOW+num_dst_pos_lg+num_dst_modes_lg-1, IMM_LOW+num_dst_pos_lg)

  io.rdWrEn := false.B
  when ((io.instr(FUNCT3_LOW) === 1.U) && (io.rd =/= 0.U)) {
    io.rdWrEn := true.B
  }
}

class DecodeALUBFU(inst_width: Int, num_regs_lg: Int) extends Module {
  val io = IO(new Bundle {
    val instr     = Input(UInt(inst_width.W))
    val rs1       = Output(UInt(num_regs_lg.W))
    val rs2       = Output(UInt(num_regs_lg.W))
    val rd        = Output(UInt(num_regs_lg.W))
    val rdWrEn    = Output(Bool())
    val addEn     = Output(Bool())
    val subEn     = Output(Bool())
    val sltEn     = Output(Bool())
    val sltuEn    = Output(Bool())
    val andEn     = Output(Bool())
    val orEn      = Output(Bool())
    val xorEn     = Output(Bool())
    val sllEn     = Output(Bool())
    val srEn      = Output(Bool())
    val srMode    = Output(Bool())
    val luiEn     = Output(Bool())
    val immSel    = Output(Bool())
    val imm       = Output(SInt(32.W))
    val mulEn     = Output(Bool())
    val mulH      = Output(Bool())
    val catEn     = Output(Bool())
    // val divEn     = Output(Bool())
    val remEn     = Output(Bool())
    val rs1Signed = Output(Bool())
    val rs2Signed = Output(Bool())
    val bfu_valid  = Output(Bool())
    val bfu_opcode = Output(UInt(6.W))
  })

  val OPCODE_LOW = 0
  val OPCODE_HIGH = OPCODE_LOW + 6
  val RD_LOW = OPCODE_HIGH + 1
  val RD_HIGH = RD_LOW + num_regs_lg - 1
  val FUNCT3_LOW = RD_HIGH + 1
  val FUNCT3_HIGH = FUNCT3_LOW + 2
  val RS1_LOW = FUNCT3_HIGH + 1
  val RS1_HIGH = RS1_LOW + num_regs_lg - 1
  val RS2_LOW = RS1_HIGH + 1
  val RS2_HIGH = RS2_LOW + num_regs_lg - 1
  val IMM_LOW = RS2_HIGH + 1
  val IMM_HIGH = IMM_LOW + 6

  val opcode = Wire(UInt(7.W))
  val funct3 = Wire(UInt(3.W))
  val imm12 = Wire(SInt(12.W))
  val imm5 = Wire(SInt(5.W))
  val imm32 = Wire(UInt(32.W))
  opcode := io.instr(OPCODE_HIGH, OPCODE_LOW)
  funct3 := io.instr(FUNCT3_HIGH, FUNCT3_LOW)
  io.rd := io.instr(RD_HIGH, RD_LOW)
  io.rs1 := io.instr(RS1_HIGH, RS1_LOW)
  io.rs2 := io.instr(RS2_HIGH, RS2_LOW)
  io.bfu_opcode := Cat(io.instr(OPCODE_LOW+6, OPCODE_LOW+4), io.instr(FUNCT3_LOW+2, FUNCT3_LOW))
  io.rdWrEn := true.B
  io.addEn  := false.B
  io.subEn  := false.B
  io.sltEn  := false.B
  io.sltuEn := false.B
  io.andEn  := false.B
  io.orEn   := false.B
  io.xorEn  := false.B
  io.sllEn  := false.B
  io.srEn   := false.B
  io.srMode := false.B
  io.luiEn  := false.B
  io.immSel := false.B
  io.mulEn  := false.B
  io.mulH   := false.B
  io.catEn  := false.B
  // io.divEn  := false.B
  io.remEn  := false.B
  io.rs1Signed := true.B
  io.rs2Signed := true.B
  io.imm := 0.S
  imm12 := io.instr(RS2_LOW+11, RS2_LOW).asSInt
  imm5 := io.instr(RS2_LOW+4, RS2_LOW).asSInt
  imm32 := Cat(io.instr(FUNCT3_LOW+19, FUNCT3_LOW), 0.U(12.W))

  when (opcode === 0x13.U && io.instr(FUNCT3_HIGH, RD_LOW) === 0.U) {
    io.rdWrEn := false.B
  } .elsewhen (opcode(3, 0) === 0xb.U) {
    when (io.rd === 0.U) {
      io.rdWrEn := false.B
    }
  } .elsewhen (opcode === 0x23.U) {
    io.rdWrEn := false.B
  }

  when (opcode(3, 0) === 0xb.U) {
    io.bfu_valid := true.B
  } .elsewhen (opcode === 3.U) {
    io.bfu_valid := true.B
  } .elsewhen (opcode === 0x23.U) {
    io.bfu_valid := true.B
  } .otherwise {
    io.bfu_valid := false.B
  }

  when (opcode === 0x13.U) {
    io.immSel := true.B
  }

  when (opcode === 0x37.U) {
    io.luiEn := true.B
    io.immSel := true.B
  }

  when (opcode === 0x13.U) {
    when (funct3(1, 0) === 1.U) {
      io.imm := imm5
    } .otherwise {
      io.imm := imm12
    }
  } .elsewhen (opcode === 0x37.U) {
    io.imm := imm32.asSInt
  } .elsewhen (opcode === 0x3.U) {
    io.imm := imm12
  } .elsewhen (opcode === 0x23.U) {
    io.imm := Cat(io.instr(IMM_HIGH, IMM_LOW), io.instr(RD_LOW+4, RD_LOW)).asSInt
  } .elsewhen (opcode(3, 0) === 0xb.U) {
    io.imm := imm12
  }

  when (opcode === 0x13.U || ((opcode === 0x33.U) && (io.instr(IMM_LOW) === 0.U))) {
    switch (funct3) {
      is (0.U) {
        when ((opcode === 0x33.U) && io.instr(IMM_LOW+5) === 1.U) {
          io.subEn := true.B
        } .otherwise {
          io.addEn := true.B
        }
      }
      is (1.U) {
        io.sllEn := true.B
      }
      is (2.U) {
        io.sltEn := true.B
      }
      is (3.U) {
        io.sltuEn := true.B
      }
      is (4.U) {
        io.xorEn := true.B
      }
      is (5.U) {
        io.srEn := true.B
        when (io.instr(IMM_LOW+5) === 1.U) {
          io.srMode := true.B
        } .otherwise {
          io.srMode := false.B
        }
      }
      is (6.U) {
        io.orEn := true.B
      }
      is (7.U) {
        io.andEn := true.B
      }
    }
  }

  when ((opcode === 0x33.U) && (io.instr(IMM_LOW) === 1.U)) {
    switch (funct3) {
      is (0.U) {
        io.mulEn := true.B
      }
      is (1.U) {
        io.mulEn := true.B
        io.mulH := true.B
      }
      is (2.U) {
        io.mulEn := true.B
        io.mulH := true.B
        io.rs1Signed := true.B
        io.rs2Signed := false.B
      }
      is (3.U) {
        io.mulEn := true.B
        io.mulH := true.B
        io.rs1Signed := false.B
        io.rs2Signed := false.B
      }
      is (4.U) {
        io.catEn := true.B
        // io.divEn := true.B
      }
      is (5.U) {
        io.catEn := true.B
        // io.divEn := true.B
        io.rs2Signed := false.B
      }
      is (6.U) {
        io.remEn := true.B
      }
      is (7.U) {
        io.remEn := true.B
        io.rs2Signed := false.B
      }
    }
  }
}

class DecodeBFU(inst_width: Int, num_regs_lg: Int) extends Module {
  val io = IO(new Bundle{
    val instr     = Input(UInt(inst_width.W))
    val valid     = Output(Bool())
    val opcode    = Output(UInt(6.W))
    val rs        = Output(Vec(2, UInt(num_regs_lg.W)))
    val rd        = Output(UInt(num_regs_lg.W))
    val rdWrEn    = Output(Bool())
    val imm       = Output(UInt(12.W))
  })

  val OPCODE_LOW = 0
  val OPCODE_HIGH = OPCODE_LOW + 6
  val RD_LOW = OPCODE_HIGH + 1
  val RD_HIGH = RD_LOW + num_regs_lg - 1
  val FUNCT3_LOW = RD_HIGH + 1
  val FUNCT3_HIGH = FUNCT3_LOW + 2
  val RS1_LOW = FUNCT3_HIGH + 1
  val RS1_HIGH = RS1_LOW + num_regs_lg - 1
  val RS2_LOW = RS1_HIGH + 1
  val RS2_HIGH = RS2_LOW + num_regs_lg - 1
  val IMM_LOW = RS2_HIGH + 1
  val IMM_HIGH = IMM_LOW + 6

  val opcode = io.instr(OPCODE_HIGH, OPCODE_LOW)
  io.opcode := Cat(io.instr(OPCODE_LOW+6, OPCODE_LOW+4), io.instr(FUNCT3_LOW+2, FUNCT3_LOW))
  io.rd := io.instr(RD_HIGH, RD_LOW)
  io.rs(0) := io.instr(RS1_HIGH, RS1_LOW)
  io.rs(1) := io.instr(RS2_HIGH, RS2_LOW)
  io.rdWrEn := false.B
  io.imm := io.instr(RS2_LOW+11, RS2_LOW)

  when (opcode(3, 0) === 0xb.U) {
    io.valid := true.B
    when (io.rd =/= 0.U && (opcode(6, 4) =/= 0.U) && (opcode(6, 4) =/= 5.U)) {
      io.rdWrEn := true.B
    }
  } .elsewhen (opcode === 3.U) {
    io.valid := true.B
    when (io.rd =/= 0.U && opcode(5) === 0.U) {
      io.rdWrEn := true.B
    }
  } .elsewhen (opcode === 0x23.U) {
    io.valid := true.B
    io.imm := Cat(io.instr(IMM_HIGH, IMM_LOW), io.instr(RD_LOW+4, RD_LOW))
  } .otherwise {
    io.valid := false.B
  }


}

class ALUMicrocodes(num_alus: Int, num_regs_lg: Int, num_src_pos_lg: Int, num_src_modes_lg: Int) extends Bundle {
  val bfu_valid = Vec(num_alus, Bool())
  val opcode    = Vec(num_alus, UInt(6.W))
  val rs1_pos   = Vec(num_alus, UInt(num_src_pos_lg.W))
  val rs1_mode  = Vec(num_alus, UInt(num_src_modes_lg.W))
  val rs2_pos   = Vec(num_alus, UInt(num_src_pos_lg.W))
  val rs2_mode  = Vec(num_alus, UInt(num_src_modes_lg.W))
  val rd        = Vec(num_alus, UInt(num_regs_lg.W))
  val addEn     = Vec(num_alus, Bool())
  val subEn     = Vec(num_alus, Bool())
  val sltEn     = Vec(num_alus, Bool())
  val sltuEn    = Vec(num_alus, Bool())
  val andEn     = Vec(num_alus, Bool())
  val orEn      = Vec(num_alus, Bool())
  val xorEn     = Vec(num_alus, Bool())
  val sllEn     = Vec(num_alus, Bool())
  val srEn      = Vec(num_alus, Bool())
  val srMode    = Vec(num_alus, Bool())
  val luiEn     = Vec(num_alus, Bool())
  val catEn     = Vec(num_alus, Bool())
  val immSel    = Vec(num_alus, Bool())
  val imm       = Vec(num_alus, SInt(32.W))

  override def cloneType = (new ALUMicrocodes(num_alus, num_regs_lg, num_src_pos_lg, num_src_modes_lg).asInstanceOf[this.type])
}

class BRMicrocodes(num_alus: Int, num_fus: Int) extends Bundle {
  val brValid   = Bool()
  val brMode    = UInt(4.W)
  val rs1       = UInt(log2Up(num_alus*3+num_fus).W)
  val rs2       = UInt(log2Up(num_alus*3+num_fus).W)
  val pcOffset  = SInt(21.W)

  override def cloneType = (new BRMicrocodes(num_alus, num_fus).asInstanceOf[this.type])
}

class BFUMicrocodes(num_bfus: Int, num_regs_lg: Int) extends Bundle {
  val opcode    = Vec(num_bfus, UInt(6.W))
  val rs        = Vec(num_bfus, Vec(2, UInt(num_regs_lg.W)))
  val rd        = Vec(num_bfus, UInt(num_regs_lg.W))
  val bimm      = Vec(num_bfus, UInt(12.W))

  override def cloneType = (new BFUMicrocodes(num_bfus, num_regs_lg).asInstanceOf[this.type])
}

class Decode(num_alus: Int, num_bfus: Int, num_fus: Int, inst_width: Int, num_regs_lg: Int, num_src_pos_lg: Int, num_src_modes_lg: Int, num_dst_pos_lg: Int, num_dst_modes_lg: Int) extends Module {
  val io = IO(new Bundle {
    val instr     = Input(UInt(((num_alus*3+num_fus+1)*inst_width).W))

    val rs1       = Output(Vec(num_fus, UInt(num_regs_lg.W)))
    val rs2       = Output(Vec(num_fus, UInt(num_regs_lg.W)))
    val rd        = Output(Vec(num_fus, UInt(num_regs_lg.W)))
    val rd_pos    = Output(Vec(num_alus, UInt(num_dst_pos_lg.W)))
    val rd_mode   = Output(Vec(num_alus, UInt(num_dst_modes_lg.W)))
    val rdWrEn    = Output(Vec(num_fus, Bool()))

    val bfuValids = Output(Vec(num_bfus, Bool()))
    val brUcodes  = Output(new BRMicrocodes(num_alus, num_fus))
    val aluUcodes = Output(new ALUMicrocodes(num_alus, num_regs_lg, num_src_pos_lg, num_src_modes_lg))
    val bfuUcodes = Output(new BFUMicrocodes(num_fus-num_alus, num_regs_lg))
  })
  // num_alus cannot exceed num_regs, or index will be truncated
  // assert(num_alus + num_bfus <= 32)

  val branchDecoder = Module(new DecodeBranch(inst_width, num_regs_lg))
  val eiuDecoders = Seq.fill(3*num_alus)(Module(new DecodeEIU(inst_width, num_regs_lg, num_src_pos_lg, num_src_modes_lg, num_dst_pos_lg, num_dst_modes_lg)))
  val aluDecoders = Seq.fill(num_alus)(Module(new DecodeALUBFU(inst_width, num_regs_lg)))
  val bfuDecoders = Seq.fill(num_fus-num_alus)(Module(new DecodeBFU(inst_width, num_regs_lg)))

  // offsets for ALU operations
  val ALU_PKT_WIDTH = 4*inst_width
  val EXTRACT0_LOW = 0
  val EXTRACT0_HIGH = EXTRACT0_LOW + inst_width - 1
  val EXTRACT1_LOW = EXTRACT0_HIGH + 1
  val EXTRACT1_HIGH = EXTRACT1_LOW + inst_width - 1
  val ALU_LOW = EXTRACT1_HIGH + 1
  val ALU_HIGH = ALU_LOW + inst_width -1 
  val INSERT_LOW = ALU_HIGH + 1
  val INSERT_HIGH = INSERT_LOW + inst_width - 1
  val BRINST_LOW = num_fus*inst_width + num_alus*3*inst_width
  val BRINST_HIGH = BRINST_LOW + inst_width - 1

  branchDecoder.io.instr := io.instr(BRINST_HIGH, BRINST_LOW)
  io.brUcodes.brValid := branchDecoder.io.brValid
  io.brUcodes.brMode := branchDecoder.io.brMode
  io.brUcodes.pcOffset := branchDecoder.io.pcOffset
  io.brUcodes.rs1 := branchDecoder.io.rs1
  io.brUcodes.rs2 := branchDecoder.io.rs2

  for (i <- 0 until num_alus) {
    eiuDecoders(3*i).io.instr := io.instr(i*ALU_PKT_WIDTH+EXTRACT0_HIGH, i*ALU_PKT_WIDTH+EXTRACT0_LOW)
    eiuDecoders(3*i+1).io.instr := io.instr(i*ALU_PKT_WIDTH+EXTRACT1_HIGH, i*ALU_PKT_WIDTH+EXTRACT1_LOW)
    aluDecoders(i).io.instr := io.instr(i*ALU_PKT_WIDTH+ALU_HIGH, i*ALU_PKT_WIDTH+ALU_LOW)
    eiuDecoders(3*i+2).io.instr := io.instr(i*ALU_PKT_WIDTH+INSERT_HIGH, i*ALU_PKT_WIDTH+INSERT_LOW)
  }

  if (num_alus < num_bfus) {
    for (i <- 0 until (num_bfus-num_alus)) {
      bfuDecoders(i).io.instr := io.instr(num_alus*ALU_PKT_WIDTH+i*inst_width+inst_width-1, num_alus*ALU_PKT_WIDTH+i*inst_width)
    }
  }

  for (i <- 0 until num_alus) {
    io.rs1(i)                 := eiuDecoders(i*3).io.rs
    io.rs2(i)                 := eiuDecoders(i*3+1).io.rs
    io.rd(i)                  := eiuDecoders(i*3+2).io.rd
    io.rd_pos(i)              := eiuDecoders(i*3+2).io.rd_pos
    io.rd_mode(i)             := eiuDecoders(i*3+2).io.rd_mode
    io.rdWrEn(i)              := eiuDecoders(i*3+2).io.rdWrEn
    io.aluUcodes.bfu_valid(i) := aluDecoders(i).io.bfu_valid
    io.aluUcodes.opcode(i)    := aluDecoders(i).io.bfu_opcode
    io.aluUcodes.rs1_pos(i)   := eiuDecoders(i*3).io.rs_pos
    io.aluUcodes.rs1_mode(i)  := eiuDecoders(i*3).io.rs_mode
    io.aluUcodes.rs2_pos(i)   := eiuDecoders(i*3+1).io.rs_pos
    io.aluUcodes.rs2_mode(i)  := eiuDecoders(i*3+1).io.rs_mode
    io.aluUcodes.rd(i)        := eiuDecoders(i*3+2).io.rd
    io.aluUcodes.addEn(i)     := aluDecoders(i).io.addEn
    io.aluUcodes.subEn(i)     := aluDecoders(i).io.subEn
    io.aluUcodes.sltEn(i)     := aluDecoders(i).io.sltEn
    io.aluUcodes.sltuEn(i)    := aluDecoders(i).io.sltuEn
    io.aluUcodes.andEn(i)     := aluDecoders(i).io.andEn
    io.aluUcodes.orEn(i)      := aluDecoders(i).io.orEn
    io.aluUcodes.xorEn(i)     := aluDecoders(i).io.xorEn
    io.aluUcodes.sllEn(i)     := aluDecoders(i).io.sllEn
    io.aluUcodes.srEn(i)      := aluDecoders(i).io.srEn
    io.aluUcodes.srMode(i)    := aluDecoders(i).io.srMode
    io.aluUcodes.luiEn(i)     := aluDecoders(i).io.luiEn
    io.aluUcodes.catEn(i)     := aluDecoders(i).io.catEn
    io.aluUcodes.immSel(i)    := aluDecoders(i).io.immSel
    io.aluUcodes.imm(i)       := aluDecoders(i).io.imm
  }

  if (num_alus == num_bfus) {
    // The last BFU is always the IO unit
    io.rdWrEn(num_alus-1) := eiuDecoders(num_alus*3-1).io.rdWrEn & (!aluDecoders(num_alus-1).io.bfu_valid)
  }

  if (num_alus < num_bfus) {
    for (i <- 0 until num_alus) {
      io.bfuValids(i) := aluDecoders(i).io.bfu_valid
    }
    for (i <- 0 until num_bfus-num_alus) {
      io.bfuValids(i+num_alus)  := bfuDecoders(i).io.valid
      io.rs1(i+num_alus)        := bfuDecoders(i).io.rs(0)
      io.rs2(i+num_alus)        := bfuDecoders(i).io.rs(1)
      io.rd(i+num_alus)         := bfuDecoders(i).io.rd
      io.rdWrEn(i+num_alus)     := bfuDecoders(i).io.rdWrEn
      io.bfuUcodes.opcode(i)    := bfuDecoders(i).io.opcode
      io.bfuUcodes.rs(i)        := bfuDecoders(i).io.rs
      io.bfuUcodes.rd(i)        := bfuDecoders(i).io.rd
      io.bfuUcodes.bimm(i)      := bfuDecoders(i).io.imm
    }
  } else {
    for (i <- 0 until num_bfus) {
      io.bfuValids(i) := aluDecoders(i).io.bfu_valid
    }
  }

}

import chisel3._

import chisel3.util._


defined class DecodeBranch
defined class DecodeEIU
defined class DecodeALUBFU
defined class DecodeBFU
defined class ALUMicrocodes
defined class BRMicrocodes
defined class BFUMicrocodes
defined class Decode

## scheduler

In [20]:
import chisel3._
import chisel3.util._
import chisel3.util.Fill

class Scheduler(num_threads: Int, num_regfile: Int) extends Module {
  val io = IO(new Bundle {
    val valid = Input(Vec(num_threads, Bool()))
    val chosen = Output(UInt((log2Up(num_threads)+1).W))
  })
  val thread = RegInit(0.U(log2Up(num_regfile).W))
  val chosen_i = Wire(Vec(num_regfile, UInt((log2Up(num_threads)+1).W)))
  for (i <- 0 until num_regfile) {
    val cases = (0 until (num_threads/num_regfile)).map( x => io.valid(x * num_regfile + i) -> (x * num_regfile + i).U)
    chosen_i(i) := MuxCase(num_threads.U, cases)
  }
  thread := thread + 1.U

  io.chosen := chosen_i(thread)
}

class Scheduler_order(num_threads: Int, num_regfile: Int) extends Module {
  val io = IO(new Bundle {
    val valid = Input(Bool())
    val tag = Input(UInt((log2Up(num_threads)).W))
    val order_ready = Input(Vec(num_threads, Bool()))
    val ready = Input(Vec(num_threads, Bool()))
    val chosen = Output(UInt((log2Up(num_threads)+1).W))
  })
  val thread = RegInit(0.U(log2Up(num_regfile).W))
  val thread_count = RegInit(0.U((log2Up(num_threads) + 1).W))
  val fifo = Module(new Queue(UInt((log2Up(num_threads)).W), num_threads))
  thread := thread + 1.U
  val chosen_i = Wire(Vec(num_regfile, UInt((log2Up(num_threads)+1).W)))
  for (i <- 0 until num_regfile) {
    val cases = (0 until (num_threads/num_regfile)).map( x => io.ready(x * num_regfile + i) -> (x * num_regfile + i).U)
    chosen_i(i) := MuxCase(num_threads.U, cases)
  }
  thread := thread + 1.U

  fifo.io.enq.valid := false.B
  fifo.io.enq.bits := DontCare
  when (io.valid) {
    fifo.io.enq.valid := true.B
    fifo.io.enq.bits := io.tag
  }

  io.chosen := num_threads.U
  fifo.io.deq.ready := false.B
  // deq fifo
  val tag = fifo.io.deq.bits
  when (fifo.io.count > 0.U && (tag(log2Up(num_regfile)-1, 0) === thread) && io.order_ready(tag)) {
    io.chosen := fifo.io.deq.bits
    fifo.io.deq.ready := true.B
  } .otherwise {
    io.chosen := chosen_i(thread)
  }
}

import chisel3._

import chisel3.util._

import chisel3.util.Fill


defined class Scheduler
defined class Scheduler_order

## Gather

In [21]:
import chisel3._
import chisel3.util._
import chisel3.util.Fill


class Gather(reg_width: Int, num_src_pos: Int, src_pos: Array[Int],
   max_out_width: Int, num_modes:Int, src_mode: Array[Int]) extends Module {
  val io = IO(new Bundle {
    val din = Input(UInt(reg_width.W))
    val shift = Input(UInt(log2Up(num_src_pos).W))
    val mode = Input(UInt(log2Up(num_modes).W))
    val dout = Output(UInt(reg_width.W))
  })

  val din_d0 = Reg(UInt(reg_width.W))
  val shift_d0 = Reg(UInt(log2Up(num_src_pos).W))
  val mode_d0 = Reg(UInt(log2Up(num_modes).W))

  val num_muxes : Int = (num_src_pos+7)/8
  val reg0 = Reg(Vec(num_muxes, UInt(max_out_width.W)))
  val src_pos_i = src_pos ++ Array(0, 0, 0, 0, 0, 0, 0)
  for (i <- 0 until num_muxes) {
    switch(io.shift(2, 0)) {
      is (0.U) {
        reg0(i) := io.din((src_pos_i(i*8)+max_out_width-1).min(reg_width-1), src_pos_i(i*8))
      }
      is (1.U) {
        reg0(i) := io.din((src_pos_i(i*8+1)+max_out_width-1).min(reg_width-1), src_pos_i(i*8+1))
      }
      is (2.U) {
        reg0(i) := io.din((src_pos_i(i*8+2)+max_out_width-1).min(reg_width-1), src_pos_i(i*8+2))
      }
      is (3.U) {
        reg0(i) := io.din((src_pos_i(i*8+3)+max_out_width-1).min(reg_width-1), src_pos_i(i*8+3))
      }
      is (4.U) {
        reg0(i) := io.din((src_pos_i(i*8+4)+max_out_width-1).min(reg_width-1), src_pos_i(i*8+4))
      }
      is (5.U) {
        reg0(i) := io.din((src_pos_i(i*8+5)+max_out_width-1).min(reg_width-1), src_pos_i(i*8+5))
      }
      is (6.U) {
        reg0(i) := io.din((src_pos_i(i*8+6)+max_out_width-1).min(reg_width-1), src_pos_i(i*8+6))
      }
      is (7.U) {
        reg0(i) := io.din((src_pos_i(i*8+7)+max_out_width-1).min(reg_width-1), src_pos_i(i*8+7))
      }
    }
  }

  din_d0 := io.din
  shift_d0 := (io.shift >> 3)
  mode_d0 := io.mode

  val reg1 = Reg(UInt(max_out_width.W))
  val din_d1 = Reg(UInt(reg_width.W))
  val mode_d1 = Reg(UInt(log2Up(num_modes).W))

  din_d1 := din_d0
  mode_d1 := mode_d0
  val cases = (0 until num_muxes).map( x => x.U -> reg0(x))
  reg1 := MuxLookup(shift_d0, DontCare, cases)

  val reg2 = Reg(UInt(max_out_width.W))
  val din_d2 = Reg(UInt(reg_width.W))
  val mode_d2 = Reg(UInt(log2Up(num_modes).W))
  din_d2 := din_d1
  mode_d2 := mode_d1

  val cases2 = (0 until num_modes).map( x => x.U -> reg1(src_mode(x)-1, 0))
  reg2 := MuxLookup(mode_d1, DontCare, cases2)
  if (max_out_width < reg_width) {
    io.dout := Cat(din_d2(reg_width-1, max_out_width), reg2)
  } else {
    io.dout := reg2
  }
}

class Scatter(reg_width: Int, lg_num_rdBlocks: Int, lg_num_modes: Int, num_wrBlocks: Int,
  num_dst_pos: Int, dst_encode: Array[Long], dst_pos: Array[Long],
  num_wbens: Int, dst_en_encode: Array[(Int, Int)], wbens: Array[Long]) extends Module {
  val io = IO(new Bundle {
    val din = Input(UInt(reg_width.W))
    val shift = Input(UInt(lg_num_rdBlocks.W))
    val mode = Input(UInt(lg_num_modes.W))
    val wren = Output(UInt(num_wrBlocks.W))
    val dout = Output(UInt(reg_width.W))
  })
  val din_w = Wire(UInt(reg_width.W))
  val dout_r = Reg(UInt(reg_width.W))
  val wben_r = Reg(UInt(num_wrBlocks.W))
  din_w := io.din

  val cases = (0 until num_dst_pos).map(i => (io.shift === dst_encode(i).U) -> (din_w << dst_pos(i)))
  dout_r := MuxCase(DontCare, cases)
  val cases2 = (0 until num_wbens).map(i => ((io.shift === dst_en_encode(i)._1.U) && ((dst_en_encode(i)._2 == -1).B || (io.mode === dst_en_encode(i)._2.S.asUInt))) -> wbens(i).U)
  wben_r := MuxCase(0.U, cases2)

  io.dout := dout_r
  io.wren := wben_r
}

class GatherScatterUnit(tag_width: Int, reg_width: Int, opcode_width: Int, num_threads: Int, ip_width: Int,
  num_src_pos: Int, src_pos: Array[Int], max_out_width: Int, num_modes:Int, src_mode: Array[Int], 
  num_regblocks: Int, num_dst_pos: Int, dst_encode: Array[Long], dst_pos: Array[Long],
  num_wbens: Int, dst_en_encode: Array[(Int, Int)], wbens: Array[Long]) extends Module {
  val io = IO(new Bundle{
    val in_valid      = Input(Bool())
    val in_tag        = Input(UInt(tag_width.W))
    val in_opcode     = Input(UInt(opcode_width.W))
    val in_imm        = Input(UInt(12.W))
    val in_bits       = Input(Vec(1, UInt(reg_width.W)))
    val in_ready      = Output(Bool())
    val out_valid     = Output(Bool())
    val out_tag       = Output(UInt(tag_width.W))
    val out_flag      = Output(UInt(ip_width.W))
    val out_bits      = Output(UInt(reg_width.W))
    val out_wben      = Output(UInt(num_regblocks.W))
    val out_ready     = Input(Bool())
  })

  val num_src_pos_lg = log2Up(num_src_pos)
  val num_dst_pos_lg = log2Up(num_dst_pos)
  val num_modes_lg = log2Up(num_modes)

  val GATHER_DELAY = 3
  val SCATTER_DELAY = 1
  val DELAY = GATHER_DELAY.max(SCATTER_DELAY)
  val DELAY_MIN = GATHER_DELAY.min(SCATTER_DELAY)

  // opcode(0), 0: extract(gather); 1: insert(scatter)
  // in_imm(num_src_pos_lg-1, 0): src_pos
  // in_imm(num_src_pos_lg+num_modes_lg-1, num_src_pos_lg): src_mode

  val shift_gather = io.in_imm(num_src_pos_lg-1, 0)
  val mode_gather = io.in_imm(num_src_pos_lg+num_modes_lg-1, num_src_pos_lg)
  val shift_scatter = io.in_imm(num_dst_pos_lg-1, 0)
  val mode_scatter = io.in_imm(num_dst_pos_lg+num_modes_lg-1, num_dst_pos_lg)
  val gather = Module(new Gather(reg_width, num_src_pos, src_pos, max_out_width, num_modes, src_mode))
  val scatter = Module(new Scatter(reg_width, num_src_pos_lg, num_modes_lg, num_regblocks, num_dst_pos, dst_encode, dst_pos, num_wbens, dst_en_encode, wbens))
  
  val valid_vec = RegInit(VecInit(Seq.fill(DELAY)(false.B)))
  val tag_vec = Reg(Vec(DELAY, UInt(tag_width.W)))
  val opcode_vec = Reg(Vec(DELAY, UInt(1.W)))
  val res_vec = Reg(Vec(DELAY-DELAY_MIN, UInt(reg_width.W)))
  val wren_vec = RegInit(VecInit(Seq.fill(DELAY-DELAY_MIN)(0.U(num_regblocks.W))))

  gather.io.din := io.in_bits(0)
  gather.io.shift := shift_gather
  gather.io.mode := mode_gather

  scatter.io.din := io.in_bits(0)
  scatter.io.shift := shift_scatter
  scatter.io.mode := mode_scatter

  valid_vec(0) := false.B
  when (io.in_valid) {
    valid_vec(0) := true.B
    tag_vec(0) := io.in_tag
    opcode_vec(0) := io.in_opcode(0)
  }

  for (i <- 1 until DELAY-1) {
    valid_vec(i) := valid_vec(i-1)
    tag_vec(i) := tag_vec(i-1)
    opcode_vec(i) := opcode_vec(i-1)
  }

  res_vec(0) := scatter.io.dout
  wren_vec(0) := scatter.io.wren
  for (i <- 1 until DELAY-DELAY_MIN-1) {
    res_vec(i) := res_vec(i-1)
    wren_vec(i) := wren_vec(i-1)
  }

  when (opcode_vec(DELAY-1) === 0.U) {
    io.out_bits := gather.io.dout
    io.out_wben := Fill(num_regblocks, 1.U)
  } .otherwise {
    io.out_bits := scatter.io.dout
    io.out_wben := scatter.io.wren
  }
  io.out_valid := valid_vec(DELAY-1)
  io.out_tag := tag_vec(DELAY-1)
  io.out_flag := 0.U

}


import chisel3._

import chisel3.util._

import chisel3.util.Fill



defined class Gather
defined class Scatter
defined class GatherScatterUnit

## BranchUnit BRU

In [22]:
import chisel3._
import chisel3.util._

class BranchUnit extends Module {
  val io = IO(new Bundle {
    val rs1       = Input(SInt(32.W))
    val rs2       = Input(SInt(32.W))
    val pc        = Input(SInt(32.W))
    val brValid   = Input(Bool())
    val brMode    = Input(UInt(4.W))
    val pcOffset  = Input(SInt(21.W))

    val pcOut     = Output(SInt(32.W))
    // val dout      = Output(SInt(32.W))
    val finish    = Output(Bool())
  })

  val pcInc = Wire(SInt(32.W))
  val pcOff = Wire(SInt(32.W))
  val pcPlain = Wire(SInt(32.W))
  val pcOut_r = Reg(SInt(32.W))
  val finish_r = RegInit(false.B)
  pcInc := io.pc + 1.S
  pcOff := io.pc + io.pcOffset
  pcPlain := io.pcOffset
  io.pcOut := pcOut_r
  // io.dout := pcInc
  io.finish := finish_r

  pcOut_r := pcInc
  finish_r := false.B
  when (io.brValid) {
    switch (io.brMode) {
      is (0.U) {
        when (io.rs1 === io.rs2) {
          pcOut_r := pcOff
        }
      }
      is (1.U) {
        when (io.rs1 =/= io.rs2) {
          pcOut_r := pcOff
        }
      }
      is (2.U) {
        pcOut_r := pcOff
      }
      is (3.U) {
        pcOut_r := pcOff
        when (pcPlain === -1.S) {
          finish_r := true.B
          println("A thread has stopped!")
        }
      }
      is (4.U) {
        when (io.rs1 < io.rs2) {
          pcOut_r := pcOff
        }
      }
      is (5.U) {
        when (io.rs1 >= io.rs2) {
          pcOut_r := pcOff
        }
      }
      is (6.U) {
        val rs1_u = io.rs1.asUInt
        val rs2_u = io.rs2.asUInt
        when (rs1_u < rs2_u) {
          pcOut_r := pcOff
        }
      }
      is (7.U) {
        val rs1_u = io.rs1.asUInt
        val rs2_u = io.rs2.asUInt
        when (rs1_u >= rs2_u) {
          pcOut_r := pcOff
        }
      }
      is (8.U) {
        pcOut_r := io.rs1 + io.pcOffset
      }
    }
  }
}

import chisel3._

import chisel3.util._


defined class BranchUnit

## primate top

In [23]:
class primate(extCompName: String) extends gComponentLeaf(new input_t, new output_t, ArrayBuffer(), extCompName + "__type__engine__MT__16__") {
  val filename = "./primate.cfg"
  val fileSource = Source.fromFile(filename)
  val lines = fileSource.getLines.toList
  var knobs:Map[String, String] = Map()
  for (line <- lines) {
    val Array(key, value) = line.split("=")
    knobs += (key -> value)
  }
  val NUM_THREADS = knobs.apply("NUM_THREADS").toInt
  val REG_WIDTH = knobs.apply("REG_WIDTH").toInt
  val NUM_REGS = knobs.apply("NUM_REGS").toInt
  val NUM_BFUS = knobs.apply("NUM_BFUS").toInt + 2 // IO and LSU are hidden lol
  val NUM_ALUS = knobs.apply("NUM_ALUS").toInt
  val IMM_WIDTH = knobs.apply("IMM_WIDTH").toInt
  val NUM_REGBLOCKS = knobs.apply("NUM_REGBLOCKS").toInt
  val NUM_SRC_POS = knobs.apply("NUM_SRC_POS").toInt
  val NUM_SRC_MODES = knobs.apply("NUM_SRC_MODES").toInt
  val NUM_DST_POS = knobs.apply("NUM_DST_POS").toInt
  val NUM_WB_ENS = knobs.apply("NUM_WB_ENS").toInt
  val MAX_FIELD_WIDTH = knobs.apply("MAX_FIELD_WIDTH").toInt
  val IP_WIDTH = knobs.apply("IP_WIDTH").toInt
  val REG_BLOCK_WIDTH:Array[Int] = knobs.apply("REG_BLOCK_WIDTH").split(" ").map(_.toInt)
  val src_pos:Array[Int] = knobs.apply("SRC_POS").split(" ").map(_.toInt)
  val src_mode:Array[Int] = knobs.apply("SRC_MODE").split(" ").map(_.toInt)
  val dst_encode:Array[Long] = knobs.apply("DST_ENCODE").split(" ").map(_.toLong)
  val dst_pos:Array[Long] = knobs.apply("DST_POS").split(" ").map(_.toLong)
  val wbens:Array[Long] = knobs.apply("DST_EN").split(" ").map(_.toLong)
  val dst_en_encode:Array[(Int, Int)] = knobs.apply("DST_EN_ENCODE").split(";").map(_.split(" ") match {case Array(a1, a2) => (a1.toInt, a2.toInt)})

  val NUM_THREADS_LG = log2Up(NUM_THREADS)
  val NUM_REGS_LG = log2Up(NUM_REGS)
  val NUM_FUOPS_LG = 6 // currently fixed to 6 due to the risc-v encoding
  val NUM_FUS = (NUM_BFUS).max(NUM_ALUS)
  val NUM_FUS_LG = log2Up(NUM_FUS)
  val NUM_RD_BANKS = scala.math.pow(2, log2Up(NUM_FUS)).toInt
  val NUM_RF_RD_PORTS = NUM_FUS
  val NUM_WR = (NUM_BFUS).max(NUM_ALUS)
  val NUM_RF_WR_PORTS = (NUM_WR+1)/2
  val NUM_WR_BANKS = scala.math.pow(2, log2Up(NUM_RF_WR_PORTS)).toInt
  val NUM_SRC_POS_LG = log2Up(NUM_SRC_POS)
  val NUM_SRC_MODES_LG = log2Up(NUM_SRC_MODES)
  val NUM_DST_POS_LG = log2Up(NUM_DST_POS)
  val NUM_DST_MODES_LG = log2Up(NUM_SRC_MODES)
  val INST_RAM_SIZE = 4096
  // FIXME
  val ALU_SRC_WIDTH = NUM_REGS_LG
  val SUBINSTR_WIDTH: Int = (math.ceil((7 + ALU_SRC_WIDTH * 3 + 3 + 7)/8) * 8).toInt
  val INSTR_WIDTH = SUBINSTR_WIDTH * (NUM_FUS + NUM_ALUS*3 + 1)

  val NONE_SELECTED = (NUM_THREADS).U((log2Up(NUM_THREADS+1)).W)

  val AllOffloadsReady = Reg(Bool())
  val AllOffloadsValid  = Reg(Vec(NUM_THREADS, Bool()))

  /////////////////////
  // DUMP THE PARAMS //
  /////////////////////
  println("PRIMATE PARAMS DUMP")
  println(" NUM_THREADS: " + NUM_THREADS)
  println(" REG_WIDTH: " + REG_WIDTH)
  println(" NUM_REGS: " + NUM_REGS)
  println(" NUM_BFUS: " + NUM_BFUS)
  println(" NUM_ALUS: " + NUM_ALUS)
  println(" NUM_FUS : " + NUM_FUS)
  println(" INSTR_WIDTH: " + INSTR_WIDTH)
  println(" SUBINSTR_WIDTH: " + SUBINSTR_WIDTH)

  /*******************Thread states*********************************/


// class gMemBundle extends Bundle with MemTrait {
//   val mem_addr      = Output(UInt(ADDR_WIDTH.W))
//   val read          = Output(Bool())
//   val write         = Output(Bool())
//   val writedata     = Output(UInt(DATA_WIDTH.W))
//   val byteenable    = Output(UInt((DATA_WIDTH/8).W))
//   val waitrequest   = Input(Bool())
//   val readdatavalid = Input(Bool())
//   val readdata      = Input(UInt(DATA_WIDTH.W))
// }
    
  // set up function units
  val ioUnit = Module(new inOutUnit(REG_WIDTH, NUM_REGS_LG, NUM_FUOPS_LG, NUM_THREADS, IP_WIDTH))
  val alu_bfuInst = Array(Module(new alu_bfu0(NUM_THREADS_LG, REG_WIDTH, NUM_FUOPS_LG, NUM_THREADS, IP_WIDTH)),
                          Module(new alu_bfu1(NUM_THREADS_LG, REG_WIDTH, NUM_FUOPS_LG, NUM_THREADS, IP_WIDTH)))  // def functionalUnits = io.elements("off")
  // def dynamicMemPort = functionalUnits.asInstanceOf[Bundle].elements("dynamicMem").asInstanceOf[gOffBundle[dyMemInput_t, llNode_t]]
  // val flowTablePort = Module(new flowTable(TAGWIDTH, NUM_THREADS))
  io.mem.mem_addr   := alu_bfuInst(0).io.mem.mem_addr   // ZX: those I/Os are all outputs from the gMemBoudle 
  io.mem.read       := alu_bfuInst(0).io.mem.read       // ZX: So is this assuming those io.mem are the same for each alu_bfu(i) ? 
  io.mem.write      := alu_bfuInst(0).io.mem.write      // ZX: where does io been defined, gInOutOffBundle at the top level?
  io.mem.writedata  := alu_bfuInst(0).io.mem.writedata
  io.mem.byteenable := alu_bfuInst(0).io.mem.byteenable
    
  alu_bfuInst(0).io.mem.waitrequest    := io.mem.waitrequest  // ZX: Those are inputs to the gMemBoudle
  alu_bfuInst(0).io.mem.readdatavalid  := io.mem.readdatavalid
  alu_bfuInst(0).io.mem.readdata       := io.mem.readdata

  for (i <- 1 until NUM_ALUS) {
    alu_bfuInst(i).io.mem.waitrequest   := false.B  // ZX: Taking some default value: always False, why?
    alu_bfuInst(i).io.mem.readdatavalid := false.B
    alu_bfuInst(i).io.mem.readdata      := DontCare
  }


  object ThreadStageEnum extends ChiselEnum {  // ZX: those state control should classified into control unit
    val idle   = Value
    val order_fetch = Value
    val fetch  = Value
    val decode = Value
    val read   = Value
    val pre    = Value
    val exec   = Value
    //val post   = Value
    val branch = Value
  }
  val threadStages = RegInit(VecInit(Seq.fill(NUM_THREADS)(ThreadStageEnum.idle))) // ZX: controls

  val ThreadStateT = new Bundle {         // ZX: controls
    val tag         = UInt((TAGWIDTH*2).W)
    val invalid     = Bool()
    // FIXME: input -> rf & rf -> output
    val ip          = UInt((IP_WIDTH+2).W)

    val bfuValids   = Vec(NUM_BFUS, Bool())
    val execValids  = Vec(NUM_BFUS, Bool())
    val io_dstPC    = UInt(IP_WIDTH.W)
    val execDone    = Bool()
    val finish      = Bool()
  }
  val threadStates  = Reg(Vec(NUM_THREADS, ThreadStateT))   // ZX: controls
    

  val regfile = Module(new RegRead(NUM_THREADS, NUM_RF_RD_PORTS, NUM_RF_WR_PORTS, NUM_REGS, REG_WIDTH, NUM_REGBLOCKS, REG_BLOCK_WIDTH))

// class BRMicrocodes(num_alus: Int, num_fus: Int) extends Bundle {
//   val brValid   = Bool()
//   val brMode    = UInt(4.W)
//   val rs1       = UInt(log2Up(num_alus*3+num_fus).W)
//   val rs2       = UInt(log2Up(num_alus*3+num_fus).W)
//   val pcOffset  = SInt(21.W)

// val io = IO(new Bundle {   // ZX: IO for ram_simple2port
//     val clock     = Input(Clock())
//     val data      = Input(UInt(width.W))
//     val rdaddress = Input(UInt(log2Up(num).W))
//     val rden      = Input(Bool())
//     val wraddress = Input(UInt(log2Up(num).W))
//     val wren      = Input(Bool())
//     val q         = Output(UInt(width.W))
    
  class ThreadMemT extends Bundle {
    val brUcodes   = new BRMicrocodes(NUM_ALUS, NUM_FUS)
    val rdWrEn     = Vec(NUM_FUS, Bool())
    val rd         = Vec(NUM_FUS, UInt(NUM_REGS_LG.W))
    val rd_pos     = Vec(NUM_ALUS, UInt(NUM_DST_POS_LG.W))
    val rd_mode    = Vec(NUM_ALUS, UInt(NUM_DST_MODES_LG.W))
  }

  class DestMemT extends Bundle {
    val dstPC      = UInt(32.W)
    val wben       = UInt(NUM_REGBLOCKS.W)
    val dest       = UInt(REG_WIDTH.W)
  }

  val threadMem = Module(new ram_simple2port(NUM_THREADS, (new ThreadMemT).getWidth))
  val destMems = Seq.fill(NUM_WR)(Module(new ram_simple2port(NUM_THREADS, (new DestMemT).getWidth)))
  threadMem.io.clock := clock 
  threadMem.io.rden := false.B
  threadMem.io.rdaddress := DontCare
  for (destMem <- destMems) {   // ZX: Init with those value?
    destMem.io.clock := clock
    destMem.io.wren := false.B
    destMem.io.rden := false.B
    destMem.io.wraddress := DontCare
    destMem.io.rdaddress := DontCare
    destMem.io.data := DontCare
  }

  /****************** Start Thread *********************************/
  // select idle thread
  val vThreadEncoder = Module(new Scheduler_order(NUM_THREADS, NUM_RD_BANKS))
  Range(0, NUM_THREADS, 1).map(i =>  // ZX: This is control, like a for loop assign state
    ioUnit.io.idle_threads(i) := threadStages(i) === ThreadStageEnum.idle) // idle_threads is bool, and vec

  ioUnit.io.in_valid := io.in.valid
  ioUnit.io.in_tag := io.in.tag
  ioUnit.io.in_data := io.in.bits.data
  ioUnit.io.in_empty := io.in.bits.empty
  ioUnit.io.in_last := io.in.last
  io.in.ready := ioUnit.io.in_ready

  when (ioUnit.io.new_thread) {
    // spawn new thread
    threadStages(ioUnit.io.new_tag) := ThreadStageEnum.order_fetch
    threadStates(ioUnit.io.new_tag).ip := 0.U((IP_WIDTH+2).W)
    vThreadEncoder.io.valid := true.B
    vThreadEncoder.io.tag := ioUnit.io.new_tag
    println("Primate core spawning new thread!")
  } .otherwise {
    vThreadEncoder.io.valid := false.B
    vThreadEncoder.io.tag := DontCare
  }

  /****************** Scheduler logic *********************************/
  // select valid thread
  val vThread = vThreadEncoder.io.chosen
  Range(0, NUM_THREADS, 1).map(i =>
    vThreadEncoder.io.order_ready(i) := (threadStages(i) === ThreadStageEnum.order_fetch))
  Range(0, NUM_THREADS, 1).map(i =>
    vThreadEncoder.io.ready(i) := (threadStages(i) === ThreadStageEnum.fetch))

  /****************** Fetch logic *********************************/
  val fetchUnit = Module(new Fetch(NUM_THREADS, IP_WIDTH, INSTR_WIDTH, INST_RAM_SIZE))
  val instr = Wire(UInt(INSTR_WIDTH.W))
  fetchUnit.io.ip := (threadStates(vThread).ip)
  instr := fetchUnit.io.instr

  when (vThread =/= NONE_SELECTED) {
      threadStages(vThread) := ThreadStageEnum.decode
  }

  /****************** Decode logic *********************************/
  val decodeThread = RegInit(NONE_SELECTED)
  decodeThread := vThread

  val decodeUnit = Module(new Decode(NUM_ALUS, NUM_BFUS, NUM_FUS, SUBINSTR_WIDTH, ALU_SRC_WIDTH, NUM_SRC_POS_LG, NUM_SRC_MODES_LG, NUM_DST_POS_LG, NUM_DST_MODES_LG))
  val aluMicrocodes_in = Wire(new ALUMicrocodes(NUM_ALUS, ALU_SRC_WIDTH, NUM_SRC_POS_LG, NUM_SRC_MODES_LG))
  val bfuMicrocodes_in = Wire(new BFUMicrocodes(NUM_FUS-NUM_ALUS, ALU_SRC_WIDTH))
  val rdWrEn_in = Wire(Vec(NUM_FUS, Bool()))
  val bfuValids_in = Wire(Vec(NUM_BFUS, Bool()))
  decodeUnit.io.instr := instr

  val BFU_RSPQ_DEPTH_LG = 3
  val BFU_RSPQ_DEPTH = 8 // (BFU_RSPQ_DEPTH/2 must be equal or greater than NUM_RF_RD_PORTS+1)
  val bfuRdReqQ = Module(new Queue(new BFU_regfile_req_t(NUM_THREADS_LG, NUM_REGS_LG), 4))
  val bfuRdRspQ = Module(new Queue(new BFU_regfile_rsp_t(REG_WIDTH), BFU_RSPQ_DEPTH))
  val bfuRdRspQ_almFull = Wire(Bool())
  val bfuRdRspQ_enq_in = Wire(Bool())
  val decode_valid = RegInit(false.B)
  val decodeThread_d1 = RegNext(decodeThread)

  bfuRdReqQ.io.deq.ready := false.B
  bfuRdReqQ.io.enq.valid := ioUnit.io.rd_req_valid
  bfuRdReqQ.io.enq.bits := ioUnit.io.rd_req
  ioUnit.io.rd_req_ready := bfuRdReqQ.io.enq.ready

  bfuRdRspQ_almFull := bfuRdRspQ.io.count(2).asBool
  bfuRdRspQ.io.enq.valid := false.B
  bfuRdRspQ_enq_in := false.B
  ioUnit.io.rd_rsp_valid := bfuRdRspQ.io.deq.valid
  ioUnit.io.rd_rsp := bfuRdRspQ.io.deq.bits
  bfuRdRspQ.io.deq.ready := ioUnit.io.rd_rsp_ready

  decode_valid := false.B
  when (decodeThread =/= NONE_SELECTED) {
    threadStates(decodeThread).bfuValids := decodeUnit.io.bfuValids
    threadStates(decodeThread).invalid := false.B
    threadStates(decodeThread).execValids := VecInit(Seq.fill(NUM_BFUS)(false.B))
    decode_valid := true.B

    println("Got a valid decode thread!")

    aluMicrocodes_in := decodeUnit.io.aluUcodes
    bfuMicrocodes_in := decodeUnit.io.bfuUcodes
    rdWrEn_in := decodeUnit.io.rdWrEn
    bfuValids_in := decodeUnit.io.bfuValids

    val threadMem_in = Wire(new ThreadMemT)
    threadMem_in.brUcodes  := decodeUnit.io.brUcodes
    threadMem_in.rdWrEn    := decodeUnit.io.rdWrEn
    threadMem_in.rd        := decodeUnit.io.rd
    threadMem_in.rd_pos    := decodeUnit.io.rd_pos
    threadMem_in.rd_mode   := decodeUnit.io.rd_mode
    threadMem.io.wraddress := decodeThread
    threadMem.io.wren      := true.B
    threadMem.io.data      := threadMem_in.asUInt
    regfile.io.thread_rd   := decodeThread
    regfile.io.rdEn        := true.B
    for (i <- 0 until NUM_RF_RD_PORTS) {
      regfile.io.rdAddr1(i) := decodeUnit.io.rs1(i)
      regfile.io.rdAddr2(i) := decodeUnit.io.rs2(i)
    }

    threadStages(decodeThread) := ThreadStageEnum.read
  }
  .otherwise {
    val initVec = RegInit(VecInit(Seq.fill(NUM_FUS)(false.B)))
    val initVecALU = RegInit(VecInit(Seq.fill(NUM_ALUS)(false.B)))
    val initVecBFU = RegInit(VecInit(Seq.fill(NUM_BFUS)(false.B)))
    aluMicrocodes_in.bfu_valid := initVecALU
    aluMicrocodes_in.opcode := DontCare
    aluMicrocodes_in.rs1_pos := DontCare
    aluMicrocodes_in.rs1_mode := DontCare
    aluMicrocodes_in.rs2_pos := DontCare
    aluMicrocodes_in.rs2_mode := DontCare
    aluMicrocodes_in.rd     := DontCare
    aluMicrocodes_in.addEn  := initVecALU
    aluMicrocodes_in.subEn  := initVecALU
    aluMicrocodes_in.sltEn  := initVecALU
    aluMicrocodes_in.sltuEn := initVecALU
    aluMicrocodes_in.andEn  := initVecALU
    aluMicrocodes_in.orEn   := initVecALU
    aluMicrocodes_in.xorEn  := initVecALU
    aluMicrocodes_in.sllEn  := initVecALU
    aluMicrocodes_in.srEn   := initVecALU
    aluMicrocodes_in.srMode := initVecALU
    aluMicrocodes_in.luiEn  := initVecALU
    aluMicrocodes_in.catEn  := initVecALU
    aluMicrocodes_in.immSel := initVecALU
    aluMicrocodes_in.imm    := DontCare

    bfuMicrocodes_in := DontCare

    rdWrEn_in := initVec
    bfuValids_in := initVecBFU

    threadMem.io.wraddress := DontCare
    threadMem.io.wren      := false.B
    threadMem.io.data      := DontCare
    threadStates(decodeThread).bfuValids := DontCare
    threadStates(decodeThread).execValids := DontCare

    regfile.io.thread_rd := DontCare
    regfile.io.rdEn := false.B
    regfile.io.rdAddr1 := DontCare
    regfile.io.rdAddr2 := DontCare

    val bfuRdTag = bfuRdReqQ.io.deq.bits.tag
    regfile.io.thread_rd := bfuRdReqQ.io.deq.bits.tag
    regfile.io.rdAddr1(0) := bfuRdReqQ.io.deq.bits.rdAddr1
    regfile.io.rdAddr2(0) := bfuRdReqQ.io.deq.bits.rdAddr2
    when (bfuRdReqQ.io.deq.valid && (!bfuRdRspQ_almFull) && ((!decode_valid) || 
      (bfuRdTag(log2Up(NUM_RF_RD_PORTS)-1, 0) =/= decodeThread_d1(log2Up(NUM_RF_RD_PORTS)-1, 0)))) {
      bfuRdRspQ_enq_in := true.B
      bfuRdReqQ.io.deq.ready := true.B
    }
  }

  /***************** Register read / Metadata FIFO enq *************************/
  val REG_DELAY = NUM_RF_RD_PORTS + 1
  val ALU_DELAY = 3 // 3 (gather)

  val readThread_vec = RegInit(VecInit(Seq.fill(REG_DELAY+ALU_DELAY)(NONE_SELECTED)))
  val aluMicrocodes_vec = Reg(Vec(REG_DELAY, new ALUMicrocodes(NUM_ALUS, ALU_SRC_WIDTH, NUM_SRC_POS_LG, NUM_SRC_MODES_LG)))
  val bfuMicrocodes_vec = Reg(Vec(REG_DELAY, new BFUMicrocodes(NUM_FUS-NUM_ALUS, ALU_SRC_WIDTH)))
  val bfuValids_vec = RegInit(VecInit(Seq.fill(REG_DELAY)(VecInit(Seq.fill(NUM_BFUS)(false.B)))))
  val bfuRdRspQ_enq_vec = RegInit(VecInit(Seq.fill(REG_DELAY)(false.B)))

  readThread_vec(REG_DELAY+ALU_DELAY-1) := decodeThread
  aluMicrocodes_vec(REG_DELAY-1) := aluMicrocodes_in
  bfuMicrocodes_vec(REG_DELAY-1) := bfuMicrocodes_in
  bfuValids_vec(REG_DELAY-1) := bfuValids_in
  bfuRdRspQ_enq_vec(REG_DELAY-1) := bfuRdRspQ_enq_in

  for (i <- 0 until REG_DELAY+ALU_DELAY-1) {
    readThread_vec(i) := readThread_vec(i+1)
  }
  for (i <- 0 until REG_DELAY-1) {
    aluMicrocodes_vec(i) := aluMicrocodes_vec(i+1)
    bfuMicrocodes_vec(i) := bfuMicrocodes_vec(i+1)
    bfuValids_vec(i) := bfuValids_vec(i+1)
    bfuRdRspQ_enq_vec(i) := bfuRdRspQ_enq_vec(i+1)
  }

  val srcA = Wire(Vec(NUM_FUS, UInt(REG_WIDTH.W))) // vector containing all the RS1s read from the regfile
  val srcB = Wire(Vec(NUM_FUS, UInt(REG_WIDTH.W))) // vector containing all the RS2s read from the regfile
  srcA := regfile.io.rdData1
  srcB := regfile.io.rdData2

  /****************** Gather Stage 0-2 *************************/
  val gatherThread = readThread_vec(ALU_DELAY)
  val aluMicrocodes_out = aluMicrocodes_vec(0)
  val bfuMicrocodes_out = bfuMicrocodes_vec(0)
  val bfuValids_out = bfuValids_vec(0)

  val gather_srcA = Seq.fill(NUM_ALUS)(Module(new Gather(REG_WIDTH, NUM_SRC_POS, src_pos, MAX_FIELD_WIDTH, NUM_SRC_MODES, src_mode)))
  val gather_srcB = Seq.fill(NUM_ALUS)(Module(new Gather(REG_WIDTH, NUM_SRC_POS, src_pos, MAX_FIELD_WIDTH, NUM_SRC_MODES, src_mode)))
  for (i <- 0 until NUM_ALUS) {
    gather_srcA(i).io.din   := srcA(i)
    gather_srcA(i).io.shift := aluMicrocodes_out.rs1_pos(i)
    gather_srcA(i).io.mode  := aluMicrocodes_out.rs1_mode(i)
    gather_srcB(i).io.din   := srcB(i)
    gather_srcB(i).io.shift := aluMicrocodes_out.rs2_pos(i)
    gather_srcB(i).io.mode  := aluMicrocodes_out.rs2_mode(i)
  }

  val bfuRdRspQ_enq_valid = bfuRdRspQ_enq_vec(0)
  val bfuRdRspQ_enq_bits = Wire(new BFU_regfile_rsp_t(REG_WIDTH))

  // BFU regfile dump rsp queue
  bfuRdRspQ_enq_bits.rdData1 := srcA(0)
  bfuRdRspQ_enq_bits.rdData2 := srcB(0)
  bfuRdRspQ.io.enq.valid := bfuRdRspQ_enq_valid
  bfuRdRspQ.io.enq.bits := bfuRdRspQ_enq_bits

  // BFU FIFOs
  val execBundle_io = new Bundle {
    val tag = UInt(NUM_THREADS_LG.W)
    val opcode = UInt(NUM_FUOPS_LG.W)
    val rd = UInt(NUM_REGS_LG.W)
    val imm = UInt(12.W)
    val bits = UInt(REG_WIDTH.W)
  }
  val fuFifos_iou = Module(new Queue(execBundle_io, NUM_THREADS))
  fuFifos_iou.io.enq.valid := false.B
  fuFifos_iou.io.enq.bits := DontCare

  if (NUM_BFUS > NUM_ALUS) {
    when (gatherThread =/= NONE_SELECTED) {
      when (bfuValids_out(NUM_BFUS-1) === true.B) {
        fuFifos_iou.io.enq.valid := true.B
        fuFifos_iou.io.enq.bits.tag := gatherThread
        fuFifos_iou.io.enq.bits.opcode := bfuMicrocodes_out.opcode(NUM_BFUS-1-NUM_ALUS)
        fuFifos_iou.io.enq.bits.rd := bfuMicrocodes_out.rd(NUM_BFUS-1-NUM_ALUS)
        fuFifos_iou.io.enq.bits.imm := bfuMicrocodes_out.bimm(NUM_BFUS-1-NUM_ALUS)
        fuFifos_iou.io.enq.bits.bits := srcA(NUM_BFUS-1)
      }
    }
  } else {
    when (gatherThread =/= NONE_SELECTED) {
      when (aluMicrocodes_out.bfu_valid(NUM_BFUS-1) === true.B) {
        fuFifos_iou.io.enq.valid := true.B
        fuFifos_iou.io.enq.bits.tag := gatherThread
        fuFifos_iou.io.enq.bits.opcode := aluMicrocodes_out.opcode(NUM_BFUS-1)
        fuFifos_iou.io.enq.bits.rd := aluMicrocodes_out.rd(NUM_BFUS-1)
        fuFifos_iou.io.enq.bits.imm := aluMicrocodes_out.imm(NUM_BFUS-1).asUInt
        fuFifos_iou.io.enq.bits.bits := srcA(NUM_BFUS-1)
      }
    }
  }


  /****************** ALU/BFU Stage 0 *************************/
  val preOpThread = readThread_vec(0)
  // ALUBFU FIFOs
  class execBundle_alu extends Bundle {
    val tag     = UInt(NUM_THREADS_LG.W)
    val opcode  = UInt(NUM_FUOPS_LG.W)
    val alu_bfu = Bool()
    val rs1     = UInt(REG_WIDTH.W)
    val rs2     = UInt(32.W)
    val addEn   = Bool()
    val subEn   = Bool()
    val sltEn   = Bool()
    val sltuEn  = Bool()
    val andEn   = Bool()
    val orEn    = Bool()
    val xorEn   = Bool()
    val sllEn   = Bool()
    val srEn    = Bool()
    val srMode  = Bool()
    val luiEn   = Bool()
    val catEn   = Bool()
    val immSel  = Bool()
    val imm     = SInt(32.W)
  }
  val fuFifos_alu = Seq.fill(NUM_ALUS)(Module(new Queue(new execBundle_alu, NUM_THREADS)))

  val aluMicrocodes_out_d0 = RegNext(aluMicrocodes_out)
  val aluMicrocodes_out_d1 = RegNext(aluMicrocodes_out_d0)
  val aluMicrocodes_out_d2 = RegNext(aluMicrocodes_out_d1)

  for (i <- 0 until NUM_ALUS) {
    fuFifos_alu(i).io.enq.valid := false.B
    fuFifos_alu(i).io.enq.bits := DontCare
    when (preOpThread =/= NONE_SELECTED) {
      val fuFifos_alu_in = Wire(new execBundle_alu)
      fuFifos_alu_in.tag     := preOpThread
      fuFifos_alu_in.opcode  := aluMicrocodes_out_d2.opcode(i)
      fuFifos_alu_in.alu_bfu := aluMicrocodes_out_d2.bfu_valid(i)
      fuFifos_alu_in.rs1     := gather_srcA(i).io.dout
      fuFifos_alu_in.rs2     := gather_srcB(i).io.dout
      fuFifos_alu_in.addEn   := aluMicrocodes_out_d2.addEn(i)
      fuFifos_alu_in.subEn   := aluMicrocodes_out_d2.subEn(i)
      fuFifos_alu_in.sltEn   := aluMicrocodes_out_d2.sltEn(i)
      fuFifos_alu_in.sltuEn  := aluMicrocodes_out_d2.sltuEn(i)
      fuFifos_alu_in.andEn   := aluMicrocodes_out_d2.andEn(i)
      fuFifos_alu_in.orEn    := aluMicrocodes_out_d2.orEn(i)
      fuFifos_alu_in.xorEn   := aluMicrocodes_out_d2.xorEn(i)
      fuFifos_alu_in.sllEn   := aluMicrocodes_out_d2.sllEn(i)
      fuFifos_alu_in.srEn    := aluMicrocodes_out_d2.srEn(i)
      fuFifos_alu_in.srMode  := aluMicrocodes_out_d2.srMode(i)
      fuFifos_alu_in.luiEn   := aluMicrocodes_out_d2.luiEn(i)
      fuFifos_alu_in.catEn   := aluMicrocodes_out_d2.catEn(i)
      fuFifos_alu_in.immSel  := aluMicrocodes_out_d2.immSel(i)
      fuFifos_alu_in.imm     := aluMicrocodes_out_d2.imm(i)
      fuFifos_alu(i).io.enq.bits := fuFifos_alu_in
      fuFifos_alu(i).io.enq.valid := true.B
    } .otherwise {
      fuFifos_alu(i).io.enq.bits := DontCare
      fuFifos_alu(i).io.enq.valid := false.B
    }
  }

  /****************** Function unit execution *********************************/
  val execThread = RegInit(NONE_SELECTED)
  val execThread_d0 = RegNext(execThread)
  execThread := preOpThread

  // FUs input
  for (i <- 0 until NUM_ALUS) {
    fuFifos_alu(i).io.deq.ready := false.B
    val deq = fuFifos_alu(i).io.deq.bits
    alu_bfuInst(i).io.in_alu_bfu := deq.alu_bfu
    alu_bfuInst(i).io.in_tag     := deq.tag
    alu_bfuInst(i).io.in_opcode  := deq.opcode
    alu_bfuInst(i).io.in_rs1     := deq.rs1
    alu_bfuInst(i).io.in_rs2     := deq.rs2
    alu_bfuInst(i).io.in_addEn   := deq.addEn
    alu_bfuInst(i).io.in_subEn   := deq.subEn
    alu_bfuInst(i).io.in_sltEn   := deq.sltEn
    alu_bfuInst(i).io.in_sltuEn  := deq.sltuEn
    alu_bfuInst(i).io.in_andEn   := deq.andEn
    alu_bfuInst(i).io.in_orEn    := deq.orEn
    alu_bfuInst(i).io.in_xorEn   := deq.xorEn
    alu_bfuInst(i).io.in_sllEn   := deq.sllEn
    alu_bfuInst(i).io.in_srEn    := deq.srEn
    alu_bfuInst(i).io.in_srMode  := deq.srMode
    alu_bfuInst(i).io.in_luiEn   := deq.luiEn
    alu_bfuInst(i).io.in_catEn   := deq.catEn
    alu_bfuInst(i).io.in_immSel  := deq.immSel
    alu_bfuInst(i).io.in_imm     := deq.imm
    when (fuFifos_alu(i).io.deq.valid) {
      alu_bfuInst(i).io.in_valid := true.B
      when (alu_bfuInst(i).io.in_ready) {
        fuFifos_alu(i).io.deq.ready := true.B
      }
    } .otherwise {
      alu_bfuInst(i).io.in_valid := false.B
    }
  }

  fuFifos_iou.io.deq.ready := false.B
  when (fuFifos_iou.io.deq.valid) {
    val deq = fuFifos_iou.io.deq
    ioUnit.io.ar_valid := true.B
    ioUnit.io.ar_tag := deq.bits.tag
    ioUnit.io.ar_opcode := deq.bits.opcode
    ioUnit.io.ar_rd := deq.bits.rd
    ioUnit.io.ar_bits := deq.bits.bits
    ioUnit.io.ar_imm := deq.bits.imm
    when (ioUnit.io.ar_ready) {
      fuFifos_iou.io.deq.ready := true.B
    }
  } .otherwise {
    ioUnit.io.ar_valid := false.B
    ioUnit.io.ar_tag := DontCare
    ioUnit.io.ar_opcode := DontCare
    ioUnit.io.ar_rd := DontCare
    ioUnit.io.ar_bits := DontCare
    ioUnit.io.ar_imm := DontCare
  }


  when (execThread_d0 =/= NONE_SELECTED) {
    threadStages(execThread_d0) := ThreadStageEnum.exec
  }

  // FUs output
  for (i <- 0 until NUM_ALUS) {
    alu_bfuInst(i).io.out_ready := true.B
    when (alu_bfuInst(i).io.out_valid) {
      val destMem_in = Wire(new DestMemT)
      destMem_in.dstPC := alu_bfuInst(i).io.out_flag
      destMem_in.dest := alu_bfuInst(i).io.out_bits
      destMem_in.wben := Fill(NUM_REGBLOCKS, 1.U)
      destMems(i).io.wren := true.B
      destMems(i).io.wraddress := alu_bfuInst(i).io.out_tag
      destMems(i).io.data := destMem_in.asUInt
      threadStates(alu_bfuInst(i).io.out_tag).execValids(i) := true.B
    }
  }

  when (ioUnit.io.w_valid) {
    threadStates(ioUnit.io.w_tag).execValids(NUM_BFUS-1) := true.B
    threadStates(ioUnit.io.w_tag).io_dstPC := ioUnit.io.w_flag
  }
  when (ioUnit.io.r_valid) {
    threadStates(ioUnit.io.r_tag).execValids(NUM_BFUS-1) := true.B
    threadStates(ioUnit.io.r_tag).io_dstPC := ioUnit.io.r_flag
  }


  /****************** Finish execution *********************************/
  val fThreadEncoder = Module(new Scheduler(NUM_THREADS, NUM_WR_BANKS))
  val fThread = fThreadEncoder.io.chosen
  val execDone = Wire(Vec(NUM_THREADS, Bool()))
  Range(0, NUM_THREADS, 1).foreach(i =>
    execDone(i) := (threadStates(i).execValids.asUInt | (~threadStates(i).bfuValids.asUInt)).andR
  )

  Range(0, NUM_THREADS, 1).map(i =>
    fThreadEncoder.io.valid(i) := (execDone(i) === true.B && threadStages(i) === ThreadStageEnum.exec))

  when (fThread =/= NONE_SELECTED) {
    threadStages(fThread) := ThreadStageEnum.branch
    for (destMem <- destMems) {
      destMem.io.rden := true.B
      destMem.io.rdaddress := fThread
    }
    threadMem.io.rden := true.B
    threadMem.io.rdaddress := fThread
  }

  /****************** Register write & branch *********************************/
  val WB_DELAY = NUM_RF_WR_PORTS.max(3)
  val branchThread_out = RegInit(NONE_SELECTED)
  val branchThread_vec = RegInit(VecInit(Seq.fill(WB_DELAY)(NONE_SELECTED)))
  branchThread_out := branchThread_vec(0)
  val branchU = Module(new BranchUnit)

  branchThread_vec(WB_DELAY-1) := fThread
  for (i <- 0 until WB_DELAY-1) {
    branchThread_vec(i) := branchThread_vec(i+1)
  }

  val threadMem_out = Wire(new ThreadMemT)
  val destMems_out = Wire(Vec(NUM_WR, (new DestMemT)))
  val destWbens_wb = RegInit(VecInit(Seq.fill(NUM_WR)(0.U(NUM_REGBLOCKS.W))))
  val dests_wb = Wire(Vec(NUM_WR, UInt(REG_WIDTH.W)))
  val dests_wb_s1 = Reg(Vec(NUM_WR, UInt(REG_WIDTH.W)))
  val rdWrEn_wb = RegInit(VecInit(Seq.fill(NUM_FUS)(false.B)))
  val rd_wb = Reg(Vec(NUM_FUS, UInt(NUM_REGS_LG.W)))
  val brUcodes_wb = Wire(new BRMicrocodes(NUM_ALUS, NUM_FUS))
  val dstPCs = Wire(Vec(NUM_FUS+3*NUM_ALUS, UInt(32.W)))

  dstPCs := DontCare
  if (NUM_ALUS < NUM_FUS) {
    // The last unit is the IO unit
    for (i <- 0 until NUM_FUS-1) {
      destMems_out(i) := destMems(i).io.q.asTypeOf(new DestMemT)
      dests_wb(i) := destMems_out(i).dest
      destWbens_wb(i) := destMems_out(i).wben
    }
    for (i <- 0 until NUM_ALUS) {
      dstPCs(4*i+2) := destMems_out(i).dstPC
    }
    for (i <- 0 until NUM_FUS-NUM_ALUS-1) {
      dstPCs(4*NUM_ALUS+i) := destMems_out(NUM_ALUS+i).dstPC
    }
    dstPCs(NUM_FUS+3*NUM_ALUS-1) := threadStates(branchThread_vec(WB_DELAY-2)).io_dstPC
  } else {
    // The IO unit is combined with an ALU unit
    for (i <- 0 until NUM_FUS) {
      destMems_out(i) := destMems(i).io.q.asTypeOf(new DestMemT)
      dests_wb(i) := destMems_out(i).dest
      destWbens_wb(i) := destMems_out(i).wben
    }
    for (i <- 0 until NUM_ALUS) {
      dstPCs(4*i+2) := destMems_out(i).dstPC
    }
    dstPCs(4*(NUM_BFUS-1)+2) := threadStates(branchThread_vec(WB_DELAY-2)).io_dstPC
  }

  threadMem_out := threadMem.io.q.asTypeOf(chiselTypeOf(threadMem_out))
  brUcodes_wb := threadMem_out.brUcodes
  rdWrEn_wb := threadMem_out.rdWrEn
  rd_wb := threadMem_out.rd
  dests_wb_s1 := dests_wb
  val pc = Reg(UInt((IP_WIDTH+2).W))
  val threadValid = RegInit(false.B)

  when (branchThread_vec(WB_DELAY-1) =/= NONE_SELECTED) {
    threadValid := true.B
    pc := threadStates(branchThread_vec(WB_DELAY-1)).ip
  } .otherwise {
    threadValid := false.B
    pc := DontCare
  }

  val thread_finish = Wire(Bool())
  val thread_new_pc = Wire(UInt((IP_WIDTH+2).W))
  val threadValid_s1 = RegNext(threadValid)
  val threadValid_s2 = RegNext(threadValid_s1)

  val caseFU = (0 until (NUM_FUS+3*NUM_ALUS)).map(i => (i.U -> dstPCs(i)))
  val branchU_rs1 = Wire(UInt(32.W))
  val branchU_rs2 = Wire(UInt(32.W))
  branchU_rs1 := MuxLookup(threadMem_out.brUcodes.rs1, DontCare, caseFU)
  branchU_rs2 := MuxLookup(threadMem_out.brUcodes.rs2, DontCare, caseFU)
  branchU.io.brValid := threadValid && brUcodes_wb.brValid
  branchU.io.rs1 := branchU_rs1.asSInt
  branchU.io.rs2 := branchU_rs2.asSInt
  branchU.io.brMode := threadMem_out.brUcodes.brMode
  branchU.io.pc := pc.asSInt
  branchU.io.pcOffset := threadMem_out.brUcodes.pcOffset

  // scatter
  val scatter = Seq.fill(NUM_ALUS)(Module(new Scatter(REG_WIDTH, NUM_SRC_POS_LG, NUM_SRC_MODES_LG, NUM_REGBLOCKS, NUM_DST_POS, dst_encode, dst_pos, NUM_WB_ENS, dst_en_encode, wbens)))
  for (i <- 0 until NUM_ALUS) {
    scatter(i).io.din := dests_wb(i)
    scatter(i).io.shift := threadMem_out.rd_pos(i)
    scatter(i).io.mode := threadMem_out.rd_mode(i)
  }

  when (threadValid_s1) {
    // writeback
    regfile.io.wrEn := true.B
    regfile.io.thread_wr := branchThread_vec(WB_DELAY-3)
    for (i <- 0 until NUM_RF_WR_PORTS) {
      if (i*2 < NUM_ALUS) {
        regfile.io.wrEn1(i) := rdWrEn_wb(i*2)
        regfile.io.wrAddr1(i) := rd_wb(i*2)
        regfile.io.wrBen1(i) := scatter(i*2).io.wren
        regfile.io.wrData1(i) := scatter(i*2).io.dout
      } else {
        regfile.io.wrEn1(i) := rdWrEn_wb(i*2)
        regfile.io.wrAddr1(i) := rd_wb(i*2)
        regfile.io.wrBen1(i) := destWbens_wb(i*2)
        regfile.io.wrData1(i) := dests_wb_s1(i*2)
      }
      if (i*2+1 < NUM_ALUS) {
        regfile.io.wrEn2(i) := rdWrEn_wb(i*2+1)
        regfile.io.wrAddr2(i) := rd_wb(i*2+1)
        regfile.io.wrBen2(i) := scatter(i*2+1).io.wren
        regfile.io.wrData2(i) := scatter(i*2+1).io.dout
      } else if (i*2+1 < NUM_WR) {
        regfile.io.wrEn2(i) := rdWrEn_wb(i*2+1)
        regfile.io.wrAddr2(i) := rd_wb(i*2+1)
        regfile.io.wrBen2(i) := destWbens_wb(i*2+1)
        regfile.io.wrData2(i) := dests_wb_s1(i*2+1)
      } else {
        regfile.io.wrEn2(i) := false.B
        regfile.io.wrAddr2(i) := DontCare
        regfile.io.wrBen2(i) := DontCare
        regfile.io.wrData2(i) := DontCare
      }
    }
  } .otherwise {
    regfile.io.wrEn := false.B
    regfile.io.thread_wr := ioUnit.io.w_tag
    regfile.io.wrEn1(0) := ioUnit.io.w_wen(0)
    regfile.io.wrAddr1(0) := ioUnit.io.w_addr(0)
    regfile.io.wrBen1(0) := Fill(NUM_REGBLOCKS, 1.U)
    regfile.io.wrData1(0) := ioUnit.io.w_data(0)(REG_WIDTH-1, 0)
    regfile.io.wrEn2(0) := ioUnit.io.w_wen(1)
    regfile.io.wrAddr2(0) := ioUnit.io.w_addr(1)
    regfile.io.wrBen2(0) := Fill(NUM_REGBLOCKS, 1.U)
    regfile.io.wrData2(0) := ioUnit.io.w_data(1)(REG_WIDTH-1, 0)
    for (i <- 1 until NUM_RF_WR_PORTS) {
      regfile.io.wrEn1(i) := false.B
      regfile.io.wrAddr1(i) := DontCare
      regfile.io.wrBen1(i) := Fill(NUM_REGBLOCKS, 0.U)
      regfile.io.wrData1(i) := DontCare
      regfile.io.wrEn2(i) := false.B
      regfile.io.wrAddr2(i) := DontCare
      regfile.io.wrBen2(i) := Fill(NUM_REGBLOCKS, 0.U)
      regfile.io.wrData2(i) := DontCare
    }
  }


  thread_finish := branchU.io.finish
  thread_new_pc := branchU.io.pcOut.asUInt
  when (threadValid_s1) {
    threadStates(branchThread_vec(WB_DELAY-3)).ip := thread_new_pc
    when (thread_finish) {
      threadStages(branchThread_vec(WB_DELAY-3)) := ThreadStageEnum.idle
    } .otherwise {
      threadStages(branchThread_vec(WB_DELAY-3)) := ThreadStageEnum.fetch
    }
  } .elsewhen (branchThread_vec(WB_DELAY-3) =/= NONE_SELECTED) {
    threadStages(branchThread_vec(WB_DELAY-3)) := ThreadStageEnum.fetch
  }

  // Output
  io.out.valid := ioUnit.io.out_valid
  io.out.tag := ioUnit.io.out_tag
  io.out.bits.data := ioUnit.io.out_data
  io.out.bits.empty := ioUnit.io.out_empty
  io.out.last := ioUnit.io.out_last
  ioUnit.io.out_ready := io.out.ready

  // Input unit output ready
  val w_tag = ioUnit.io.w_tag
  val lastThreadWr = RegNext(branchThread_vec(WB_DELAY-3))
  ioUnit.io.w_ready := (!threadValid_s1) && ((!threadValid_s2) || (w_tag(log2Up(NUM_RF_WR_PORTS)-1, 0) =/= lastThreadWr(log2Up(NUM_RF_WR_PORTS)-1, 0)))

}


defined class primate

In [24]:
class inOutUnitBundle(reg_width: Int, 
                      num_regs_lg: Int, 
                      opcode_width: Int, 
                      num_threads: Int, 
                      ip_width: Int) extends Bundle {
  val in_valid     = Input(Bool())
  val in_tag       = Input(UInt(log2Up(num_threads).W))
  val in_data      = Input(UInt(512.W))
  val in_empty     = Input(UInt(6.W))
  val in_last      = Input(Bool())
  val in_ready     = Output(Bool())

  val out_ready    = Input(Bool())
  val out_valid    = Output(Bool())
  val out_tag      = Output(UInt(log2Up(num_threads).W))
  val out_data     = Output(UInt(512.W))
  val out_empty    = Output(UInt(6.W))
  val out_last     = Output(Bool())

  val ar_valid     = Input(Bool())
  val ar_tag       = Input(UInt(log2Up(num_threads).W))
  val ar_opcode    = Input(UInt(opcode_width.W))
  val ar_rd        = Input(UInt(num_regs_lg.W))
  val ar_bits      = Input(UInt(reg_width.W))
  val ar_imm       = Input(UInt(32.W))
  val ar_ready     = Output(Bool())

  val w_ready      = Input(Bool())
  val w_valid      = Output(Bool())
  val w_tag        = Output(UInt(log2Up(num_threads).W))
  val w_flag       = Output(UInt(ip_width.W))
  val w_wen        = Output(Vec(2, Bool()))
  val w_addr       = Output(Vec(2, UInt(num_regs_lg.W)))
  val w_data       = Output(Vec(2, UInt(reg_width.W)))

  val idle_threads = Input(Vec(num_threads, Bool()))
  val new_thread   = Output(Bool())
  val new_tag      = Output(UInt(log2Up(num_threads).W))

  val r_valid      = Output(Bool())
  val r_flag       = Output(UInt(ip_width.W))
  val r_tag        = Output(UInt(log2Up(num_threads).W))

  val rd_req_valid = Output(Bool())
  val rd_req_ready = Input(Bool())
  val rd_req       = Output(new BFU_regfile_req_t(log2Up(num_threads), num_regs_lg))
  val rd_rsp_valid = Input(Bool())
  val rd_rsp_ready = Output(Bool())
  val rd_rsp       = Input(new BFU_regfile_rsp_t(reg_width))
}


defined class inOutUnitBundle

## new top level control

In [24]:
// class ControlUnit(NUM_THREADS: Int, 
//                   TAGWIDTH: Int, 
//                   IP_WIDTH: Int, 
//                   NUM_BFUS: Int, 
//                   NUM_RD_BANKS: Int, 
//                   INSTR_WIDTH: Int, 
//                   INST_RAM_SIZE: Int, 
//                   REG_WIDTH: Int, 
//                   NUM_REGS_LG: Int, 
//                   NUM_FUOPS_LG: Int
//                  ) extends Module {

//     val io = IO(new Bundle {
//         // Add ioUnit interface
//         val ioUnit = new inOutUnitBundle(NUM_THREADS, NUM_RD_BANKS, 512, 6, IP_WIDTH)  // reg_width: Int, num_regs_lg: Int, opcode_width: Int, num_threads: Int, ip_width: Int
//                                                                                     // REG_WIDTH,      NUM_REGS_LG,      NUM_FUOPS_LG,      NUM_THREADS,      IP_WIDTH
        
//         // Add other ports
//         // val data_str = Input(UInt(8.W)) // Example of an additional regular input port
//         // val fetch_done = Output(Bool()) // Example of an additional regular output port
//     })
//     val NONE_SELECTED = (NUM_THREADS).U((log2Up(NUM_THREADS+1)).W)
    
//     /*******************Thread states*********************************/
//     object ThreadStageEnum extends ChiselEnum {  // ZX: those state control should classified into control unit
//         val idle   = Value
//         val order_fetch = Value
//         val fetch  = Value
//         val decode = Value
//         val read   = Value
//         val pre    = Value
//         val exec   = Value
//         //val post   = Value
//         val branch = Value
//     }
//     val threadStages = RegInit(VecInit(Seq.fill(NUM_THREADS)(ThreadStageEnum.idle))) // ZX: controls, init all the thread stage as idle
    
//     val ThreadStateT = new Bundle {         // ZX: controls
//         val tag         = UInt((TAGWIDTH*2).W)
//         val invalid     = Bool()
//         // FIXME: input -> rf & rf -> output
//         val ip          = UInt((IP_WIDTH+2).W)
        
//         val bfuValids   = Vec(NUM_BFUS, Bool())
//         val execValids  = Vec(NUM_BFUS, Bool())
//         val io_dstPC    = UInt(IP_WIDTH.W)
//         val execDone    = Bool()
//         val finish      = Bool()
//     }
    
//     val threadStates  = Reg(Vec(NUM_THREADS, ThreadStateT))

//     /****************** Start Thread *********************************/
//     // select idle thread
//     val vThreadEncoder = Module(new Scheduler_order(NUM_THREADS, NUM_RD_BANKS))
//     Range(0, NUM_THREADS, 1).map(i =>
//         io.ioUnit.idle_threads(i) := threadStages(i) === ThreadStageEnum.idle) // idle_threads is bool, and vec   //?? Here, all those be true.B??
//                                                                         // !!! need output port for: ioUnit.io.idle_threads(i) !!!, or make a boundle to take all the io of ioUnit

//     when (io.ioUnit.new_thread) {
//         // spawn new thread
//         threadStages(io.ioUnit.new_tag) := ThreadStageEnum.order_fetch  // !!! need input port for: ioUnit.io.new_thread, ioUnit.io.new_tag !!!
//         threadStates(io.ioUnit.new_tag).ip := 0.U((IP_WIDTH+2).W)
//         vThreadEncoder.io.valid := true.B
//         vThreadEncoder.io.tag := io.ioUnit.new_tag
//         println("Primate core spawning new thread!")
//     } .otherwise {
//         vThreadEncoder.io.valid := false.B
//         vThreadEncoder.io.tag := DontCare
//     }
    

//     /****************** Scheduler logic *********************************/
//     // select valid thread
//     val vThread = vThreadEncoder.io.chosen
//     Range(0, NUM_THREADS, 1).map(i =>
//         vThreadEncoder.io.order_ready(i) := (threadStages(i) === ThreadStageEnum.order_fetch))
//     Range(0, NUM_THREADS, 1).map(i =>
//         vThreadEncoder.io.ready(i) := (threadStages(i) === ThreadStageEnum.fetch))
    

//     /****************** Fetch logic *********************************/
//     val fetchUnit = Module(new Fetch(NUM_THREADS, IP_WIDTH, INSTR_WIDTH, INST_RAM_SIZE))
//     val instr = Wire(UInt(INSTR_WIDTH.W))
//     fetchUnit.io.ip := (threadStates(vThread).ip)
//     instr := fetchUnit.io.instr
    
//     when (vThread =/= NONE_SELECTED) {
//         threadStages(vThread) := ThreadStageEnum.decode
//     }
// }

    
    
// //     /****************** Decode logic *********************************/
// //     val decodeThread = RegInit(NONE_SELECTED)
// //     decodeThread := vThread
    
// //     val decodeUnit = Module(new Decode(NUM_ALUS, NUM_BFUS, NUM_FUS, SUBINSTR_WIDTH, ALU_SRC_WIDTH, NUM_SRC_POS_LG, NUM_SRC_MODES_LG, NUM_DST_POS_LG, NUM_DST_MODES_LG)) // control
// //     val aluMicrocodes_in = Wire(new ALUMicrocodes(NUM_ALUS, ALU_SRC_WIDTH, NUM_SRC_POS_LG, NUM_SRC_MODES_LG))  // op code etc, control
// //     val bfuMicrocodes_in = Wire(new BFUMicrocodes(NUM_FUS-NUM_ALUS, ALU_SRC_WIDTH))
// //     val rdWrEn_in = Wire(Vec(NUM_FUS, Bool()))
// //     val bfuValids_in = Wire(Vec(NUM_BFUS, Bool()))
// //     decodeUnit.io.instr := instr
    
// //     val BFU_RSPQ_DEPTH_LG = 3  
// //     val BFU_RSPQ_DEPTH = 8 // (BFU_RSPQ_DEPTH/2 must be equal or greater than NUM_RF_RD_PORTS+1) 
// //     val bfuRdReqQ = Module(new Queue(new BFU_regfile_req_t(NUM_THREADS_LG, NUM_REGS_LG), 4))   // ZX: Diffcult to extract the bfuRdReqQ out because of the if statement
// //     val bfuRdRspQ = Module(new Queue(new BFU_regfile_rsp_t(REG_WIDTH), BFU_RSPQ_DEPTH))   
// //     val bfuRdRspQ_almFull = Wire(Bool())   
// //     val bfuRdRspQ_enq_in = Wire(Bool())    
// //     val decode_valid = RegInit(false.B) 
// //     val decodeThread_d1 = RegNext(decodeThread)
    
// //     bfuRdReqQ.io.deq.ready := false.B
// //     bfuRdReqQ.io.enq.valid := ioUnit.io.rd_req_valid // !!! need input port for: ioUnit.io.rd_req_valid, rd_req, rd_rsp_ready !!!
// //     bfuRdReqQ.io.enq.bits := ioUnit.io.rd_req
// //     ioUnit.io.rd_req_ready := bfuRdReqQ.io.enq.ready  // !!! need output port for: ioUnit.io.rd_req_ready, rd_rsp_valid, rd_rsp !!!
    
// //     bfuRdRspQ_almFull := bfuRdRspQ.io.count(2).asBool
// //     bfuRdRspQ.io.enq.valid := false.B
// //     bfuRdRspQ_enq_in := false.B
// //     ioUnit.io.rd_rsp_valid := bfuRdRspQ.io.deq.valid
// //     ioUnit.io.rd_rsp := bfuRdRspQ.io.deq.bits
// //     bfuRdRspQ.io.deq.ready := ioUnit.io.rd_rsp_ready

// //   decode_valid := false.B
// //   when (decodeThread =/= NONE_SELECTED) {
// //     threadStates(decodeThread).bfuValids := decodeUnit.io.bfuValids
// //     threadStates(decodeThread).invalid := false.B
// //     threadStates(decodeThread).execValids := VecInit(Seq.fill(NUM_BFUS)(false.B))
// //     decode_valid := true.B

// //     println("Got a valid decode thread!")

// //     aluMicrocodes_in := decodeUnit.io.aluUcodes
// //     bfuMicrocodes_in := decodeUnit.io.bfuUcodes
// //     rdWrEn_in := decodeUnit.io.rdWrEn
// //     bfuValids_in := decodeUnit.io.bfuValids

// //     val threadMem_in = Wire(new ThreadMemT)  
// //     threadMem_in.brUcodes  := decodeUnit.io.brUcodes
// //     threadMem_in.rdWrEn    := decodeUnit.io.rdWrEn
// //     threadMem_in.rd        := decodeUnit.io.rd
// //     threadMem_in.rd_pos    := decodeUnit.io.rd_pos
// //     threadMem_in.rd_mode   := decodeUnit.io.rd_mode

// //     threadMem.io.wraddress := decodeThread        // !!! need output port for: threadMem.io.wraddress, wren, data !!!
// //     threadMem.io.wren      := true.B
// //     threadMem.io.data      := threadMem_in.asUInt  // ZX: assign the threadMem_in Wire as int?
      
// //     regfile.io.thread_rd   := decodeThread   // !!! need  port for: regfile !!!
// //     regfile.io.rdEn        := true.B
      
// //     for (i <- 0 until NUM_RF_RD_PORTS) {
// //       regfile.io.rdAddr1(i) := decodeUnit.io.rs1(i)
// //       regfile.io.rdAddr2(i) := decodeUnit.io.rs2(i)
// //     }

// //     threadStages(decodeThread) := ThreadStageEnum.read
// //   }
// //   .otherwise {
// //     val initVec = RegInit(VecInit(Seq.fill(NUM_FUS)(false.B)))
// //     val initVecALU = RegInit(VecInit(Seq.fill(NUM_ALUS)(false.B)))
// //     val initVecBFU = RegInit(VecInit(Seq.fill(NUM_BFUS)(false.B)))
// //     aluMicrocodes_in.bfu_valid := initVecALU
// //     aluMicrocodes_in.opcode := DontCare
// //     aluMicrocodes_in.rs1_pos := DontCare
// //     aluMicrocodes_in.rs1_mode := DontCare
// //     aluMicrocodes_in.rs2_pos := DontCare
// //     aluMicrocodes_in.rs2_mode := DontCare
// //     aluMicrocodes_in.rd     := DontCare
// //     aluMicrocodes_in.addEn  := initVecALU  // ZX: what is this doing? is initVecALU one bit?
// //     aluMicrocodes_in.subEn  := initVecALU
// //     aluMicrocodes_in.sltEn  := initVecALU
// //     aluMicrocodes_in.sltuEn := initVecALU
// //     aluMicrocodes_in.andEn  := initVecALU
// //     aluMicrocodes_in.orEn   := initVecALU
// //     aluMicrocodes_in.xorEn  := initVecALU
// //     aluMicrocodes_in.sllEn  := initVecALU
// //     aluMicrocodes_in.srEn   := initVecALU
// //     aluMicrocodes_in.srMode := initVecALU
// //     aluMicrocodes_in.luiEn  := initVecALU
// //     aluMicrocodes_in.catEn  := initVecALU
// //     aluMicrocodes_in.immSel := initVecALU
// //     aluMicrocodes_in.imm    := DontCare

// //     bfuMicrocodes_in := DontCare

// //     rdWrEn_in := initVec
// //     bfuValids_in := initVecBFU

// //     threadMem.io.wraddress := DontCare
// //     threadMem.io.wren      := false.B
// //     threadMem.io.data      := DontCare
// //     threadStates(decodeThread).bfuValids := DontCare
// //     threadStates(decodeThread).execValids := DontCare

// //     regfile.io.thread_rd := DontCare
// //     regfile.io.rdEn := false.B
// //     regfile.io.rdAddr1 := DontCare
// //     regfile.io.rdAddr2 := DontCare

// //     val bfuRdTag = bfuRdReqQ.io.deq.bits.tag
// //     regfile.io.thread_rd := bfuRdReqQ.io.deq.bits.tag
// //     regfile.io.rdAddr1(0) := bfuRdReqQ.io.deq.bits.rdAddr1
// //     regfile.io.rdAddr2(0) := bfuRdReqQ.io.deq.bits.rdAddr2
// //     when (bfuRdReqQ.io.deq.valid && (!bfuRdRspQ_almFull) && ((!decode_valid) || 
// //         (bfuRdTag(log2Up(NUM_RF_RD_PORTS)-1, 0) =/= decodeThread_d1(log2Up(NUM_RF_RD_PORTS)-1, 0)))) {
// //       bfuRdRspQ_enq_in := true.B
// //       bfuRdReqQ.io.deq.ready := true.B
// //     }
// //   }
    
    
// // }

## Modified Top

In [24]:
// class primate_m(extCompName: String) extends gComponentLeaf(new input_t, new output_t, ArrayBuffer(), extCompName + "__type__engine__MT__16__") {
//     val filename = "./primate.cfg"
//     val fileSource = Source.fromFile(filename)
//     val lines = fileSource.getLines.toList
//     var knobs:Map[String, String] = Map()
//     for (line <- lines) {
//     val Array(key, value) = line.split("=")
//         knobs += (key -> value)
//     }
//     val NUM_THREADS = knobs.apply("NUM_THREADS").toInt
//     val REG_WIDTH = knobs.apply("REG_WIDTH").toInt
//     val NUM_REGS = knobs.apply("NUM_REGS").toInt
//     val NUM_BFUS = knobs.apply("NUM_BFUS").toInt + 2 // IO and LSU are hidden lol
//     val NUM_ALUS = knobs.apply("NUM_ALUS").toInt
//     val IMM_WIDTH = knobs.apply("IMM_WIDTH").toInt
//     val NUM_REGBLOCKS = knobs.apply("NUM_REGBLOCKS").toInt
//     val NUM_SRC_POS = knobs.apply("NUM_SRC_POS").toInt
//     val NUM_SRC_MODES = knobs.apply("NUM_SRC_MODES").toInt
//     val NUM_DST_POS = knobs.apply("NUM_DST_POS").toInt
//     val NUM_WB_ENS = knobs.apply("NUM_WB_ENS").toInt
//     val MAX_FIELD_WIDTH = knobs.apply("MAX_FIELD_WIDTH").toInt
//     val IP_WIDTH = knobs.apply("IP_WIDTH").toInt
//     val REG_BLOCK_WIDTH:Array[Int] = knobs.apply("REG_BLOCK_WIDTH").split(" ").map(_.toInt)
//     val src_pos:Array[Int] = knobs.apply("SRC_POS").split(" ").map(_.toInt)
//     val src_mode:Array[Int] = knobs.apply("SRC_MODE").split(" ").map(_.toInt)
//     val dst_encode:Array[Long] = knobs.apply("DST_ENCODE").split(" ").map(_.toLong)
//     val dst_pos:Array[Long] = knobs.apply("DST_POS").split(" ").map(_.toLong)
//     val wbens:Array[Long] = knobs.apply("DST_EN").split(" ").map(_.toLong)
//     val dst_en_encode:Array[(Int, Int)] = knobs.apply("DST_EN_ENCODE").split(";").map(_.split(" ") match {case Array(a1, a2) => (a1.toInt, a2.toInt)})
    
//     val NUM_THREADS_LG = log2Up(NUM_THREADS)
//     val NUM_REGS_LG = log2Up(NUM_REGS)
//     val NUM_FUOPS_LG = 6 // currently fixed to 6 due to the risc-v encoding
//     val NUM_FUS = (NUM_BFUS).max(NUM_ALUS)
//     val NUM_FUS_LG = log2Up(NUM_FUS)
//     val NUM_RD_BANKS = scala.math.pow(2, log2Up(NUM_FUS)).toInt
//     val NUM_RF_RD_PORTS = NUM_FUS
//     val NUM_WR = (NUM_BFUS).max(NUM_ALUS)
//     val NUM_RF_WR_PORTS = (NUM_WR+1)/2
//     val NUM_WR_BANKS = scala.math.pow(2, log2Up(NUM_RF_WR_PORTS)).toInt
//     val NUM_SRC_POS_LG = log2Up(NUM_SRC_POS)
//     val NUM_SRC_MODES_LG = log2Up(NUM_SRC_MODES)
//     val NUM_DST_POS_LG = log2Up(NUM_DST_POS)
//     val NUM_DST_MODES_LG = log2Up(NUM_SRC_MODES)
//     val INST_RAM_SIZE = 4096
//     // FIXME
//     val ALU_SRC_WIDTH = NUM_REGS_LG
//     val SUBINSTR_WIDTH: Int = (math.ceil((7 + ALU_SRC_WIDTH * 3 + 3 + 7)/8) * 8).toInt
//     val INSTR_WIDTH = SUBINSTR_WIDTH * (NUM_FUS + NUM_ALUS*3 + 1)
    
//     val NONE_SELECTED = (NUM_THREADS).U((log2Up(NUM_THREADS+1)).W)
    
//     val AllOffloadsReady = Reg(Bool())
//     val AllOffloadsValid  = Reg(Vec(NUM_THREADS, Bool()))

//     /////////////////////
//     // DUMP THE PARAMS //
//     /////////////////////
//     println("PRIMATE PARAMS DUMP")
//     println(" NUM_THREADS: " + NUM_THREADS)
//     println(" REG_WIDTH: " + REG_WIDTH)
//     println(" NUM_REGS: " + NUM_REGS)
//     println(" NUM_BFUS: " + NUM_BFUS)
//     println(" NUM_ALUS: " + NUM_ALUS)
//     println(" NUM_FUS : " + NUM_FUS)
//     println(" INSTR_WIDTH: " + INSTR_WIDTH)
//     println(" SUBINSTR_WIDTH: " + SUBINSTR_WIDTH)
    
//     // set up function units, and init io port
//     val ioUnit = Module(new inOutUnit(REG_WIDTH, NUM_REGS_LG, NUM_FUOPS_LG, NUM_THREADS, IP_WIDTH))
//     // Those are inputs to the ioUnit
//     ioUnit.io.in_valid := io.in.valid
//     ioUnit.io.in_tag := io.in.tag
//     ioUnit.io.in_data := io.in.bits.data
//     ioUnit.io.in_empty := io.in.bits.empty
//     ioUnit.io.in_last := io.in.last
//     io.in.ready := ioUnit.io.in_ready

    
//     // Instantiate ControlUnit
//     val controlUnit = Module(new ControlUnit(
//                                             NUM_THREADS, 
//                                             TAGWIDTH, 
//                                             IP_WIDTH, 
//                                             NUM_BFUS, 
//                                             NUM_RD_BANKS, 
//                                             INSTR_WIDTH, 
//                                             INST_RAM_SIZE, 
//                                             REG_WIDTH, 
//                                             NUM_REGS_LG, 
//                                             NUM_FUOPS_LG))
    
//     // Connect specific ioUnit ports to ControlUnit
//     controlUnit.io.ioUnit.new_thread := ioUnit.io.new_thread 
//     controlUnit.io.ioUnit.new_tag := ioUnit.io.new_tag
//     ioUnit.io.idle_threads := controlUnit.io.ioUnit.idle_threads
    
//     // Connect other specific ports if needed
//     // controlUnit.io.data_str := io.in.bits.data(7, 0) // Example: 8-bit data from the input
//     // controlUnit.io.fetch_done := DontCare // Initialize with a placeholder if unused initially
//     //////

//     // create an array of bfu, and init the port connection
//     val alu_bfuInst = Array(Module(new alu_bfu0(NUM_THREADS_LG, REG_WIDTH, NUM_FUOPS_LG, NUM_THREADS, IP_WIDTH)),
//                           Module(new alu_bfu1(NUM_THREADS_LG, REG_WIDTH, NUM_FUOPS_LG, NUM_THREADS, IP_WIDTH)))  // def functionalUnits = io.elements("off")
//     // def dynamicMemPort = functionalUnits.asInstanceOf[Bundle].elements("dynamicMem").asInstanceOf[gOffBundle[dyMemInput_t, llNode_t]]
//     // val flowTablePort = Module(new flowTable(TAGWIDTH, NUM_THREADS))

//     // output from the bfu
//     io.mem.mem_addr   := alu_bfuInst(0).io.mem.mem_addr
//     io.mem.read       := alu_bfuInst(0).io.mem.read       // ZX: So is this assuming those io.mem are the same for each alu_bfu(i), but only use (0) ? 
//     io.mem.write      := alu_bfuInst(0).io.mem.write
//     io.mem.writedata  := alu_bfuInst(0).io.mem.writedata
//     io.mem.byteenable := alu_bfuInst(0).io.mem.byteenable
    
//     // input to the bfu
//     alu_bfuInst(0).io.mem.waitrequest    := io.mem.waitrequest
//     alu_bfuInst(0).io.mem.readdatavalid  := io.mem.readdatavalid
//     alu_bfuInst(0).io.mem.readdata       := io.mem.readdata
//     for (i <- 1 until NUM_ALUS) {
//         alu_bfuInst(i).io.mem.waitrequest   := false.B 
//         alu_bfuInst(i).io.mem.readdatavalid := false.B
//         alu_bfuInst(i).io.mem.readdata      := DontCare
//     }

//     val regfile = Module(new RegRead(NUM_THREADS, NUM_RF_RD_PORTS, NUM_RF_WR_PORTS, NUM_REGS, REG_WIDTH, NUM_REGBLOCKS, REG_BLOCK_WIDTH))

//     // define the IO bundle for TreadMem and DestMem
//     class ThreadMemT extends Bundle {
//         val brUcodes   = new BRMicrocodes(NUM_ALUS, NUM_FUS)
//         val rdWrEn     = Vec(NUM_FUS, Bool())
//         val rd         = Vec(NUM_FUS, UInt(NUM_REGS_LG.W))
//         val rd_pos     = Vec(NUM_ALUS, UInt(NUM_DST_POS_LG.W))
//         val rd_mode    = Vec(NUM_ALUS, UInt(NUM_DST_MODES_LG.W))
//     }

//     class DestMemT extends Bundle {
//         val dstPC      = UInt(32.W)
//         val wben       = UInt(NUM_REGBLOCKS.W)
//         val dest       = UInt(REG_WIDTH.W)
//     }

//     // init the memorys and init their IO
//     val threadMem = Module(new ram_simple2port(NUM_THREADS, (new ThreadMemT).getWidth))
//     val destMems = Seq.fill(NUM_WR)(Module(new ram_simple2port(NUM_THREADS, (new DestMemT).getWidth)))
//     threadMem.io.clock := clock 
//     threadMem.io.rden := false.B
//     threadMem.io.rdaddress := DontCare
//     for (destMem <- destMems) {
//         destMem.io.clock := clock
//         destMem.io.wren := false.B
//         destMem.io.rden := false.B
//         destMem.io.wraddress := DontCare
//         destMem.io.rdaddress := DontCare
//         destMem.io.data := DontCare
//     }

    
    

// }


    

## TOP V2

In [25]:
object ThreadStageEnum extends ChiselEnum {  // ZX: those state control should classified into control unit
    val idle   = Value
    val order_fetch = Value
    val fetch  = Value
    val decode = Value
    val read   = Value
    val pre    = Value
    val exec   = Value
    //val post   = Value
    val branch = Value
}

defined object ThreadStageEnum

### Scheduler logic

In [26]:
  // /****************** Start Thread *********************************/
  // /****************** Scheduler logic *********************************/
  // // select idle thread
  // val vThreadEncoder = Module(new Scheduler_order(NUM_THREADS, NUM_RD_BANKS))

  // when (ioUnit.io.new_thread) {  // make ioUnit.io.new_thread as an input 
  //   // spawn new thread
  //   threadStages(ioUnit.io.new_tag) := ThreadStageEnum.order_fetch  // threadStages and threadStates as output 
  //   threadStates(ioUnit.io.new_tag).ip := 0.U((IP_WIDTH+2).W)       // ioUnit.io.new_tag as input, name it as ioUnit_new_tag
  //   vThreadEncoder.io.valid := true.B
  //   vThreadEncoder.io.tag := ioUnit.io.new_tag                    
  //   println("Primate core spawning new thread!")
  // } .otherwise {
  //   vThreadEncoder.io.valid := false.B
  //   vThreadEncoder.io.tag := DontCare
  // }

  
  // // select valid thread
  // val vThread = vThreadEncoder.io.chosen              // vThread as output
  // Range(0, NUM_THREADS, 1).map(i =>
  //   vThreadEncoder.io.order_ready(i) := (threadStages(i) === ThreadStageEnum.order_fetch))  // no warries on ThreadStageEnum, it can be view gloabal
  // Range(0, NUM_THREADS, 1).map(i =>
  //   vThreadEncoder.io.ready(i) := (threadStages(i) === ThreadStageEnum.fetch))

class SchedulerLogic(NUM_THREADS: Int, NUM_RD_BANKS: Int, IP_WIDTH: Int) extends Module {
    val io = IO(new Bundle {
        // Inputs
        val ioUnit_new_thread = Input(Bool())                          // Signal to indicate a new thread spawn
        val ioUnit_new_tag = Input(UInt(log2Up(NUM_THREADS).W))        // Signal for the new thread's tag   // Output(UInt(log2Up(num_threads).W))
        val threadStages = Input(Vec(NUM_THREADS, ThreadStageEnum.Type()))   // To update thread stages
        // Outputs
        val vThread = Output(UInt((log2Up(NUM_THREADS)+1).W))        // Chosen valid thread   //Output(UInt((log2Up(num_threads)+1).W))
    })

    // Scheduler module instantiation
    val vThreadEncoder = Module(new Scheduler_order(NUM_THREADS, NUM_RD_BANKS))
    
    // New thread spawning logic
    when(io.ioUnit_new_thread) {
        // Configure vThreadEncoder for a new thread
        vThreadEncoder.io.valid := true.B
        vThreadEncoder.io.tag := io.ioUnit_new_tag
        println("Primate core spawning new thread!")
    }.otherwise {
        vThreadEncoder.io.valid := false.B
        vThreadEncoder.io.tag := DontCare
    }

    // Select a valid thread
    io.vThread := vThreadEncoder.io.chosen

    Range(0, NUM_THREADS, 1).map(i =>
      vThreadEncoder.io.order_ready(i) := (io.threadStages(i) === ThreadStageEnum.order_fetch))
    Range(0, NUM_THREADS, 1).map(i =>
      vThreadEncoder.io.ready(i) := (io.threadStages(i) === ThreadStageEnum.fetch))
}

defined class SchedulerLogic

### Decode logic

In [27]:
  // /****************** Decode logic *********************************/
  // val decodeThread = RegInit(NONE_SELECTED) //assume this declared outside, make this as the input
  // decodeThread := vThread
  // val decodeThread_d1 = RegNext(decodeThread)  //assume this declared outside, make this as the input

  // val decodeUnit = Module(new Decode(NUM_ALUS, NUM_BFUS, NUM_FUS, SUBINSTR_WIDTH, ALU_SRC_WIDTH, NUM_SRC_POS_LG, NUM_SRC_MODES_LG, NUM_DST_POS_LG, NUM_DST_MODES_LG))  // local
  // val aluMicrocodes_in = Wire(new ALUMicrocodes(NUM_ALUS, ALU_SRC_WIDTH, NUM_SRC_POS_LG, NUM_SRC_MODES_LG)) // local output
  // val bfuMicrocodes_in = Wire(new BFUMicrocodes(NUM_FUS-NUM_ALUS, ALU_SRC_WIDTH))  // local output
  // val rdWrEn_in = Wire(Vec(NUM_FUS, Bool()))   //  local output
  // val bfuValids_in = Wire(Vec(NUM_BFUS, Bool()))    // local output
  // decodeUnit.io.instr := instr            // instr as input

  // val BFU_RSPQ_DEPTH_LG = 3
  // val BFU_RSPQ_DEPTH = 8 // (BFU_RSPQ_DEPTH/2 must be equal or greater than NUM_RF_RD_PORTS+1)
  // val bfuRdReqQ = Module(new Queue(new BFU_regfile_req_t(NUM_THREADS_LG, NUM_REGS_LG), 4))   // local module 
  // val bfuRdRspQ = Module(new Queue(new BFU_regfile_rsp_t(REG_WIDTH), BFU_RSPQ_DEPTH))  // local module
  // val bfuRdRspQ_almFull = Wire(Bool())  // local module
  // val bfuRdRspQ_enq_in = Wire(Bool())  // output
  // val decode_valid = RegInit(false.B)  // local
  

  // bfuRdReqQ.io.deq.ready := false.B
  // bfuRdReqQ.io.enq.valid := ioUnit.io.rd_req_valid // ioUnit.io.rd_req_valid is input name it as ioUnit_rd_req_valid
  // bfuRdReqQ.io.enq.bits := ioUnit.io.rd_req  // ioUnit.io.rd_req is input name it as ioUnit_rd_req
  // ioUnit.io.rd_req_ready := bfuRdReqQ.io.enq.ready // ioUnit.io.rd_req_ready is output, name it as ioUnit_rd_req_ready

  // bfuRdRspQ_almFull := bfuRdRspQ.io.count(2).asBool
  // bfuRdRspQ.io.enq.valid := false.B
  // bfuRdRspQ_enq_in := false.B
  // ioUnit.io.rd_rsp_valid := bfuRdRspQ.io.deq.valid  // ioUnit.io.rd_rsp_valid is output
  // ioUnit.io.rd_rsp := bfuRdRspQ.io.deq.bits   // ioUnit.io.rd_rsp is output
  // bfuRdRspQ.io.deq.ready := ioUnit.io.rd_rsp_ready  // ioUnit.io.rd_rsp_ready is input

  // decode_valid := false.B
  // when (decodeThread =/= NONE_SELECTED) {
  //     ///exclude start
  //   threadStates(decodeThread).bfuValids := decodeUnit.io.bfuValids   // make decodeUnit.io.bfuValids as an output
  //   threadStates(decodeThread).invalid := false.B    // exclude the threadStates in the extract block, I will define them outside. 
  //   threadStates(decodeThread).execValids := VecInit(Seq.fill(NUM_BFUS)(false.B))   // exclude the threadStates in the extract block, I will define them outside. 
  //     ///exclude end
  //   decode_valid := true.B   // something local

  //   println("Got a valid decode thread!")

  //   aluMicrocodes_in := decodeUnit.io.aluUcodes // local
  //   bfuMicrocodes_in := decodeUnit.io.bfuUcodes
  //   rdWrEn_in := decodeUnit.io.rdWrEn
  //   bfuValids_in := decodeUnit.io.bfuValids

  //   val threadMem_in = Wire(new ThreadMemT)  // ZX: what is threadMem_in connected to 
  //   threadMem_in.brUcodes  := decodeUnit.io.brUcodes
  //   threadMem_in.rdWrEn    := decodeUnit.io.rdWrEn
  //   threadMem_in.rd        := decodeUnit.io.rd
  //   threadMem_in.rd_pos    := decodeUnit.io.rd_pos
  //   threadMem_in.rd_mode   := decodeUnit.io.rd_mode
      
  //   threadMem.io.wraddress := decodeThread   // threadMem.io.wraddress is output
  //   threadMem.io.wren      := true.B     // threadMem.io.wren is output
  //   threadMem.io.data      := threadMem_in.asUInt  // threadMem.io.data is output
      
  //   regfile.io.thread_rd   := decodeThread  // regfile.io.thread_rd is output
  //   regfile.io.rdEn        := true.B  // regfile.io.rdEn is output
      
  //   for (i <- 0 until NUM_RF_RD_PORTS) {
  //     regfile.io.rdAddr1(i) := decodeUnit.io.rs1(i)  // regfile.io.rdAddr1 output
  //     regfile.io.rdAddr2(i) := decodeUnit.io.rs2(i)  // regfile.io.rdAddr2 output
  //   }
  //   /// execlude start
  //   threadStages(decodeThread) := ThreadStageEnum.read
  //   /// execlude end
  // }
  // .otherwise {
  //   val initVec = RegInit(VecInit(Seq.fill(NUM_FUS)(false.B)))     // local reg
  //   val initVecALU = RegInit(VecInit(Seq.fill(NUM_ALUS)(false.B))) // local reg
  //   val initVecBFU = RegInit(VecInit(Seq.fill(NUM_BFUS)(false.B))) // local reg
  //   aluMicrocodes_in.bfu_valid := initVecALU
  //   aluMicrocodes_in.opcode := DontCare
  //   aluMicrocodes_in.rs1_pos := DontCare
  //   aluMicrocodes_in.rs1_mode := DontCare
  //   aluMicrocodes_in.rs2_pos := DontCare
  //   aluMicrocodes_in.rs2_mode := DontCare
  //   aluMicrocodes_in.rd     := DontCare
  //   aluMicrocodes_in.addEn  := initVecALU 
  //   aluMicrocodes_in.subEn  := initVecALU
  //   aluMicrocodes_in.sltEn  := initVecALU
  //   aluMicrocodes_in.sltuEn := initVecALU
  //   aluMicrocodes_in.andEn  := initVecALU
  //   aluMicrocodes_in.orEn   := initVecALU
  //   aluMicrocodes_in.xorEn  := initVecALU
  //   aluMicrocodes_in.sllEn  := initVecALU
  //   aluMicrocodes_in.srEn   := initVecALU
  //   aluMicrocodes_in.srMode := initVecALU
  //   aluMicrocodes_in.luiEn  := initVecALU
  //   aluMicrocodes_in.catEn  := initVecALU
  //   aluMicrocodes_in.immSel := initVecALU
  //   aluMicrocodes_in.imm    := DontCare

  //   bfuMicrocodes_in := DontCare

  //   rdWrEn_in := initVec
  //   bfuValids_in := initVecBFU

  //   threadMem.io.wraddress := DontCare
  //   threadMem.io.wren      := false.B
  //   threadMem.io.data      := DontCare
  //   threadStates(decodeThread).bfuValids := DontCare
  //   threadStates(decodeThread).execValids := DontCare

  //   regfile.io.thread_rd := DontCare
  //   regfile.io.rdEn := false.B
  //   regfile.io.rdAddr1 := DontCare
  //   regfile.io.rdAddr2 := DontCare

  //   val bfuRdTag = bfuRdReqQ.io.deq.bits.tag
  //   regfile.io.thread_rd := bfuRdReqQ.io.deq.bits.tag
  //   regfile.io.rdAddr1(0) := bfuRdReqQ.io.deq.bits.rdAddr1
  //   regfile.io.rdAddr2(0) := bfuRdReqQ.io.deq.bits.rdAddr2
  //   when (bfuRdReqQ.io.deq.valid && (!bfuRdRspQ_almFull) && ((!decode_valid) || 
  //     (bfuRdTag(log2Up(NUM_RF_RD_PORTS)-1, 0) =/= decodeThread_d1(log2Up(NUM_RF_RD_PORTS)-1, 0)))) {
  //     bfuRdRspQ_enq_in := true.B
  //     bfuRdReqQ.io.deq.ready := true.B
  //   }
  // }


class DecodeLogic(
    NUM_ALUS: Int,
    NUM_BFUS: Int,
    NUM_FUS: Int,
    INSTR_WIDTH: Int,
    SUBINSTR_WIDTH: Int,
    ALU_SRC_WIDTH: Int,
    NUM_SRC_POS_LG: Int,
    NUM_SRC_MODES_LG: Int,
    NUM_DST_POS_LG: Int,
    NUM_DST_MODES_LG: Int,
    NUM_THREADS: Int,
    NUM_THREADS_LG: Int,
    NUM_REGS_LG: Int,
    REG_WIDTH: Int,
    THREAD_MEM_WIDTH: Int,
    NUM_RF_RD_PORTS: Int
) extends Module {
  val io = IO(new Bundle {
    // Inputs
    val instr = Input(UInt(INSTR_WIDTH.W)) // Instruction
    val decodeThread = Input(UInt(log2Up(NUM_THREADS + 1).W))
    val decodeThread_d1 = Input(UInt((log2Up(NUM_THREADS+1)).W))
    val ioUnit_rd_req_valid = Input(Bool())
    val ioUnit_rd_req = Input(new BFU_regfile_req_t(log2Up(NUM_THREADS), NUM_REGS_LG))   // Output(new BFU_regfile_req_t(log2Up(num_threads), num_regs_lg))
    val ioUnit_rd_rsp_ready = Input(Bool())
    val bfuRdRspQ_enq_valid = Input(Bool())
    val bfuRdRspQ_enq_bits = Input(new BFU_regfile_rsp_t(REG_WIDTH))

    // Outputs
    val decodeUnit_bfuValids = Output(Vec(NUM_BFUS, Bool()))
    val aluMicrocodes_in = Output(new ALUMicrocodes(NUM_ALUS, ALU_SRC_WIDTH, NUM_SRC_POS_LG, NUM_SRC_MODES_LG))
    val bfuMicrocodes_in = Output(new BFUMicrocodes(NUM_FUS-NUM_ALUS, ALU_SRC_WIDTH))
    val rdWrEn_in = Output(Vec(NUM_FUS, Bool()))
    val bfuValids_in = Output(Vec(NUM_BFUS, Bool())) 
    val bfuRdRspQ_enq_in = Output(Bool())
    val ioUnit_rd_req_ready = Output(Bool())
    val ioUnit_rd_rsp_valid = Output(Bool())
    val ioUnit_rd_rsp = Output(new BFU_regfile_rsp_t(REG_WIDTH))  // (new BFU_regfile_rsp_t(REG_WIDTH))
    val threadMem_wraddress = Output(UInt(log2Up(NUM_THREADS).W)) // ram_simple2port: Input(UInt(log2Up(NUM_THREADS).W))
    val threadMem_wren = Output(Bool())
    val threadMem_data = Output(UInt(THREAD_MEM_WIDTH.W))  // ram_simple2port: Input(UInt(THREAD_MEM_WIDTH.W))
    val regfile_thread_rd = Output(UInt(NUM_THREADS_LG.W))   // ==Input(UInt(log2Up(NUM_THREADS).W))
    val regfile_rdEn = Output(Bool())
    val regfile_rdAddr1 = Output(Vec(NUM_RF_RD_PORTS, UInt(NUM_REGS_LG.W))) // ==Input(Vec(NUM_RF_RD_PORTS, UInt(log2Up(NUM_REGS).W)))
    val regfile_rdAddr2 = Output(Vec(NUM_RF_RD_PORTS, UInt(NUM_REGS_LG.W))) 
  })

  val NONE_SELECTED = (NUM_THREADS).U((log2Up(NUM_THREADS+1)).W)
  val BFU_RSPQ_DEPTH_LG = 3
  val BFU_RSPQ_DEPTH = 8 // (BFU_RSPQ_DEPTH/2 must be equal or greater than NUM_RF_RD_PORTS+1)
    
  val decodeUnit = Module(new Decode(NUM_ALUS, NUM_BFUS, NUM_FUS, SUBINSTR_WIDTH, ALU_SRC_WIDTH, NUM_SRC_POS_LG, NUM_SRC_MODES_LG, NUM_DST_POS_LG, NUM_DST_MODES_LG))
  val bfuRdReqQ = Module(new Queue(new BFU_regfile_req_t(NUM_THREADS_LG, NUM_REGS_LG), 4))   // local module 
  val bfuRdRspQ = Module(new Queue(new BFU_regfile_rsp_t(REG_WIDTH), BFU_RSPQ_DEPTH))  // local module
  val bfuRdRspQ_almFull = Wire(Bool())  // local module
  val decode_valid = RegInit(false.B)

  bfuRdRspQ.io.enq.valid := io.bfuRdRspQ_enq_valid  
  bfuRdRspQ.io.enq.bits := io.bfuRdRspQ_enq_bits

  // Connections
  decodeUnit.io.instr := io.instr

  bfuRdReqQ.io.deq.ready := false.B
  bfuRdReqQ.io.enq.valid := io.ioUnit_rd_req_valid
  bfuRdReqQ.io.enq.bits := io.ioUnit_rd_req
  io.ioUnit_rd_req_ready := bfuRdReqQ.io.enq.ready

  bfuRdRspQ_almFull := bfuRdRspQ.io.count(2).asBool
  bfuRdRspQ.io.enq.valid := false.B
  io.bfuRdRspQ_enq_in := false.B
  io.ioUnit_rd_rsp_valid := bfuRdRspQ.io.deq.valid
  io.ioUnit_rd_rsp := bfuRdRspQ.io.deq.bits
  bfuRdRspQ.io.deq.ready := io.ioUnit_rd_rsp_ready

  decode_valid := false.B
  io.decodeUnit_bfuValids := decodeUnit.io.bfuValids 

  when(io.decodeThread =/= NONE_SELECTED) {
    decode_valid := true.B
    println("Got a valid decode thread!")
    io.aluMicrocodes_in := decodeUnit.io.aluUcodes
    io.bfuMicrocodes_in := decodeUnit.io.bfuUcodes
    io.rdWrEn_in := decodeUnit.io.rdWrEn
    io.bfuValids_in := decodeUnit.io.bfuValids

    class ThreadMemT extends Bundle {
        val brUcodes   = new BRMicrocodes(NUM_ALUS, NUM_FUS)
        val rdWrEn     = Vec(NUM_FUS, Bool())
        val rd         = Vec(NUM_FUS, UInt(NUM_REGS_LG.W))
        val rd_pos     = Vec(NUM_ALUS, UInt(NUM_DST_POS_LG.W))
        val rd_mode    = Vec(NUM_ALUS, UInt(NUM_DST_MODES_LG.W))
    }

    val threadMem_in = Wire(new ThreadMemT)  
    threadMem_in.brUcodes  := decodeUnit.io.brUcodes
    threadMem_in.rdWrEn    := decodeUnit.io.rdWrEn
    threadMem_in.rd        := decodeUnit.io.rd
    threadMem_in.rd_pos    := decodeUnit.io.rd_pos
    threadMem_in.rd_mode   := decodeUnit.io.rd_mode
      
    io.threadMem_wraddress := io.decodeThread
    io.threadMem_wren := true.B
    io.threadMem_data := threadMem_in.asUInt

    io.regfile_thread_rd := io.decodeThread
    io.regfile_rdEn := true.B
      
    for (i <- 0 until NUM_RF_RD_PORTS) {
      io.regfile_rdAddr1(i) := decodeUnit.io.rs1(i)
      io.regfile_rdAddr2(i) := decodeUnit.io.rs2(i)
    }
  }.otherwise {
    val initVec = RegInit(VecInit(Seq.fill(NUM_FUS)(false.B)))     // local reg
    val initVecALU = RegInit(VecInit(Seq.fill(NUM_ALUS)(false.B))) // local reg
    val initVecBFU = RegInit(VecInit(Seq.fill(NUM_BFUS)(false.B))) // local reg
    io.aluMicrocodes_in.bfu_valid := initVecALU
    io.aluMicrocodes_in.opcode := DontCare
    io.aluMicrocodes_in.rs1_pos := DontCare
    io.aluMicrocodes_in.rs1_mode := DontCare
    io.aluMicrocodes_in.rs2_pos := DontCare
    io.aluMicrocodes_in.rs2_mode := DontCare
    io.aluMicrocodes_in.rd     := DontCare
    io.aluMicrocodes_in.addEn  := initVecALU 
    io.aluMicrocodes_in.subEn  := initVecALU
    io.aluMicrocodes_in.sltEn  := initVecALU
    io.aluMicrocodes_in.sltuEn := initVecALU
    io.aluMicrocodes_in.andEn  := initVecALU
    io.aluMicrocodes_in.orEn   := initVecALU
    io.aluMicrocodes_in.xorEn  := initVecALU
    io.aluMicrocodes_in.sllEn  := initVecALU
    io.aluMicrocodes_in.srEn   := initVecALU
    io.aluMicrocodes_in.srMode := initVecALU
    io.aluMicrocodes_in.luiEn  := initVecALU
    io.aluMicrocodes_in.catEn  := initVecALU
    io.aluMicrocodes_in.immSel := initVecALU
    io.aluMicrocodes_in.imm    := DontCare

    io.bfuMicrocodes_in := DontCare
    io.rdWrEn_in := initVec
    io.bfuValids_in := initVecBFU

    io.threadMem_wraddress := DontCare
    io.threadMem_wren := false.B
    io.threadMem_data := DontCare

    io.regfile_thread_rd := DontCare
    io.regfile_rdEn := false.B
    io.regfile_rdAddr1 := DontCare
    io.regfile_rdAddr2 := DontCare

    val bfuRdTag = bfuRdReqQ.io.deq.bits.tag
    io.regfile_thread_rd := bfuRdReqQ.io.deq.bits.tag
    io.regfile_rdAddr1(0) := bfuRdReqQ.io.deq.bits.rdAddr1
    io.regfile_rdAddr2(0) := bfuRdReqQ.io.deq.bits.rdAddr2

    when (bfuRdReqQ.io.deq.valid && (!bfuRdRspQ_almFull) && ((!decode_valid) || 
        (bfuRdTag(log2Up(NUM_RF_RD_PORTS)-1, 0) =/= io.decodeThread_d1(log2Up(NUM_RF_RD_PORTS)-1, 0)))) {
        io.bfuRdRspQ_enq_in := true.B
        bfuRdReqQ.io.deq.ready := true.B
    }
  }

}


defined class DecodeLogic

### gatherLogic

In [28]:

  // /****************** Gather Stage 0-2 *************************/
  // val gatherThread = readThread_vec(ALU_DELAY)  // gatherThread as input
  // val aluMicrocodes_out = aluMicrocodes_vec(0)  // aluMicrocodes_out is the input because there is no write to it, aluMicrocodes_vec(0) not in the module (exluded) (the bitwidth can be tell by val bfuMicrocodes_vec = Reg(Vec(REG_DELAY, new BFUMicrocodes(NUM_FUS-NUM_ALUS, ALU_SRC_WIDTH))))
  // val bfuMicrocodes_out = bfuMicrocodes_vec(0)  // aluMicrocodes_out is the input, 
  // val bfuValids_out = bfuValids_vec(0)          // bfuValids_out is the input, bfuValids_vec(0) is input  (val bfuValids_vec = RegInit(VecInit(Seq.fill(REG_DELAY)(VecInit(Seq.fill(NUM_BFUS)(false.B))))))

  // ///// exclude start
  // val gather_srcA = Seq.fill(NUM_ALUS)(Module(new Gather(REG_WIDTH, NUM_SRC_POS, src_pos, MAX_FIELD_WIDTH, NUM_SRC_MODES, src_mode)))
  // val gather_srcB = Seq.fill(NUM_ALUS)(Module(new Gather(REG_WIDTH, NUM_SRC_POS, src_pos, MAX_FIELD_WIDTH, NUM_SRC_MODES, src_mode)))  
  // for (i <- 0 until NUM_ALUS) {
  //   gather_srcA(i).io.din   := srcA(i)    // srcA(i) is the input (val srcA = Wire(Vec(NUM_FUS, UInt(REG_WIDTH.W))))
  //   gather_srcA(i).io.shift := aluMicrocodes_out.rs1_pos(i)
  //   gather_srcA(i).io.mode  := aluMicrocodes_out.rs1_mode(i)
  //   gather_srcB(i).io.din   := srcB(i)
  //   gather_srcB(i).io.shift := aluMicrocodes_out.rs2_pos(i)
  //   gather_srcB(i).io.mode  := aluMicrocodes_out.rs2_mode(i)
  // }
  // ///// exclude end

  // // BFU FIFOs
  // val execBundle_io = new Bundle {
  //   val tag = UInt(NUM_THREADS_LG.W)
  //   val opcode = UInt(NUM_FUOPS_LG.W)
  //   val rd = UInt(NUM_REGS_LG.W)
  //   val imm = UInt(12.W)
  //   val bits = UInt(REG_WIDTH.W)
  // }

  // // exclude start
  // val fuFifos_iou = Module(new Queue(execBundle_io, NUM_THREADS))
  // // exclude end
  // fuFifos_iou.io.enq.valid := false.B
  // fuFifos_iou.io.enq.bits := DontCare
  

  // if (NUM_BFUS > NUM_ALUS) {
  //   when (gatherThread =/= NONE_SELECTED) {
  //     when (bfuValids_out(NUM_BFUS-1) === true.B) {
  //       fuFifos_iou.io.enq.valid := true.B  // make an output boundle name as fuFifos_enq.valid
  //       fuFifos_iou.io.enq.bits.tag := gatherThread  // output fuFifos_enq.bit_tag
  //       fuFifos_iou.io.enq.bits.opcode := bfuMicrocodes_out.opcode(NUM_BFUS-1-NUM_ALUS) // output
  //       fuFifos_iou.io.enq.bits.rd := bfuMicrocodes_out.rd(NUM_BFUS-1-NUM_ALUS) // output
  //       fuFifos_iou.io.enq.bits.imm := bfuMicrocodes_out.bimm(NUM_BFUS-1-NUM_ALUS) // output
  //       fuFifos_iou.io.enq.bits.bits := srcA(NUM_BFUS-1) // output
  //     }
  //   }
  // } else {
  //   when (gatherThread =/= NONE_SELECTED) {
  //     when (aluMicrocodes_out.bfu_valid(NUM_BFUS-1) === true.B) {
  //       fuFifos_iou.io.enq.valid := true.B
  //       fuFifos_iou.io.enq.bits.tag := gatherThread
  //       fuFifos_iou.io.enq.bits.opcode := aluMicrocodes_out.opcode(NUM_BFUS-1)
  //       fuFifos_iou.io.enq.bits.rd := aluMicrocodes_out.rd(NUM_BFUS-1)
  //       fuFifos_iou.io.enq.bits.imm := aluMicrocodes_out.imm(NUM_BFUS-1).asUInt
  //       fuFifos_iou.io.enq.bits.bits := srcA(NUM_BFUS-1)
  //     }
  //   }
  // }


class gatherLogic(
    NUM_THREADS: Int,
    NUM_BFUS: Int,
    NUM_ALUS: Int,
    NUM_FUS: Int,
    REG_WIDTH: Int,
    NUM_THREADS_LG: Int,
    NUM_REGS_LG: Int,
    NUM_FUOPS_LG: Int,
    ALU_SRC_WIDTH: Int,
    NUM_SRC_POS_LG: Int, 
    NUM_SRC_MODES_LG: Int, 
    NONE_SELECTED: UInt  // !type check please!
   
) extends Module {
  val io = IO(new Bundle {
    // Inputs
    val gatherThread = Input(UInt(log2Up(NUM_THREADS + 1).W)) // Input gather thread
    val aluMicrocodes_out = Input(new ALUMicrocodes(NUM_ALUS, ALU_SRC_WIDTH, NUM_SRC_POS_LG, NUM_SRC_MODES_LG)) 
    val bfuMicrocodes_out = Input(new BFUMicrocodes(NUM_FUS-NUM_ALUS, ALU_SRC_WIDTH))
    val bfuValids_out = Input(Vec(NUM_BFUS, Bool())) // Validity vector for BFU
    val srcA = Input(Vec(NUM_FUS, UInt(REG_WIDTH.W)))
    // Outputs
    val fuFifos_enq_valid = Output(Bool()) // FIFO enqueue valid signal
    val fuFifos_enq_bits_tag = Output(UInt(log2Up(NUM_THREADS + 1).W)) // Enqueue tag
    val fuFifos_enq_bits_opcode = Output(UInt(NUM_FUOPS_LG.W)) // Enqueue opcode              
    val fuFifos_enq_bits_rd = Output(UInt(NUM_REGS_LG.W)) // Enqueue destination register    
    val fuFifos_enq_bits_imm = Output(UInt(12.W)) // Enqueue immediate value              
    val fuFifos_enq_bits_bits = Output(UInt(REG_WIDTH.W)) // Enqueue bits
  })

  // Default values for outputs
  io.fuFifos_enq_valid := false.B
  io.fuFifos_enq_bits_tag := DontCare
  io.fuFifos_enq_bits_opcode := DontCare
  io.fuFifos_enq_bits_rd := DontCare
  io.fuFifos_enq_bits_imm := DontCare
  io.fuFifos_enq_bits_bits := DontCare

  if (NUM_BFUS > NUM_ALUS) {
    when (io.gatherThread =/= NONE_SELECTED) {
      when (io.bfuValids_out(NUM_BFUS - 1) === true.B) {
        io.fuFifos_enq_valid := true.B  
        io.fuFifos_enq_bits_tag := io.gatherThread  
        io.fuFifos_enq_bits_opcode := io.bfuMicrocodes_out.opcode(NUM_BFUS - 1 - NUM_ALUS)
        io.fuFifos_enq_bits_rd := io.bfuMicrocodes_out.rd(NUM_BFUS - 1 - NUM_ALUS)
        io.fuFifos_enq_bits_imm := io.bfuMicrocodes_out.bimm(NUM_BFUS - 1 - NUM_ALUS)
        io.fuFifos_enq_bits_bits := io.srcA(NUM_BFUS - 1)
      }
    }
  } else {
    when (io.gatherThread =/= NONE_SELECTED) {
      when (io.aluMicrocodes_out.bfu_valid(NUM_BFUS - 1) === true.B) {
        io.fuFifos_enq_valid := true.B
        io.fuFifos_enq_bits_tag := io.gatherThread
        io.fuFifos_enq_bits_opcode := io.aluMicrocodes_out.opcode(NUM_BFUS - 1)
        io.fuFifos_enq_bits_rd := io.aluMicrocodes_out.rd(NUM_BFUS - 1)
        io.fuFifos_enq_bits_imm := io.aluMicrocodes_out.imm(NUM_BFUS - 1).asUInt
        io.fuFifos_enq_bits_bits := io.srcA(NUM_BFUS - 1)
      }
    }
  }
    
}

defined class gatherLogic

### FinishExecutionLogic

In [29]:
  // /****************** Finish execution *********************************/
  // val fThreadEncoder = Module(new Scheduler(NUM_THREADS, NUM_WR_BANKS))  // local module
  // val fThread = fThreadEncoder.io.chosen // output
  // val execDone = Wire(Vec(NUM_THREADS, Bool())) // local
  // Range(0, NUM_THREADS, 1).foreach(i =>
  //   execDone(i) := (threadStates(i).execValids.asUInt | (~threadStates(i).bfuValids.asUInt)).andR // pass all threadStates(i) as inputs
  // )

  // Range(0, NUM_THREADS, 1).map(i =>
  //   fThreadEncoder.io.valid(i) := (execDone(i) === true.B && threadStages(i) === ThreadStageEnum.exec)) /// all threadStages(i) as inputs

  // when (fThread =/= NONE_SELECTED) {
  //   threadStages(fThread) := ThreadStageEnum.branch
  //   for (destMem <- destMems) {  // where val destMems = Seq.fill(NUM_WR)(Module(new ram_simple2port(NUM_THREADS, (new DestMemT).getWidth)))
  //     destMem.io.rden := true.B   // make destMem.io.rden as output as destMem.rden
  //     destMem.io.rdaddress := fThread // make  destMem.io.rdaddress as output
  //   }
  //   threadMem.io.rden := true.B // output
  //   threadMem.io.rdaddress := fThread  //output
  // }


class FinishExecutionLogic(
    NUM_THREADS: Int,
    NUM_WR: Int,
    NUM_WR_BANKS: Int,
    NUM_BFUS: Int,
    NONE_SELECTED: UInt
) extends Module {
  val io = IO(new Bundle {
    // Inputs
    val threadStates_bfuValids = Input(Vec(NUM_THREADS, Vec(NUM_BFUS, Bool())))
    val threadStates_execValids = Input(Vec(NUM_THREADS, Vec(NUM_BFUS, Bool())))
    val threadStages = Input(Vec(NUM_THREADS, ThreadStageEnum.Type())) // Thread stages
    // Outputs
    val fThread = Output(UInt(log2Up(NUM_THREADS + 1).W)) // Chosen thread 
    val threadMem_rden = Output(Bool()) // Thread memory read enable
    val threadMem_rdaddress = Output(UInt(log2Up(NUM_THREADS + 1).W)) // Thread memory read address
    val destMems_rden = Output(Vec(NUM_WR, Bool())) // Destination memory read enable
    val destMems_rdaddress = Output(Vec(NUM_WR, UInt(log2Up(NUM_THREADS + 1).W))) // Destination memory read address
  })

  // Scheduler instantiation
  val fThreadEncoder = Module(new Scheduler(NUM_THREADS, NUM_WR_BANKS))
  io.fThread := fThreadEncoder.io.chosen

  // Compute execDone for each thread
  val execDone = Wire(Vec(NUM_THREADS, Bool()))
  for (i <- 0 until NUM_THREADS) {
    execDone(i) := (io.threadStates_execValids(i).asUInt | (~io.threadStates_bfuValids(i).asUInt)).andR
  }

  // Set valid signals for the scheduler
  for (i <- 0 until NUM_THREADS) {
    fThreadEncoder.io.valid(i) := (execDone(i) === true.B && io.threadStages(i) === ThreadStageEnum.exec)
  }

  // Default outputs
  io.threadMem_rden := false.B
  io.threadMem_rdaddress := DontCare
  io.destMems_rden := VecInit(Seq.fill(NUM_WR)(false.B))
  io.destMems_rdaddress := VecInit(Seq.fill(NUM_WR)(DontCare))

  // If a thread is selected, update outputs
  when(io.fThread =/= NONE_SELECTED) {
    io.threadMem_rden := true.B
    io.threadMem_rdaddress := io.fThread
    for (i <- 0 until NUM_WR) {
      io.destMems_rden(i) := true.B
      io.destMems_rdaddress(i) := io.fThread
    }
  }
}


defined class FinishExecutionLogic

### lastpart

In [30]:

  //   /****************** Finish execution *********************************/
  //   val finishExecutionLogicBlock = Module(new FinishExecutionLogic(NUM_THREADS, NUM_WR, NUM_WR_BANKS, NUM_BFUS, NONE_SELECTED))

  //   // Provide inputs to the extracted module
  //   finishExecutionLogicBlock.io.threadStates_execValids := threadStates.map(_.execValids)
  //   finishExecutionLogicBlock.io.threadStates_bfuValids := threadStates.map(_.bfuValids)
  //   finishExecutionLogicBlock.io.threadStages := threadStages
    
  //   // Extract outputs from the module
  //   val fThread = finishExecutionLogicBlock.io.fThread
  //   threadMem.io.rden := finishExecutionLogicBlock.io.threadMem_rden
  //   threadMem.io.rdaddress := finishExecutionLogicBlock.io.threadMem_rdaddress
    
  //   for (i <- 0 until NUM_WR) {
  //       destMems(i).io.rden := finishExecutionLogicBlock.io.destMems_rden(i)
  //       destMems(i).io.rdaddress := finishExecutionLogicBlock.io.destMems_rdaddress(i)
  //   }

  //   when (fThread =/= NONE_SELECTED) {
  //       threadStages(fThread) := ThreadStageEnum.branch
  //   }


  // /****************** Register write & branch *********************************/
  // val WB_DELAY = NUM_RF_WR_PORTS.max(3)
  // val branchThread_out = RegInit(NONE_SELECTED)
  // val branchThread_vec = RegInit(VecInit(Seq.fill(WB_DELAY)(NONE_SELECTED)))
  // branchThread_out := branchThread_vec(0)
  // val branchU = Module(new BranchUnit)  // local module, define it in the submodule

  // branchThread_vec(WB_DELAY-1) := fThread
  // for (i <- 0 until WB_DELAY-1) {
  //   branchThread_vec(i) := branchThread_vec(i+1)   // Ater update the value make branchThread_vec as an input
  // }
    
  // //// excluded and remain on the top level, start (this part is assign values to the input)
  // val threadMem_out = Wire(new ThreadMemT)  // defined at the top level, but also a input
  // val destMems_out = Wire(Vec(NUM_WR, (new DestMemT))) // execluded from module, live at top level
  // val destWbens_wb = RegInit(VecInit(Seq.fill(NUM_WR)(0.U(NUM_REGBLOCKS.W)))) // input
  // val dests_wb = Wire(Vec(NUM_WR, UInt(REG_WIDTH.W)))  // input
  // val dstPCs = Wire(Vec(NUM_FUS+3*NUM_ALUS, UInt(32.W)))  // input

  
  // dstPCs := DontCare
  // if (NUM_ALUS < NUM_FUS) {
  //   // The last unit is the IO unit
  //   for (i <- 0 until NUM_FUS-1) {
  //     destMems_out(i) := destMems(i).io.q.asTypeOf(new DestMemT) 
  //     dests_wb(i) := destMems_out(i).dest
  //     destWbens_wb(i) := destMems_out(i).wben
  //   }
  //   for (i <- 0 until NUM_ALUS) {
  //     dstPCs(4*i+2) := destMems_out(i).dstPC
  //   }
  //   for (i <- 0 until NUM_FUS-NUM_ALUS-1) {
  //     dstPCs(4*NUM_ALUS+i) := destMems_out(NUM_ALUS+i).dstPC
  //   }
  //   dstPCs(NUM_FUS+3*NUM_ALUS-1) := threadStates(branchThread_vec(WB_DELAY-2)).io_dstPC
  // } else {
  //   // The IO unit is combined with an ALU unit
  //   for (i <- 0 until NUM_FUS) {
  //     destMems_out(i) := destMems(i).io.q.asTypeOf(new DestMemT)
  //     dests_wb(i) := destMems_out(i).dest
  //     destWbens_wb(i) := destMems_out(i).wben
  //   }
  //   for (i <- 0 until NUM_ALUS) {
  //     dstPCs(4*i+2) := destMems_out(i).dstPC
  //   }
  //   dstPCs(4*(NUM_BFUS-1)+2) := threadStates(branchThread_vec(WB_DELAY-2)).io_dstPC
  // }
    
  // threadMem_out := threadMem.io.q.asTypeOf(chiselTypeOf(threadMem_out)) 

  // val pc = Reg(UInt((IP_WIDTH+2).W))  // pc as an input
  // val threadValid = RegInit(false.B)  // tthreadValid as input

  // when (branchThread_vec(WB_DELAY-1) =/= NONE_SELECTED) {
  //   threadValid := true.B
  //   pc := threadStates(branchThread_vec(WB_DELAY-1)).ip
  // } .otherwise {
  //   threadValid := false.B
  //   pc := DontCare
  // }
  // ///// execluded end


  // ////extract as module, make sure include val branchU = Module(new BranchUnit)

  // val dests_wb_s1 = Reg(Vec(NUM_WR, UInt(REG_WIDTH.W)))   // dests_wb_s1 at local module
  // val rdWrEn_wb = RegInit(VecInit(Seq.fill(NUM_FUS)(false.B)))  // local
  // val rd_wb = Reg(Vec(NUM_FUS, UInt(NUM_REGS_LG.W)))  // local
  // val brUcodes_wb = Wire(new BRMicrocodes(NUM_ALUS, NUM_FUS))  // local
    
  // brUcodes_wb := threadMem_out.brUcodes   // local assign for brUcodes_wb  threadMem_out is input
  // rdWrEn_wb := threadMem_out.rdWrEn  // local assign for rdWrEn_wb
  // rd_wb := threadMem_out.rd  // local assign for rd_wb
  // dests_wb_s1 := dests_wb  // dests_wb_s1 at local module

    
  // val thread_finish = Wire(Bool())
  // val thread_new_pc = Wire(UInt((IP_WIDTH+2).W))
  // val threadValid_s1 = RegNext(threadValid)
  // val threadValid_s2 = RegNext(threadValid_s1)

  // val caseFU = (0 until (NUM_FUS+3*NUM_ALUS)).map(i => (i.U -> dstPCs(i)))  // dstPCs is input
  // val branchU_rs1 = Wire(UInt(32.W))
  // val branchU_rs2 = Wire(UInt(32.W))
  // branchU_rs1 := MuxLookup(brUcodes_wb.rs1, DontCare, caseFU)  // brUcodes_wb is input
  // branchU_rs2 := MuxLookup(brUcodes_wb.rs2, DontCare, caseFU)
  // branchU.io.brValid := threadValid && brUcodes_wb.brValid  // threadValid is input
  // branchU.io.rs1 := branchU_rs1.asSInt
  // branchU.io.rs2 := branchU_rs2.asSInt
  // branchU.io.brMode := brUcodes_wb.brMode 
  // branchU.io.pc := pc.asSInt  //  pc is input
  // branchU.io.pcOffset := brUcodes_wb.pcOffset  

  // // scatter
  // val scatter = Seq.fill(NUM_ALUS)(Module(new Scatter(REG_WIDTH, NUM_SRC_POS_LG, NUM_SRC_MODES_LG, NUM_REGBLOCKS, NUM_DST_POS, dst_encode, dst_pos, NUM_WB_ENS, dst_en_encode, wbens)))
  // for (i <- 0 until NUM_ALUS) {
  //   scatter(i).io.din := dests_wb(i)  // dests_wb is input
  //   scatter(i).io.shift := threadMem_out.rd_pos(i)  // threadMem_out is input
  //   scatter(i).io.mode := threadMem_out.rd_mode(i)
  // }

  // when (threadValid_s1) {
  //   // writeback
  //   regfile.io.wrEn := true.B  // regfile.io.wrEn as output, make following regfile.io. as output as regfile_
  //   regfile.io.thread_wr := branchThread_vec(WB_DELAY-3)
  //   for (i <- 0 until NUM_RF_WR_PORTS) {
  //     if (i*2 < NUM_ALUS) {
  //       regfile.io.wrEn1(i) := rdWrEn_wb(i*2)
  //       regfile.io.wrAddr1(i) := rd_wb(i*2)
  //       regfile.io.wrBen1(i) := scatter(i*2).io.wren
  //       regfile.io.wrData1(i) := scatter(i*2).io.dout
  //     } else {
  //       regfile.io.wrEn1(i) := rdWrEn_wb(i*2)
  //       regfile.io.wrAddr1(i) := rd_wb(i*2)
  //       regfile.io.wrBen1(i) := destWbens_wb(i*2)  // destWbens_wb is input
  //       regfile.io.wrData1(i) := dests_wb_s1(i*2)  // dests_wb_s1 is local
  //     }
  //     if (i*2+1 < NUM_ALUS) {
  //       regfile.io.wrEn2(i) := rdWrEn_wb(i*2+1)
  //       regfile.io.wrAddr2(i) := rd_wb(i*2+1)
  //       regfile.io.wrBen2(i) := scatter(i*2+1).io.wren
  //       regfile.io.wrData2(i) := scatter(i*2+1).io.dout
  //     } else if (i*2+1 < NUM_WR) {
  //       regfile.io.wrEn2(i) := rdWrEn_wb(i*2+1)
  //       regfile.io.wrAddr2(i) := rd_wb(i*2+1)
  //       regfile.io.wrBen2(i) := destWbens_wb(i*2+1)
  //       regfile.io.wrData2(i) := dests_wb_s1(i*2+1)
  //     } else {
  //       regfile.io.wrEn2(i) := false.B
  //       regfile.io.wrAddr2(i) := DontCare
  //       regfile.io.wrBen2(i) := DontCare
  //       regfile.io.wrData2(i) := DontCare
  //     }
  //   }
  // } .otherwise {
  //   regfile.io.wrEn := false.B
  //   regfile.io.thread_wr := ioUnit.io.w_tag   // make ioUnit.io. as input as ioUnit_
  //   regfile.io.wrEn1(0) := ioUnit.io.w_wen(0)
  //   regfile.io.wrAddr1(0) := ioUnit.io.w_addr(0)
  //   regfile.io.wrBen1(0) := Fill(NUM_REGBLOCKS, 1.U)
  //   regfile.io.wrData1(0) := ioUnit.io.w_data(0)(REG_WIDTH-1, 0)
  //   regfile.io.wrEn2(0) := ioUnit.io.w_wen(1)
  //   regfile.io.wrAddr2(0) := ioUnit.io.w_addr(1)
  //   regfile.io.wrBen2(0) := Fill(NUM_REGBLOCKS, 1.U)
  //   regfile.io.wrData2(0) := ioUnit.io.w_data(1)(REG_WIDTH-1, 0)
  //   for (i <- 1 until NUM_RF_WR_PORTS) {
  //     regfile.io.wrEn1(i) := false.B
  //     regfile.io.wrAddr1(i) := DontCare
  //     regfile.io.wrBen1(i) := Fill(NUM_REGBLOCKS, 0.U)
  //     regfile.io.wrData1(i) := DontCare
  //     regfile.io.wrEn2(i) := false.B
  //     regfile.io.wrAddr2(i) := DontCare
  //     regfile.io.wrBen2(i) := Fill(NUM_REGBLOCKS, 0.U)
  //     regfile.io.wrData2(i) := DontCare
  //   }
  // }


  // thread_finish := branchU.io.finish  // branchU.io.finish as output
  // thread_new_pc := branchU.io.pcOut.asUInt // branchU.io.pcOut as output

  //   /////////////// end extract


class RegisterWriteAndBranchLogic(
    NUM_THREADS: Int,
    NUM_WR: Int,
    NUM_FUS: Int,
    NUM_ALUS: Int,
    NUM_REGS: Int,
    NUM_RF_WR_PORTS: Int,
    NUM_REGBLOCKS: Int,
    NUM_SRC_POS_LG: Int,
    NUM_DST_POS_LG: Int,
    NUM_SRC_MODES_LG: Int,
    REG_WIDTH: Int,
    IP_WIDTH: Int,
    NUM_REGS_LG: Int,
    NUM_DST_MODES_LG: Int,
    NUM_DST_POS: Int,
    WB_DELAY: Int,
    dst_encode: Array[Long],
    dst_pos: Array[Long],
    NUM_WB_ENS: Int,
    dst_en_encode: Array[(Int, Int)],
    wbens: Array[Long]
) extends Module {
    
  class ThreadMemT extends Bundle {
    val brUcodes   = new BRMicrocodes(NUM_ALUS, NUM_FUS)
    val rdWrEn     = Vec(NUM_FUS, Bool())
    val rd         = Vec(NUM_FUS, UInt(NUM_REGS_LG.W))
    val rd_pos     = Vec(NUM_ALUS, UInt(NUM_DST_POS_LG.W))
    val rd_mode    = Vec(NUM_ALUS, UInt(NUM_DST_MODES_LG.W))
  }
    
  val io = IO(new Bundle {
    // Inputs
    val branchThread_vec = Input(Vec(NUM_THREADS, UInt(log2Up(NUM_THREADS + 1).W))) // Branch thread vector  //log2Up(NUM_THREADS + 1).W
    val threadMem_out = Input(new ThreadMemT)
    // val destMems_out = Input(Vec(NUM_WR, new DestMemT))
    val destWbens_wb = Input(Vec(NUM_WR, UInt(NUM_REGBLOCKS.W)))
    val dests_wb = Input(Vec(NUM_WR, UInt(REG_WIDTH.W)))
    val dstPCs = Input(Vec(NUM_FUS + 3 * NUM_ALUS, UInt(32.W)))
    val threadValid = Input(Bool())
    val pc = Input(UInt((IP_WIDTH + 2).W))
    val ioUnit_w_tag = Input(UInt(log2Up(NUM_THREADS).W))               
    val ioUnit_w_wen = Input(Vec(2, Bool()))
    val ioUnit_w_addr = Input(Vec(2, UInt(NUM_REGS_LG.W)))          // val w_addr       = Output(Vec(2, UInt(NUM_REGS_LG.W)))
    val ioUnit_w_data = Input(Vec(2, UInt(REG_WIDTH.W)))               

    // Outputs
    val thread_finish = Output(Bool())
    val thread_new_pc = Output(UInt((IP_WIDTH + 2).W))
    // val branchU_finish = Output(Bool())
    // val branchU_pcOut = Output(UInt((IP_WIDTH + 2).W))
        // val regfile = Module(new RegRead(NUM_THREADS,     NUM_RF_RD_PORTS, NUM_RF_WR_PORTS, NUM_REGS,      REG_WIDTH,  NUM_REGBLOCKS,   REG_BLOCK_WIDTH))
        //                                 (threadnum: Int, num_rd: Int,     num_wr: Int,      num_regs: Int, reg_w: Int, num_blocks: Int, block_widths: Array[Int])
    val regfile_wrEn = Output(Bool())
    val regfile_thread_wr = Output(UInt(log2Up(NUM_THREADS).W))  // (UInt(log2Up(NUM_THREADS).W))
    val regfile_wrEn1 = Output(Vec(NUM_RF_WR_PORTS, Bool()))   
    val regfile_wrAddr1 = Output(Vec(NUM_RF_WR_PORTS, UInt(log2Up(NUM_REGS).W))) // (Vec(NUM_RF_WR_PORTS, UInt(log2Up(NUM_REGS).W)))
    val regfile_wrBen1 = Output(Vec(NUM_RF_WR_PORTS, UInt(NUM_REGBLOCKS.W)))     // (Vec(NUM_RF_WR_PORTS, UInt(NUM_REGBLOCKS.W)))
    val regfile_wrData1 = Output(Vec(NUM_RF_WR_PORTS, UInt(REG_WIDTH.W)))        // (Vec(NUM_RF_WR_PORTS, UInt(REG_WIDTH.W)))
    val regfile_wrEn2 = Output(Vec(NUM_RF_WR_PORTS, Bool())) 
    val regfile_wrAddr2 = Output(Vec(NUM_RF_WR_PORTS, UInt(log2Up(NUM_REGS).W)))
    val regfile_wrBen2 = Output(Vec(NUM_RF_WR_PORTS, UInt(NUM_REGBLOCKS.W)))
    val regfile_wrData2 = Output(Vec(NUM_RF_WR_PORTS, UInt(REG_WIDTH.W)))
  })

  // Local signals
  val dests_wb_s1 = Reg(Vec(NUM_WR, UInt(REG_WIDTH.W)))
  val rdWrEn_wb = RegInit(VecInit(Seq.fill(NUM_FUS)(false.B)))
  val rd_wb = Reg(Vec(NUM_FUS, UInt(log2Up(NUM_THREADS).W)))
  val brUcodes_wb = Wire(new BRMicrocodes(NUM_ALUS, NUM_FUS))

  val branchU = Module(new BranchUnit)

  // Logic assignments
  brUcodes_wb := io.threadMem_out.brUcodes
  rdWrEn_wb := io.threadMem_out.rdWrEn
  rd_wb := io.threadMem_out.rd
  dests_wb_s1 := io.dests_wb

  val threadValid_s1 = RegNext(io.threadValid)
  val threadValid_s2 = RegNext(threadValid_s1)

  val caseFU = (0 until (NUM_FUS + 3 * NUM_ALUS)).map(i => (i.U -> io.dstPCs(i)))
  val branchU_rs1 = Wire(UInt(32.W))
  val branchU_rs2 = Wire(UInt(32.W))
  branchU_rs1 := MuxLookup(brUcodes_wb.rs1, DontCare, caseFU)
  branchU_rs2 := MuxLookup(brUcodes_wb.rs2, DontCare, caseFU)
  branchU.io.brValid := io.threadValid && brUcodes_wb.brValid
  branchU.io.rs1 := branchU_rs1.asSInt
  branchU.io.rs2 := branchU_rs2.asSInt
  branchU.io.brMode := brUcodes_wb.brMode
  branchU.io.pc := io.pc.asSInt
  branchU.io.pcOffset := brUcodes_wb.pcOffset
    
  val scatter = Seq.fill(NUM_ALUS)(Module(new Scatter(REG_WIDTH, NUM_SRC_POS_LG, NUM_SRC_MODES_LG, NUM_REGBLOCKS, NUM_DST_POS, dst_encode, dst_pos, NUM_WB_ENS, dst_en_encode, wbens)))
  for (i <- 0 until NUM_ALUS) {
    scatter(i).io.din := io.dests_wb(i)
    scatter(i).io.shift := io.threadMem_out.rd_pos(i)
    scatter(i).io.mode := io.threadMem_out.rd_mode(i)
  } 

  // Writeback logic
  when (threadValid_s1) {
    io.regfile_wrEn := true.B
    io.regfile_thread_wr := io.branchThread_vec(WB_DELAY-3)
    for (i <- 0 until NUM_RF_WR_PORTS) {
      if (i * 2 < NUM_ALUS) {
        io.regfile_wrEn1(i) := rdWrEn_wb(i * 2)
        io.regfile_wrAddr1(i) := rd_wb(i * 2)
        io.regfile_wrBen1(i) := scatter(i * 2).io.wren
        io.regfile_wrData1(i) := scatter(i * 2).io.dout
      } else {
        io.regfile_wrEn1(i) := rdWrEn_wb(i * 2)
        io.regfile_wrAddr1(i) := rd_wb(i * 2)
        io.regfile_wrBen1(i) := io.destWbens_wb(i * 2)
        io.regfile_wrData1(i) := dests_wb_s1(i * 2)
      }
      if (i * 2 + 1 < NUM_ALUS) {
        io.regfile_wrEn2(i) := rdWrEn_wb(i * 2 + 1)
        io.regfile_wrAddr2(i) := rd_wb(i * 2 + 1)
        io.regfile_wrBen2(i) := scatter(i * 2 + 1).io.wren
        io.regfile_wrData2(i) := scatter(i * 2 + 1).io.dout
      } else if (i * 2 + 1 < NUM_WR) {
        io.regfile_wrEn2(i) := rdWrEn_wb(i * 2 + 1)
        io.regfile_wrAddr2(i) := rd_wb(i * 2 + 1)
        io.regfile_wrBen2(i) := io.destWbens_wb(i * 2 + 1)
        io.regfile_wrData2(i) := dests_wb_s1(i * 2 + 1)
      } else {
        io.regfile_wrEn2(i) := false.B
        io.regfile_wrAddr2(i) := DontCare
        io.regfile_wrBen2(i) := DontCare
        io.regfile_wrData2(i) := DontCare
      }
    }
  }.otherwise {
    io.regfile_wrEn := false.B
    io.regfile_thread_wr := io.ioUnit_w_tag
    io.regfile_wrEn1(0) := io.ioUnit_w_wen(0)
    io.regfile_wrAddr1(0) := io.ioUnit_w_addr(0)
    io.regfile_wrBen1(0) := Fill(NUM_REGBLOCKS, 1.U)
    io.regfile_wrData1(0) := io.ioUnit_w_data(0)(REG_WIDTH - 1, 0)
    io.regfile_wrEn2(0) := io.ioUnit_w_wen(1)
    io.regfile_wrAddr2(0) := io.ioUnit_w_addr(1)
    io.regfile_wrBen2(0) := Fill(NUM_REGBLOCKS, 1.U)
    io.regfile_wrData2(0) := io.ioUnit_w_data(1)(REG_WIDTH - 1, 0)
    for (i <- 1 until NUM_RF_WR_PORTS) {
      io.regfile_wrEn1(i) := false.B
      io.regfile_wrAddr1(i) := DontCare
      io.regfile_wrBen1(i) := Fill(NUM_REGBLOCKS, 0.U)
      io.regfile_wrData1(i) := DontCare
      io.regfile_wrEn2(i) := false.B
      io.regfile_wrAddr2(i) := DontCare
      io.regfile_wrBen2(i) := Fill(NUM_REGBLOCKS, 0.U)
      io.regfile_wrData2(i) := DontCare
    }
  }

  io.thread_finish := branchU.io.finish
  io.thread_new_pc := branchU.io.pcOut.asUInt
}


defined class RegisterWriteAndBranchLogic

### primate_V2

In [31]:
class primate_V2(extCompName: String) extends gComponentLeaf(new input_t, new output_t, ArrayBuffer(), extCompName + "__type__engine__MT__16__") {
    val filename = "./primate.cfg"
    val fileSource = Source.fromFile(filename)
    val lines = fileSource.getLines.toList
    var knobs:Map[String, String] = Map()
    for (line <- lines) {
        val Array(key, value) = line.split("=")
        knobs += (key -> value)
    }
    
    // global parameters
    val NUM_THREADS = knobs.apply("NUM_THREADS").toInt
    val REG_WIDTH = knobs.apply("REG_WIDTH").toInt
    val NUM_REGS = knobs.apply("NUM_REGS").toInt
    val NUM_BFUS = knobs.apply("NUM_BFUS").toInt + 2 // IO and LSU are hidden lol
    val NUM_ALUS = knobs.apply("NUM_ALUS").toInt
    val IMM_WIDTH = knobs.apply("IMM_WIDTH").toInt
    val NUM_REGBLOCKS = knobs.apply("NUM_REGBLOCKS").toInt
    val NUM_SRC_POS = knobs.apply("NUM_SRC_POS").toInt
    val NUM_SRC_MODES = knobs.apply("NUM_SRC_MODES").toInt
    val NUM_DST_POS = knobs.apply("NUM_DST_POS").toInt
    val NUM_WB_ENS = knobs.apply("NUM_WB_ENS").toInt
    val MAX_FIELD_WIDTH = knobs.apply("MAX_FIELD_WIDTH").toInt
    val IP_WIDTH = knobs.apply("IP_WIDTH").toInt
    val REG_BLOCK_WIDTH:Array[Int] = knobs.apply("REG_BLOCK_WIDTH").split(" ").map(_.toInt)
    val src_pos:Array[Int] = knobs.apply("SRC_POS").split(" ").map(_.toInt)
    val src_mode:Array[Int] = knobs.apply("SRC_MODE").split(" ").map(_.toInt)
    val dst_encode:Array[Long] = knobs.apply("DST_ENCODE").split(" ").map(_.toLong)
    val dst_pos:Array[Long] = knobs.apply("DST_POS").split(" ").map(_.toLong)
    val wbens:Array[Long] = knobs.apply("DST_EN").split(" ").map(_.toLong)
    val dst_en_encode:Array[(Int, Int)] = knobs.apply("DST_EN_ENCODE").split(";").map(_.split(" ") match {case Array(a1, a2) => (a1.toInt, a2.toInt)})
    
    val NUM_THREADS_LG = log2Up(NUM_THREADS)
    val NUM_REGS_LG = log2Up(NUM_REGS)
    val NUM_FUOPS_LG = 6 // currently fixed to 6 due to the risc-v encoding
    val NUM_FUS = (NUM_BFUS).max(NUM_ALUS)
    val NUM_FUS_LG = log2Up(NUM_FUS)
    val NUM_RD_BANKS = scala.math.pow(2, log2Up(NUM_FUS)).toInt
    val NUM_RF_RD_PORTS = NUM_FUS
    val NUM_WR = (NUM_BFUS).max(NUM_ALUS)
    val NUM_RF_WR_PORTS = (NUM_WR+1)/2
    val NUM_WR_BANKS = scala.math.pow(2, log2Up(NUM_RF_WR_PORTS)).toInt
    val NUM_SRC_POS_LG = log2Up(NUM_SRC_POS)
    val NUM_SRC_MODES_LG = log2Up(NUM_SRC_MODES)
    val NUM_DST_POS_LG = log2Up(NUM_DST_POS)
    val NUM_DST_MODES_LG = log2Up(NUM_SRC_MODES)
    val INST_RAM_SIZE = 4096
    // FIXME
    val ALU_SRC_WIDTH = NUM_REGS_LG
    val SUBINSTR_WIDTH: Int = (math.ceil((7 + ALU_SRC_WIDTH * 3 + 3 + 7)/8) * 8).toInt
    val INSTR_WIDTH = SUBINSTR_WIDTH * (NUM_FUS + NUM_ALUS*3 + 1)
    
    val NONE_SELECTED = (NUM_THREADS).U((log2Up(NUM_THREADS+1)).W)
    
    val AllOffloadsReady = Reg(Bool())
    val AllOffloadsValid  = Reg(Vec(NUM_THREADS, Bool()))
    
    /////////////////////
    // DUMP THE PARAMS //
    /////////////////////
    println("PRIMATE PARAMS DUMP")
    println(" NUM_THREADS: " + NUM_THREADS)
    println(" REG_WIDTH: " + REG_WIDTH)
    println(" NUM_REGS: " + NUM_REGS)
    println(" NUM_BFUS: " + NUM_BFUS)
    println(" NUM_ALUS: " + NUM_ALUS)
    println(" NUM_FUS : " + NUM_FUS)
    println(" INSTR_WIDTH: " + INSTR_WIDTH)
    println(" SUBINSTR_WIDTH: " + SUBINSTR_WIDTH)
    
    
    // Global visible Units
    val ioUnit = Module(new inOutUnit(REG_WIDTH, NUM_REGS_LG, NUM_FUOPS_LG, NUM_THREADS, IP_WIDTH))
    
    val alu_bfuInst = Array(Module(new alu_bfu0(NUM_THREADS_LG, REG_WIDTH, NUM_FUOPS_LG, NUM_THREADS, IP_WIDTH)),
                          Module(new alu_bfu1(NUM_THREADS_LG, REG_WIDTH, NUM_FUOPS_LG, NUM_THREADS, IP_WIDTH)))  // def functionalUnits = io.elements("off")
    // def dynamicMemPort = functionalUnits.asInstanceOf[Bundle].elements("dynamicMem").asInstanceOf[gOffBundle[dyMemInput_t, llNode_t]]
    // val flowTablePort = Module(new flowTable(TAGWIDTH, NUM_THREADS))

    // output from the alu_bfuInst
    io.mem.mem_addr   := alu_bfuInst(0).io.mem.mem_addr   // ZX: those I/Os are all outputs from the gMemBoudle 
    io.mem.read       := alu_bfuInst(0).io.mem.read       // ZX: So is this assuming those io.mem are the same for each alu_bfu(i) ? 
    io.mem.write      := alu_bfuInst(0).io.mem.write      // ZX: where does io been defined, gInOutOffBundle at the top level?
    io.mem.writedata  := alu_bfuInst(0).io.mem.writedata
    io.mem.byteenable := alu_bfuInst(0).io.mem.byteenable
    // input to the alu_bfuInst
    alu_bfuInst(0).io.mem.waitrequest    := io.mem.waitrequest  // ZX: Those are inputs to the gMemBoudle
    alu_bfuInst(0).io.mem.readdatavalid  := io.mem.readdatavalid
    alu_bfuInst(0).io.mem.readdata       := io.mem.readdata
    for (i <- 1 until NUM_ALUS) {
        alu_bfuInst(i).io.mem.waitrequest   := false.B  // ZX: Taking some default value: always False, why?
        alu_bfuInst(i).io.mem.readdatavalid := false.B
        alu_bfuInst(i).io.mem.readdata      := DontCare
    }
    
    // Gobal unit, init the regfile
    val regfile = Module(new RegRead(NUM_THREADS, NUM_RF_RD_PORTS, NUM_RF_WR_PORTS, NUM_REGS, REG_WIDTH, NUM_REGBLOCKS, REG_BLOCK_WIDTH))
    
    // Gobal unit, init the threadMem and destMems
    class ThreadMemT extends Bundle {
        val brUcodes   = new BRMicrocodes(NUM_ALUS, NUM_FUS)
        val rdWrEn     = Vec(NUM_FUS, Bool())
        val rd         = Vec(NUM_FUS, UInt(NUM_REGS_LG.W))
        val rd_pos     = Vec(NUM_ALUS, UInt(NUM_DST_POS_LG.W))
        val rd_mode    = Vec(NUM_ALUS, UInt(NUM_DST_MODES_LG.W))
    }
    class DestMemT extends Bundle {
        val dstPC      = UInt(32.W)
        val wben       = UInt(NUM_REGBLOCKS.W)
        val dest       = UInt(REG_WIDTH.W)
    }
    val THREAD_MEM_WIDTH = (new ThreadMemT).getWidth
    val threadMem = Module(new ram_simple2port(NUM_THREADS, THREAD_MEM_WIDTH))
    val destMems = Seq.fill(NUM_WR)(Module(new ram_simple2port(NUM_THREADS, (new DestMemT).getWidth)))
    threadMem.io.clock := clock 
    threadMem.io.rden := false.B
    threadMem.io.rdaddress := DontCare
    for (destMem <- destMems) {   
        destMem.io.clock := clock
        destMem.io.wren := false.B
        destMem.io.rden := false.B
        destMem.io.wraddress := DontCare
        destMem.io.rdaddress := DontCare
        destMem.io.data := DontCare
    }

    // global states
    val threadStages = RegInit(VecInit(Seq.fill(NUM_THREADS)(ThreadStageEnum.idle))) 

    val ThreadStateT = new Bundle {         // ZX: controls
        val tag         = UInt((TAGWIDTH*2).W)
        val invalid     = Bool()
        // FIXME: input -> rf & rf -> output
        val ip          = UInt((IP_WIDTH+2).W)
        
        val bfuValids   = Vec(NUM_BFUS, Bool())
        val execValids  = Vec(NUM_BFUS, Bool())
        val io_dstPC    = UInt(IP_WIDTH.W)
        val execDone    = Bool()
        val finish      = Bool()
    }
    val threadStates  = Reg(Vec(NUM_THREADS, ThreadStateT))   // ZX: controls
    // connect the thread to ioUnit
    Range(0, NUM_THREADS, 1).map(i =>  
        ioUnit.io.idle_threads(i) := threadStages(i) === ThreadStageEnum.idle) // idle_threads is bool, and vec
    ioUnit.io.in_valid := io.in.valid
    ioUnit.io.in_tag := io.in.tag
    ioUnit.io.in_data := io.in.bits.data
    ioUnit.io.in_empty := io.in.bits.empty
    ioUnit.io.in_last := io.in.last
    io.in.ready := ioUnit.io.in_ready



    

    /****************** Start Thread *********************************/
    /****************** Scheduler logic *********************************/
    val schedulerLogicBlock = Module(new SchedulerLogic(NUM_THREADS, NUM_RD_BANKS, IP_WIDTH))
    schedulerLogicBlock.io.ioUnit_new_thread := ioUnit.io.new_thread
    schedulerLogicBlock.io.ioUnit_new_tag := ioUnit.io.new_tag
    schedulerLogicBlock.io.threadStages := threadStages
    val vThread = schedulerLogicBlock.io.vThread
    
    
    when (ioUnit.io.new_thread) {
        // spawn new thread
        threadStages(ioUnit.io.new_tag) := ThreadStageEnum.order_fetch 
        threadStates(ioUnit.io.new_tag).ip := 0.U((IP_WIDTH+2).W)           
        println("Primate core spawning new thread!")
    }
    /****************** Fetch logic *********************************/
    val fetchUnit = Module(new Fetch(NUM_THREADS, IP_WIDTH, INSTR_WIDTH, INST_RAM_SIZE))
    val instr = Wire(UInt(INSTR_WIDTH.W))
    fetchUnit.io.ip := (threadStates(vThread).ip)
    instr := fetchUnit.io.instr
    
    when (vThread =/= NONE_SELECTED) {
        threadStages(vThread) := ThreadStageEnum.decode
    }

    /****************** Decode logic *********************************/
    val decodeThread = RegInit(NONE_SELECTED) //assume this declared outside, make this as the input
    decodeThread := vThread
    val decodeThread_d1 = RegNext(decodeThread)  //assume this declared outside, make this as the input

    val decodeLogicBlock = Module(new DecodeLogic(NUM_ALUS, NUM_BFUS, NUM_FUS, INSTR_WIDTH, SUBINSTR_WIDTH, ALU_SRC_WIDTH, NUM_SRC_POS_LG, 
                                                  NUM_SRC_MODES_LG, NUM_DST_POS_LG,  NUM_DST_MODES_LG, NUM_THREADS, NUM_THREADS_LG, NUM_REGS_LG, 
                                                  REG_WIDTH, THREAD_MEM_WIDTH, NUM_RF_RD_PORTS))

    val decodeUnit_bfuValids = Wire(Vec(NUM_BFUS, Bool()))  // !Make sure!
    val aluMicrocodes_in = Wire(new ALUMicrocodes(NUM_ALUS, ALU_SRC_WIDTH, NUM_SRC_POS_LG, NUM_SRC_MODES_LG))
    val bfuMicrocodes_in = Wire(new BFUMicrocodes(NUM_FUS-NUM_ALUS, ALU_SRC_WIDTH))  
    val rdWrEn_in = Wire(Vec(NUM_FUS, Bool()))   // ZX: never used
    val bfuValids_in = Wire(Vec(NUM_BFUS, Bool()))
    val bfuRdRspQ_almFull = Wire(Bool())
    val bfuRdRspQ_enq_in = Wire(Bool())
    
    // Output from the decodeLogicBlock
    decodeUnit_bfuValids := decodeLogicBlock.io.decodeUnit_bfuValids
    aluMicrocodes_in := decodeLogicBlock.io.aluMicrocodes_in
    bfuMicrocodes_in := decodeLogicBlock.io.bfuMicrocodes_in
    rdWrEn_in := decodeLogicBlock.io.rdWrEn_in
    bfuValids_in := decodeLogicBlock.io.bfuValids_in
    bfuRdRspQ_enq_in := decodeLogicBlock.io.bfuRdRspQ_enq_in
    ioUnit.io.rd_req_ready := decodeLogicBlock.io.ioUnit_rd_req_ready
    ioUnit.io.rd_rsp_valid := decodeLogicBlock.io.ioUnit_rd_rsp_valid
    ioUnit.io.rd_rsp := decodeLogicBlock.io.ioUnit_rd_rsp
    threadMem.io.wraddress := decodeLogicBlock.io.threadMem_wraddress
    threadMem.io.wren := decodeLogicBlock.io.threadMem_wren
    threadMem.io.data := decodeLogicBlock.io.threadMem_data
    regfile.io.thread_rd := decodeLogicBlock.io.regfile_thread_rd
    regfile.io.rdEn := decodeLogicBlock.io.regfile_rdEn
    regfile.io.rdAddr1:= decodeLogicBlock.io.regfile_rdAddr1
    regfile.io.rdAddr2:= decodeLogicBlock.io.regfile_rdAddr2

    // inputs to the decodeLogicBlock
    decodeLogicBlock.io.instr := instr
    decodeLogicBlock.io.decodeThread := decodeThread
    decodeLogicBlock.io.decodeThread_d1 := decodeThread_d1
    decodeLogicBlock.io.ioUnit_rd_req_valid := ioUnit.io.rd_req_valid
    decodeLogicBlock.io.ioUnit_rd_req := ioUnit.io.rd_req
    decodeLogicBlock.io.ioUnit_rd_rsp_ready := ioUnit.io.rd_rsp_ready

    when (decodeThread =/= NONE_SELECTED) {
        threadStates(decodeThread).bfuValids := decodeUnit_bfuValids 
        threadStates(decodeThread).invalid := false.B    
        threadStates(decodeThread).execValids := VecInit(Seq.fill(NUM_BFUS)(false.B))
        threadStages(decodeThread) := ThreadStageEnum.read
    } 
    .otherwise {
        threadStates(decodeThread).bfuValids := DontCare
        threadStates(decodeThread).execValids := DontCare
    }

  /***************** Register read / Metadata FIFO enq *************************/
  val REG_DELAY = NUM_RF_RD_PORTS + 1
  val ALU_DELAY = 3 // 3 (gather)

  val readThread_vec = RegInit(VecInit(Seq.fill(REG_DELAY+ALU_DELAY)(NONE_SELECTED)))
  val aluMicrocodes_vec = Reg(Vec(REG_DELAY, new ALUMicrocodes(NUM_ALUS, ALU_SRC_WIDTH, NUM_SRC_POS_LG, NUM_SRC_MODES_LG)))
  val bfuMicrocodes_vec = Reg(Vec(REG_DELAY, new BFUMicrocodes(NUM_FUS-NUM_ALUS, ALU_SRC_WIDTH)))
  val bfuValids_vec = RegInit(VecInit(Seq.fill(REG_DELAY)(VecInit(Seq.fill(NUM_BFUS)(false.B)))))
  val bfuRdRspQ_enq_vec = RegInit(VecInit(Seq.fill(REG_DELAY)(false.B)))

  readThread_vec(REG_DELAY+ALU_DELAY-1) := decodeThread
  aluMicrocodes_vec(REG_DELAY-1) := aluMicrocodes_in
  bfuMicrocodes_vec(REG_DELAY-1) := bfuMicrocodes_in
  bfuValids_vec(REG_DELAY-1) := bfuValids_in
  bfuRdRspQ_enq_vec(REG_DELAY-1) := bfuRdRspQ_enq_in

  for (i <- 0 until REG_DELAY+ALU_DELAY-1) {
    readThread_vec(i) := readThread_vec(i+1)
  }
  for (i <- 0 until REG_DELAY-1) {
    aluMicrocodes_vec(i) := aluMicrocodes_vec(i+1)
    bfuMicrocodes_vec(i) := bfuMicrocodes_vec(i+1)
    bfuValids_vec(i) := bfuValids_vec(i+1)
    bfuRdRspQ_enq_vec(i) := bfuRdRspQ_enq_vec(i+1)
  }

  val srcA = Wire(Vec(NUM_FUS, UInt(REG_WIDTH.W))) // vector containing all the RS1s read from the regfile
  val srcB = Wire(Vec(NUM_FUS, UInt(REG_WIDTH.W))) // vector containing all the RS2s read from the regfile
  srcA := regfile.io.rdData1
  srcB := regfile.io.rdData2  // what is the purpose 

  // moved from Gather stage to here
  val bfuRdRspQ_enq_valid = bfuRdRspQ_enq_vec(0)
  val bfuRdRspQ_enq_bits = Wire(new BFU_regfile_rsp_t(REG_WIDTH))

  // BFU regfile dump rsp queue
  bfuRdRspQ_enq_bits.rdData1 := srcA(0)
  bfuRdRspQ_enq_bits.rdData2 := srcB(0)
  decodeLogicBlock.io.bfuRdRspQ_enq_valid := bfuRdRspQ_enq_valid
  decodeLogicBlock.io.bfuRdRspQ_enq_bits := bfuRdRspQ_enq_bits 


    /****************** Gather Stage 0-2 *************************/
    val gatherThread = readThread_vec(ALU_DELAY) 
    val aluMicrocodes_out = aluMicrocodes_vec(0)  
    val bfuMicrocodes_out = bfuMicrocodes_vec(0)  
    val bfuValids_out = bfuValids_vec(0)          
    
    val gather_srcA = Seq.fill(NUM_ALUS)(Module(new Gather(REG_WIDTH, NUM_SRC_POS, src_pos, MAX_FIELD_WIDTH, NUM_SRC_MODES, src_mode)))
    val gather_srcB = Seq.fill(NUM_ALUS)(Module(new Gather(REG_WIDTH, NUM_SRC_POS, src_pos, MAX_FIELD_WIDTH, NUM_SRC_MODES, src_mode)))  
    for (i <- 0 until NUM_ALUS) {
        gather_srcA(i).io.din   := srcA(i)    // srcA(i) is the input (val srcA = Wire(Vec(NUM_FUS, UInt(REG_WIDTH.W))))
        gather_srcA(i).io.shift := aluMicrocodes_out.rs1_pos(i)
        gather_srcA(i).io.mode  := aluMicrocodes_out.rs1_mode(i)
        gather_srcB(i).io.din   := srcB(i)
        gather_srcB(i).io.shift := aluMicrocodes_out.rs2_pos(i)
        gather_srcB(i).io.mode  := aluMicrocodes_out.rs2_mode(i)
    }
    
    
    // BFU FIFOs
    val execBundle_io = new Bundle {
        val tag = UInt(NUM_THREADS_LG.W)
        val opcode = UInt(NUM_FUOPS_LG.W)
        val rd = UInt(NUM_REGS_LG.W)
        val imm = UInt(12.W)
        val bits = UInt(REG_WIDTH.W)
    }
    
    val fuFifos_iou = Module(new Queue(execBundle_io, NUM_THREADS))
    fuFifos_iou.io.enq.valid := false.B
    fuFifos_iou.io.enq.bits := DontCare
    
    val gatherLogicBlock = Module(new gatherLogic(NUM_THREADS, NUM_BFUS, NUM_ALUS, NUM_FUS, REG_WIDTH, NUM_THREADS_LG, NUM_REGS_LG, 
                                                  NUM_FUOPS_LG, ALU_SRC_WIDTH, NUM_SRC_POS_LG, NUM_SRC_MODES_LG, NONE_SELECTED))
    // inputs
    gatherLogicBlock.io.gatherThread := gatherThread
    gatherLogicBlock.io.aluMicrocodes_out := aluMicrocodes_out
    gatherLogicBlock.io.bfuMicrocodes_out := bfuMicrocodes_out
    gatherLogicBlock.io.bfuValids_out := bfuValids_out
    gatherLogicBlock.io.srcA := srcA

    // Outputs
    fuFifos_iou.io.enq.valid := gatherLogicBlock.io.fuFifos_enq_valid
    fuFifos_iou.io.enq.bits.tag := gatherLogicBlock.io.fuFifos_enq_bits_tag
    fuFifos_iou.io.enq.bits.opcode := gatherLogicBlock.io.fuFifos_enq_bits_opcode
    fuFifos_iou.io.enq.bits.rd := gatherLogicBlock.io.fuFifos_enq_bits_rd
    fuFifos_iou.io.enq.bits.imm := gatherLogicBlock.io.fuFifos_enq_bits_imm
    fuFifos_iou.io.enq.bits.bits:= gatherLogicBlock.io.fuFifos_enq_bits_bits


  /****************** ALU/BFU Stage 0 *************************/
  val preOpThread = readThread_vec(0)
  // ALUBFU FIFOs
  class execBundle_alu extends Bundle {
    val tag     = UInt(NUM_THREADS_LG.W)
    val opcode  = UInt(NUM_FUOPS_LG.W)
    val alu_bfu = Bool()
    val rs1     = UInt(REG_WIDTH.W)
    val rs2     = UInt(32.W)
    val addEn   = Bool()
    val subEn   = Bool()
    val sltEn   = Bool()
    val sltuEn  = Bool()
    val andEn   = Bool()
    val orEn    = Bool()
    val xorEn   = Bool()
    val sllEn   = Bool()
    val srEn    = Bool()
    val srMode  = Bool()
    val luiEn   = Bool()
    val catEn   = Bool()
    val immSel  = Bool()
    val imm     = SInt(32.W)
  }
  val fuFifos_alu = Seq.fill(NUM_ALUS)(Module(new Queue(new execBundle_alu, NUM_THREADS)))

  val aluMicrocodes_out_d0 = RegNext(aluMicrocodes_out)
  val aluMicrocodes_out_d1 = RegNext(aluMicrocodes_out_d0)
  val aluMicrocodes_out_d2 = RegNext(aluMicrocodes_out_d1)

  for (i <- 0 until NUM_ALUS) {
    fuFifos_alu(i).io.enq.valid := false.B
    fuFifos_alu(i).io.enq.bits := DontCare
    when (preOpThread =/= NONE_SELECTED) {
      val fuFifos_alu_in = Wire(new execBundle_alu)
      fuFifos_alu_in.tag     := preOpThread
      fuFifos_alu_in.opcode  := aluMicrocodes_out_d2.opcode(i)
      fuFifos_alu_in.alu_bfu := aluMicrocodes_out_d2.bfu_valid(i)
      fuFifos_alu_in.rs1     := gather_srcA(i).io.dout
      fuFifos_alu_in.rs2     := gather_srcB(i).io.dout
      fuFifos_alu_in.addEn   := aluMicrocodes_out_d2.addEn(i)
      fuFifos_alu_in.subEn   := aluMicrocodes_out_d2.subEn(i)
      fuFifos_alu_in.sltEn   := aluMicrocodes_out_d2.sltEn(i)
      fuFifos_alu_in.sltuEn  := aluMicrocodes_out_d2.sltuEn(i)
      fuFifos_alu_in.andEn   := aluMicrocodes_out_d2.andEn(i)
      fuFifos_alu_in.orEn    := aluMicrocodes_out_d2.orEn(i)
      fuFifos_alu_in.xorEn   := aluMicrocodes_out_d2.xorEn(i)
      fuFifos_alu_in.sllEn   := aluMicrocodes_out_d2.sllEn(i)
      fuFifos_alu_in.srEn    := aluMicrocodes_out_d2.srEn(i)
      fuFifos_alu_in.srMode  := aluMicrocodes_out_d2.srMode(i)
      fuFifos_alu_in.luiEn   := aluMicrocodes_out_d2.luiEn(i)
      fuFifos_alu_in.catEn   := aluMicrocodes_out_d2.catEn(i)
      fuFifos_alu_in.immSel  := aluMicrocodes_out_d2.immSel(i)
      fuFifos_alu_in.imm     := aluMicrocodes_out_d2.imm(i)
      fuFifos_alu(i).io.enq.bits := fuFifos_alu_in
      fuFifos_alu(i).io.enq.valid := true.B
    } .otherwise {
      fuFifos_alu(i).io.enq.bits := DontCare
      fuFifos_alu(i).io.enq.valid := false.B
    }
  }

  /****************** Function unit execution *********************************/  // No need to refactor
  val execThread = RegInit(NONE_SELECTED)  
  execThread := preOpThread
  val execThread_d0 = RegNext(execThread) 
    
  // FUs
  for (i <- 0 until NUM_ALUS) {
    fuFifos_alu(i).io.deq.ready := false.B
    alu_bfuInst(i).io.in_valid := false.B
    val deq = fuFifos_alu(i).io.deq.bits
    alu_bfuInst(i).io.in_alu_bfu := deq.alu_bfu
    alu_bfuInst(i).io.in_tag     := deq.tag
    alu_bfuInst(i).io.in_opcode  := deq.opcode
    alu_bfuInst(i).io.in_rs1     := deq.rs1
    alu_bfuInst(i).io.in_rs2     := deq.rs2
    alu_bfuInst(i).io.in_addEn   := deq.addEn
    alu_bfuInst(i).io.in_subEn   := deq.subEn
    alu_bfuInst(i).io.in_sltEn   := deq.sltEn
    alu_bfuInst(i).io.in_sltuEn  := deq.sltuEn
    alu_bfuInst(i).io.in_andEn   := deq.andEn
    alu_bfuInst(i).io.in_orEn    := deq.orEn
    alu_bfuInst(i).io.in_xorEn   := deq.xorEn
    alu_bfuInst(i).io.in_sllEn   := deq.sllEn
    alu_bfuInst(i).io.in_srEn    := deq.srEn
    alu_bfuInst(i).io.in_srMode  := deq.srMode
    alu_bfuInst(i).io.in_luiEn   := deq.luiEn
    alu_bfuInst(i).io.in_catEn   := deq.catEn
    alu_bfuInst(i).io.in_immSel  := deq.immSel
    alu_bfuInst(i).io.in_imm     := deq.imm
    when (fuFifos_alu(i).io.deq.valid) {
      alu_bfuInst(i).io.in_valid := true.B
      when (alu_bfuInst(i).io.in_ready) {
        fuFifos_alu(i).io.deq.ready := true.B
      }
    }
  }

  fuFifos_iou.io.deq.ready := false.B
  when (fuFifos_iou.io.deq.valid) {
    val deq = fuFifos_iou.io.deq
    ioUnit.io.ar_valid := true.B
    ioUnit.io.ar_tag := deq.bits.tag
    ioUnit.io.ar_opcode := deq.bits.opcode
    ioUnit.io.ar_rd := deq.bits.rd
    ioUnit.io.ar_bits := deq.bits.bits
    ioUnit.io.ar_imm := deq.bits.imm
    when (ioUnit.io.ar_ready) {
      fuFifos_iou.io.deq.ready := true.B
    }
  } .otherwise {
    ioUnit.io.ar_valid := false.B
    ioUnit.io.ar_tag := DontCare
    ioUnit.io.ar_opcode := DontCare
    ioUnit.io.ar_rd := DontCare
    ioUnit.io.ar_bits := DontCare
    ioUnit.io.ar_imm := DontCare
  }


  when (execThread_d0 =/= NONE_SELECTED) {
    threadStages(execThread_d0) := ThreadStageEnum.exec
  }

  // FUs output
  for (i <- 0 until NUM_ALUS) {
    alu_bfuInst(i).io.out_ready := true.B
    when (alu_bfuInst(i).io.out_valid) {
      val destMem_in = Wire(new DestMemT)
      destMem_in.dstPC := alu_bfuInst(i).io.out_flag
      destMem_in.dest := alu_bfuInst(i).io.out_bits
      destMem_in.wben := Fill(NUM_REGBLOCKS, 1.U)
      destMems(i).io.wren := true.B
      destMems(i).io.wraddress := alu_bfuInst(i).io.out_tag
      destMems(i).io.data := destMem_in.asUInt
      threadStates(alu_bfuInst(i).io.out_tag).execValids(i) := true.B
    }
  }

  when (ioUnit.io.w_valid) {
    threadStates(ioUnit.io.w_tag).execValids(NUM_BFUS-1) := true.B
    threadStates(ioUnit.io.w_tag).io_dstPC := ioUnit.io.w_flag
  }
  when (ioUnit.io.r_valid) {
    threadStates(ioUnit.io.r_tag).execValids(NUM_BFUS-1) := true.B
    threadStates(ioUnit.io.r_tag).io_dstPC := ioUnit.io.r_flag
  }


    /****************** Finish execution *********************************/
    val finishExecutionLogicBlock = Module(new FinishExecutionLogic(NUM_THREADS, NUM_WR, NUM_WR_BANKS, NUM_BFUS, NONE_SELECTED))

    // Provide inputs to the extracted module
    finishExecutionLogicBlock.io.threadStates_execValids := threadStates.map(_.execValids)
    finishExecutionLogicBlock.io.threadStates_bfuValids := threadStates.map(_.bfuValids)
    finishExecutionLogicBlock.io.threadStages := threadStages
    
    // Extract outputs from the module
    val fThread = finishExecutionLogicBlock.io.fThread
    threadMem.io.rden := finishExecutionLogicBlock.io.threadMem_rden
    threadMem.io.rdaddress := finishExecutionLogicBlock.io.threadMem_rdaddress
    
    for (i <- 0 until NUM_WR) {
        destMems(i).io.rden := finishExecutionLogicBlock.io.destMems_rden(i)
        destMems(i).io.rdaddress := finishExecutionLogicBlock.io.destMems_rdaddress(i)
    }

    when (fThread =/= NONE_SELECTED) {
        threadStages(fThread) := ThreadStageEnum.branch
    }


  /****************** Register write & branch *********************************/
  val WB_DELAY = NUM_RF_WR_PORTS.max(3)
  val branchThread_out = RegInit(NONE_SELECTED)   // where does this go
  val branchThread_vec = RegInit(VecInit(Seq.fill(WB_DELAY)(NONE_SELECTED)))
  branchThread_out := branchThread_vec(0)
  // val branchU = Module(new BranchUnit)  // local module, define it in the submodule

  branchThread_vec(WB_DELAY-1) := fThread
  for (i <- 0 until WB_DELAY-1) {
    branchThread_vec(i) := branchThread_vec(i+1)   // Ater update the value make branchThread_vec as an input
  }
    
  //// excluded and remain on the top level, start (this part is assign values to the input)
  val threadMem_out = Wire(new ThreadMemT)  // defined at the top level, but also a input
  val destMems_out = Wire(Vec(NUM_WR, (new DestMemT))) // execluded from module, live at top level
  val destWbens_wb = RegInit(VecInit(Seq.fill(NUM_WR)(0.U(NUM_REGBLOCKS.W)))) // input
  val dests_wb = Wire(Vec(NUM_WR, UInt(REG_WIDTH.W)))  // input
  val dstPCs = Wire(Vec(NUM_FUS+3*NUM_ALUS, UInt(32.W)))  // input

  
  dstPCs := DontCare
  if (NUM_ALUS < NUM_FUS) {
    // The last unit is the IO unit
    for (i <- 0 until NUM_FUS-1) {
      destMems_out(i) := destMems(i).io.q.asTypeOf(new DestMemT) 
      dests_wb(i) := destMems_out(i).dest
      destWbens_wb(i) := destMems_out(i).wben
    }
    for (i <- 0 until NUM_ALUS) {
      dstPCs(4*i+2) := destMems_out(i).dstPC
    }
    for (i <- 0 until NUM_FUS-NUM_ALUS-1) {
      dstPCs(4*NUM_ALUS+i) := destMems_out(NUM_ALUS+i).dstPC
    }
    dstPCs(NUM_FUS+3*NUM_ALUS-1) := threadStates(branchThread_vec(WB_DELAY-2)).io_dstPC
  } else {
    // The IO unit is combined with an ALU unit
    for (i <- 0 until NUM_FUS) {
      destMems_out(i) := destMems(i).io.q.asTypeOf(new DestMemT)
      dests_wb(i) := destMems_out(i).dest
      destWbens_wb(i) := destMems_out(i).wben
    }
    for (i <- 0 until NUM_ALUS) {
      dstPCs(4*i+2) := destMems_out(i).dstPC
    }
    dstPCs(4*(NUM_BFUS-1)+2) := threadStates(branchThread_vec(WB_DELAY-2)).io_dstPC
  }
    
  threadMem_out := threadMem.io.q.asTypeOf(chiselTypeOf(threadMem_out)) 

  val pc = Reg(UInt((IP_WIDTH+2).W))  
  val threadValid = RegInit(false.B)  

  when (branchThread_vec(WB_DELAY-1) =/= NONE_SELECTED) {
    threadValid := true.B
    pc := threadStates(branchThread_vec(WB_DELAY-1)).ip
  } .otherwise {
    threadValid := false.B
    pc := DontCare
  }
  ///// execluded end
  val threadValid_s1 = RegNext(threadValid)
  val threadValid_s2 = RegNext(threadValid_s1)

  val wbBranchBlock = Module(new RegisterWriteAndBranchLogic(NUM_THREADS,NUM_WR, NUM_FUS, NUM_ALUS, NUM_REGS, NUM_RF_WR_PORTS, NUM_REGBLOCKS, NUM_SRC_POS_LG, 
                                                             NUM_DST_POS_LG, NUM_SRC_MODES_LG, REG_WIDTH, IP_WIDTH, NUM_REGS_LG, NUM_DST_MODES_LG, NUM_DST_POS, 
                                                             WB_DELAY, dst_encode, dst_pos, NUM_WB_ENS, dst_en_encode, wbens))
  
    wbBranchBlock.io.branchThread_vec := branchThread_vec
    wbBranchBlock.io.threadMem_out := threadMem_out
    wbBranchBlock.io.destWbens_wb := destWbens_wb
    wbBranchBlock.io.dests_wb := dests_wb
    wbBranchBlock.io.dstPCs := dstPCs
    wbBranchBlock.io.threadValid := threadValid
    wbBranchBlock.io.pc := pc
    wbBranchBlock.io.ioUnit_w_tag := ioUnit.io.w_tag
    wbBranchBlock.io.ioUnit_w_wen := ioUnit.io.w_wen
    wbBranchBlock.io.ioUnit_w_addr := ioUnit.io.w_addr
    wbBranchBlock.io.ioUnit_w_data := ioUnit.io.w_data

    val thread_finish = Wire(Bool())
    val thread_new_pc = Wire(UInt((IP_WIDTH+2).W))
    
    thread_finish := wbBranchBlock.io.thread_finish
    thread_new_pc:= wbBranchBlock.io.thread_new_pc
    regfile.io.wrEn := wbBranchBlock.io.regfile_wrEn
    regfile.io.thread_wr := wbBranchBlock.io.regfile_thread_wr
    regfile.io.wrEn1 := wbBranchBlock.io.regfile_wrEn1
    regfile.io.wrAddr1 := wbBranchBlock.io.regfile_wrAddr1
    regfile.io.wrBen1 := wbBranchBlock.io.regfile_wrBen1
    regfile.io.wrData1 := wbBranchBlock.io.regfile_wrData1
    regfile.io.wrEn2 := wbBranchBlock.io.regfile_wrEn2
    regfile.io.wrAddr2 := wbBranchBlock.io.regfile_wrAddr2
    regfile.io.wrBen2 := wbBranchBlock.io.regfile_wrBen2
    regfile.io.wrData2 := wbBranchBlock.io.regfile_wrData2
    
  when (threadValid_s1) {
    threadStates(branchThread_vec(WB_DELAY-3)).ip := thread_new_pc
    when (thread_finish) {
      threadStages(branchThread_vec(WB_DELAY-3)) := ThreadStageEnum.idle
    } .otherwise {
      threadStages(branchThread_vec(WB_DELAY-3)) := ThreadStageEnum.fetch
    }
  } .elsewhen (branchThread_vec(WB_DELAY-3) =/= NONE_SELECTED) {
    threadStages(branchThread_vec(WB_DELAY-3)) := ThreadStageEnum.fetch
  }


  // Output
  io.out.valid := ioUnit.io.out_valid
  io.out.tag := ioUnit.io.out_tag
  io.out.bits.data := ioUnit.io.out_data
  io.out.bits.empty := ioUnit.io.out_empty
  io.out.last := ioUnit.io.out_last
  ioUnit.io.out_ready := io.out.ready

  // Input unit output ready
  val w_tag = ioUnit.io.w_tag
  val lastThreadWr = RegNext(branchThread_vec(WB_DELAY-3))
  ioUnit.io.w_ready := (!threadValid_s1) && ((!threadValid_s2) || (w_tag(log2Up(NUM_RF_WR_PORTS)-1, 0) =/= lastThreadWr(log2Up(NUM_RF_WR_PORTS)-1, 0)))

}


defined class primate_V2